# Research data supporting "Reliable Insights into Surface Chemistry Enabled by Combining Experiments with Fast and Accurate Theory"

This notebook accompanies our paper: **Reliable Insights into Surface Chemistry Enabled by Combining Experiments with Fast and Accurate Theory**. It can be found on GitHub at https://github.com/benshi97/Data_Atomistic_Insights and explored interactively on [Colab](https://colab.research.google.com/github/benshi97/Data_Atomistic_Insights/blob/main/analyse.ipynb).

### Abstract

<!-- The adsorption energy of a molecule onto the surface of a material underpins a wide array of applications, spanning heterogeneous catalysis, gas storage and many more. It is the key quantity where experimental measurements and theoretical calculations meet, with agreement being necessary for reliable predictions of reaction rates and mechanisms. The prototypical molecule-surface system is CO adsorbed on MgO, but despite intense scrutiny from theory and experiment, there is still no consensus on its adsorption energy. In particular, the large cost of accurate many-body methods makes reaching converged theoretical estimates difficult, generating a wide range of values. In this work, we address this challenge, leveraging the latest advances in diffusion Monte Carlo (DMC) and coupled cluster theory [CCSD(T)], to obtain accurate predictions for CO on MgO. These reliable theoretical estimates allow us to evaluate the inconsistencies in published temperature programmed desorption experiments, revealing that they arise from variations in employed pre-exponential factors. Using this insight, we derive new experimental estimates of the (electronic) adsorption energy from a more precise pre-exponential factor. With this effort, we are able to reach consensus between multiple theoretical calculations and multiple experiments for the first time. In addition, we show that our recently developed cluster-based CCSD(T) approach provides a low cost route towards achieving accurate adsorption energies. This sets the stage for affordable and reliable theoretical predictions of reaction mechanisms and rates to guide the realization of new catalysts and gas storage materials. -->


## Contents
* [Table S1 and Figure 1 - Past CO on MgO *E*<sub>ads</sub> and Current Work](#tables1fig1)
* [Table S2 and S3 - Validating the revPBE-D4 geometry and Computing its $\Delta_\textrm{geom}$](#tables2s3)
* [Table S4 and S5 - Periodic CCSD(T) Convergence](#tables4)
* [Table S6 - Periodic DMC Convergence](#tables6)
* [Table S7 - DFT Convergence](#tables7)
* [Table S9 to S11 and Figure 2 - SKZCAM Cluster CCSD(T) Convergence](#tables9-s11)
* [Table S8 - Final periodic CCSD(T), periodic DMC and cluster CCSD(T) $E_\textrm{ads}$ and their individual contributions](#tables8)
* [Table S12 - Analysis of previous computational work on CO on MgO](#tables12)
* [Table S13 - Computation of zero-point energy and thermal contribution terms to convert $H_\textrm{ads}$ to $E_\textrm{ads}$](#tables13)
* [Figure 3 and S1 - Converting previous experiment $H_\textrm{ads}$ or $E_\textrm{act}$ (for TPD) to $E_\textrm{ads}$](#fig3)
* [Table S14 - Final best estimate of TPD experimental adsorption energies](#tables14)
* [Table S15 - Effect of CO coverage on $E_\textrm{ads}$](#tables15)


In [1]:
# Check if we are in Google Colab environment
try:
    import google.colab
    IN_COLAB = True
    usetex = False
    texfalse_import()

except:
    import os
    IN_COLAB = False
    if os.path.expanduser('~') == '/home/shixubenjamin':
        usetex = True
        textrue_import()
    else:
        usetex = False    
        texfalse_import()


# If in Google Colab, install the necessary data and set up the necessary environment
if IN_COLAB == True:
    !rm -rf /content/Data_CO_on_MgO-main /content/main.zip
    !wget https://github.com/benshi97/Data_CO_on_MgO/archive/refs/heads/main.zip
    !unzip /content/main.zip
    ! apt install ase
    !pip install pyblock datetime
    %pwd
    %cd /content/Data_CO_on_MgO-main

In [2]:
from ase import io
from ase.units import mol, kcal, kJ, Hartree
import pandas as pd
import pyblock
from Scripts.jup_plot import *
if usetex == True:
    textrue_import()
else:
    texfalse_import()

from Scripts.cluster_scripts import *
import Scripts.extrapolate as extrapolate

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle


overall_systems = {
    'MgO': ['CH4','C2H6','CO','N2O_1','N2O_2','C6H6','H2O','NH3','CO2_2','CO2_3','NO_1','NO_2','NO_3','NO_4','NO_5','NO_6','CH3OH_1','CH3OH_2','CH3OH_4','H2O_4','CH4_ML','C2H6_ML'],
    'TiO2_rutile': ['CH4','CH3OH','CO2_1','CO2_2','H2O'],
    'TiO2_anatase': ['H2O','NH3']
}

all_system_labels = [f'{molecule} {surface}' for surface in overall_systems for molecule in overall_systems[surface]]

overall_systems_num_monomers = {y: {x: 1 for x in overall_systems[y]} for y in overall_systems}
overall_systems_num_monomers['MgO']['H2O_4'] = 4
overall_systems_num_monomers['MgO']['CH3OH_4'] = 4
overall_systems_num_monomers['MgO']['CH4_ML'] = 4
overall_systems_num_monomers['MgO']['C2H6_ML'] = 4
overall_systems_num_monomers['MgO']['NO_6'] = 2
overall_systems_num_monomers['MgO']['CH4_ML'] = 4
overall_systems_num_monomers['MgO']['C2H6_ML'] = 4



large_systems_mgo = ['C6H6','H2O_4','CH3OH_4']
cluster_systems_mgo = ['NO_6','H2O_4','CH3OH_4']



# # Initialize the dictionary of final adsorption energies and the individual contributions to each adsorption energy
# ene_final = np.load("Data/Misc/methods_eads.npy", allow_pickle="TRUE").item()

# # Get the best estimate of the experimental adsorption energies
# best_expt_eads = np.loadtxt("Data/Misc/best_expt.txt")

# List of DFT functionals used in this study
dft_functionals = {
    'MgO': ["01_PBE-D2-Ne",
    "02_revPBE-D4",
    "03_vdW-DF",
    "04_rev-vdW-DF2",
    "05_PBE0-D4",
    "06_B3LYP-D2-Ne"],
    'TiO2_rutile': ["01_PBE-U-TSHI",
    "02_revPBE-D4",
    "03_vdW-DF",
    "04_rev-vdW-DF2",
    "05_R2SCAN-rVV10",
    "06_HSE06-D4"],
    'TiO2_anatase': ["01_PBE-U-TSHI",
    "02_revPBE-D4",
    "03_vdW-DF",
    "04_rev-vdW-DF2",
    "05_R2SCAN-rVV10",
    "06_HSE06-D4"]
}

dft_functionals_conversion = {
    '01_PBE-D2-Ne': 'PBE-D2[Ne]',
    '02_revPBE-D4': 'revPBE-D4',
    '03_vdW-DF': 'vdW-DF',
    '04_rev-vdW-DF2': 'rev-vdW-DF2',
    '05_PBE0-D4': 'PBE0-D4',
    '06_B3LYP-D2-Ne': 'B3LYP-D2[Ne]',
    '01_PBE-U-TSHI': 'PBE+U-TS/HI',
    '05_R2SCAN-rVV10': r'r$^2$SCAN-rVV10',
    '06_HSE06-D4': 'HSE06-D4'
}

# A dictionary which converts the DFT functional, surface and adsorbate dictionary labels to LaTeX formatted labels
convert_to_nice_labels = {
    'xc_functionals': {
        '01_PBE-D2-Ne': 'PBE-D2[Ne]',
        '02_revPBE-D4': 'revPBE-D4',
        '03_vdW-DF': 'vdW-DF',
        '04_rev-vdW-DF2': 'rev-vdW-DF2',
        '05_PBE0-D4': 'PBE0-D4',
        '06_B3LYP-D2-Ne': 'B3LYP-D2[Ne]',
        '01_PBE-U-TSHI': 'PBE+U-TS/HI',
        '05_R2SCAN-rVV10': r'r$^2$SCAN-rVV10',
        '06_HSE06-D4': 'HSE06-D4'
    },
    'surface': {
        'MgO': 'MgO (001)',
        'TiO2_rutile': r'\ce{TiO2} rutile (110)',
        'TiO2_anatase': r'\ce{TiO2} anatase (101)'
    },
    'adsorbate': {
        'MgO CH4': r'\ce{CH4}',
        'MgO C2H6': r'\ce{C2H6}',
        'MgO CO': r'\ce{CO}',
        'MgO C6H6': r'\ce{C6H6}',
        'MgO H2O': r'\ce{H2O}',
        'MgO NH3': r'\ce{NH3}',
        'MgO NO_1': r'Vertical-Hollow \ce{NO}',
        'MgO NO_2': r'Vertical-Mg \ce{NO}',
        'MgO NO_3': r'Bent-Bridge \ce{NO}',
        'MgO NO_4': r'Bent-Mg \ce{NO}',
        'MgO NO_5': r'Bent-O \ce{NO}',
        'MgO NO_6': r'Dimer \ce{NO}',
        'MgO N2O_1': r'Horizontal \ce{N2O}',
        'MgO N2O_2': r'Vertical \ce{N2O}',
        'MgO CO2_2': r'Physisorbed \ce{CO2}',
        'MgO CO2_3': r'Chemisorbed \ce{CO2}',
        'MgO CH3OH_1': r'Monomer \ce{CH3OH}',
        'MgO CH3OH_2': r'Dimer \ce{CH3OH}',
        'MgO CH3OH_4': r'Tetramer \ce{CH3OH}',
        'MgO CH4_ML': r'Monolayer \ce{CH4}',
        'MgO C2H6_ML': r'Monolayer \ce{C2H6}',
        'MgO H2O_4': r'Tetramer \ce{H2O}',
        'TiO2_rutile CH4': r'\ce{CH4}',
        'TiO2_rutile CH3OH': r'\ce{CH3OH}',
        'TiO2_rutile CO2_1': r'Parallel \ce{CO2}',
        'TiO2_rutile CO2_2': r'Tilted \ce{CO2}',
        'TiO2_rutile H2O': r'\ce{H2O}',
        'TiO2_anatase H2O': r'\ce{H2O}',
        'TiO2_anatase NH3': r'\ce{NH3}'
    }
}

# Some basic conversion factors
kcalmol_to_meV = kcal / mol * 1000
kjmol_to_meV = kJ / mol * 1000
mha_to_meV = Hartree

In [3]:
def convert_df_to_latex_input(
    df,
    start_input = '\\begin{table}\n',
    end_input = '\n\\end{table}',
    label = "tab:default",
    caption = "This is a table",
    replace_input = {},
    df_latex_skip = 0,
    adjustbox = 0,
    scalebox = False,
    multiindex_sep = "",
    filename = "./table.tex",
    index = True,
    column_format = None
):
    if column_format is None:
        column_format = "l" + "r" * len(df.columns)
    
    if label != "":
        label_input = r"\label{" + label + r"}"
    else:
        label_input = ""
    caption_input = r"\caption{" + label_input + caption +  "}"

    with pd.option_context("max_colwidth", 1000):
        df_latex_input = df.to_latex(escape=False, column_format=column_format,multicolumn_format='c', multicolumn=True,index=index)
    for key in replace_input:
        df_latex_input = df_latex_input.replace(key, replace_input[key])
    
    df_latex_input_lines = df_latex_input.splitlines()[df_latex_skip:]
    # Get index of line with midrule
    toprule_index = [i for i, line in enumerate(df_latex_input_lines) if "toprule" in line][0]
    df_latex_input_lines[toprule_index+1] = df_latex_input_lines[toprule_index+1] + ' ' + multiindex_sep
    df_latex_input = '\n'.join(df_latex_input_lines)

    with open(filename, "w") as f:
        f.write(start_input + "\n")
        f.write(caption_input + "\n")
        if adjustbox > 0:
            f.write(r"\adjustbox{max width=" + f"{adjustbox}" + r"\textwidth}" + "{\n")
        if scalebox:
            f.write(r"\begin{adjustbox}{scale=" + f"{scalebox}" + "}" + "\n")
        f.write(df_latex_input)
        if adjustbox:
            f.write("\n}")
        f.write("\n" + end_input)
        

In [4]:
# Plot the TPD curves

# Pandas DataFrame containing the TPD data
tpd_schematic_df = pd.read_csv("Data/TPD_schematic_data.csv")

# Make the header a row then set new header  'Temperature (K)' followed by 0.1 to 1.0 in 0.1 increments
tpd_schematic_df.columns = ['Temperature (K)'] + [f'{i:.1f}' for i in np.arange(0.1, 1.1, 0.1)]

# Plot each of the columns with the first column as x and the rest as y
fig, axs = plt.subplots(figsize=(3.365, 2.5),dpi=450,constrained_layout=True)
for column in tpd_schematic_df.columns[1:]:
    axs.plot(tpd_schematic_df['Temperature (K)'], tpd_schematic_df[column], label=r'$\theta{=}$' + column)

axs.legend(loc='upper left',fontsize=8)
axs.set_xlabel('Temperature (K)')
axs.set_ylabel(r'Desorption rate ${-}d\theta/dt$ (ML/s)')

axs.set_xlim([55,76])
axs.set_yticks([])
axs.set_xticks([70])
axs.set_xticklabels([r'T$_{p}$'])

plt.savefig('Fig_SI_TPD_schematic.png')


In [5]:
# Effects of layer convergence and other factors on the adsorption energy for Table S7

# Create a dictionary 'eads_dft_conv' to store different parameters and their corresponding adsorption energy calculations
dft_parameters_dict = {
    'MgO': {
        '(meta)GGA': {
            'Supercell Size': '4x4' ,
            'Number of Layers': '4(2)' ,
            'K-point Mesh': '2x2x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Mg_pv (4e core)',
        },
        'hybrid': {
            'Supercell Size': '4x4' ,
            'Number of Layers': '4(2)' ,
            'K-point Mesh': '2x2x1(2x2x1)',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Mg_pv (4e core)',
        }
    },
    'TiO2_rutile': {
        '(meta)GGA': {
            'Supercell Size': 'p(4x2)' ,
            'Number of Layers': '5(3) O-Ti-O' ,
            'K-point Mesh': '2x2x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Ti_pv (12e core)',
        },
        'hybrid': {
            'Supercell Size': 'p(4x2)' ,
            'Number of Layers': '5(2) O-Ti-O' ,
            'K-point Mesh': '2x2x1(1x1x1)',
            'Energy cutoff': '520',
            'Vacuum': '13',
            'Metal PAW potential': 'Ti (18e core)',
        }
    },
    'TiO2_anatase': {
        '(meta)GGA': {
            'Supercell Size': '3x1' ,
            'Number of Layers': '8(2) O-Ti-O layers',
            'K-point Mesh': '3x3x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Ti_pv (12e core)',
        },
        'hybrid': {
            'Supercell Size': '3x1' ,
            'Number of Layers': '8(2) O-Ti-O' ,
            'K-point Mesh': '3x3x1(1x1x1)',
            'Energy cutoff': '520',
            'Vacuum': '13',
            'Metal PAW potential': 'Ti (18e core)',
        }
    }
}

dft_parameters_df = pd.DataFrame.from_dict(
    {(convert_to_nice_labels['surface'][surface], xc_func): parameters
     for surface, xc_func_dict in dft_parameters_dict.items()
     for xc_func, parameters in xc_func_dict.items()},
    orient='index'
).transpose()

# start_latex_input

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_parameters_df,
    start_input = r'\begin{table}',
    label = 'tab:dft_parameters',
    caption = 'DFT parameters used for the three different surfaces. The parameters for hybrids are also different from those for the metaGGA, GGA and vdW-inclusive functionals, grouped as (meta)GGA in the table. The number of layers in parentheses indicates the number of layers fixed at the bottom of the slab. The k-points in parenthesis indicates the k-point mesh used for the exact exchange potential',
    end_input = r'\end{table}',
    replace_input = {
        '_': r'\_',
        'x': r'${\times}$',
    },
    adjustbox = 1,
    df_latex_skip = 0,
    multiindex_sep = r'\cmidrule(lr){2-3}  \cmidrule(lr){4-5} \cmidrule(lr){6-7}',
    filename = 'Tables/dft_parameters.tex',
    column_format = 'l' + 'r'*len(dft_parameters_df.columns)
)
# dft_parameters_df.to_latex('Tables/dft_parameters.tex', escape=False, column_format='l' + 'r'*len(dft_parameters_df.columns))
# Create a DataFrame 'df' from the 'eads_dft_conv' dictionary
# df = pd.DataFrame(eads_dft_conv).T

# # Round the 'Eads' and 'Diff' columns to integers
# df['Eads'] = df['Eads'].apply(lambda x: round(x))
# df['Diff'] = df['Diff'].apply(lambda x: round(x))

# # Display the final DataFrame 'df'
# df

In [6]:
dft_parameters_df

MgO (001)                  \ce{TiO2} rutile (110)  \
                           (meta)GGA           hybrid              (meta)GGA   
Supercell Size                   4x4              4x4                 p(4x2)   
Number of Layers                4(2)             4(2)            5(3) O-Ti-O   
K-point Mesh                   2x2x1     2x2x1(2x2x1)                  2x2x1   
Energy cutoff                    600              600                    600   
Vacuum                            15               15                     15   
Metal PAW potential  Mg_pv (4e core)  Mg_pv (4e core)       Ti_pv (12e core)   

                                   \ce{TiO2} anatase (101)                 
                            hybrid               (meta)GGA         hybrid  
Supercell Size              p(4x2)                     3x1            3x1  
Number of Layers       5(2) O-Ti-O      8(2) O-Ti-O layers    8(2) O-Ti-O  
K-point Mesh          2x2x1(1x1x1)                   3x3x1   3x3x1(1x1x1)  
Energy cutoff                  520                     600            520  
Vacuum                          13                      15             13  
Metal PAW potential  Ti (18e core)        Ti_pv (12e core)  Ti (18e core)

In [7]:
dft_parameters_dict

{'MgO': {'(meta)GGA': {'Supercell Size': '4x4',
   'Number of Layers': '4(2)',
   'K-point Mesh': '2x2x1',
   'Energy cutoff': '600',
   'Vacuum': '15',
   'Metal PAW potential': 'Mg_pv (4e core)'},
  'hybrid': {'Supercell Size': '4x4',
   'Number of Layers': '4(2)',
   'K-point Mesh': '2x2x1(2x2x1)',
   'Energy cutoff': '600',
   'Vacuum': '15',
   'Metal PAW potential': 'Mg_pv (4e core)'}},
 'TiO2_rutile': {'(meta)GGA': {'Supercell Size': 'p(4x2)',
   'Number of Layers': '5(3) O-Ti-O',
   'K-point Mesh': '2x2x1',
   'Energy cutoff': '600',
   'Vacuum': '15',
   'Metal PAW potential': 'Ti_pv (12e core)'},
  'hybrid': {'Supercell Size': 'p(4x2)',
   'Number of Layers': '5(2) O-Ti-O',
   'K-point Mesh': '2x2x1(1x1x1)',
   'Energy cutoff': '520',
   'Vacuum': '13',
   'Metal PAW potential': 'Ti (18e core)'}},
 'TiO2_anatase': {'(meta)GGA': {'Supercell Size': '3x1',
   'Number of Layers': '8(2) O-Ti-O layers',
   'K-point Mesh': '3x3x1',
   'Energy cutoff': '600',
   'Vacuum': '15',
   'M

In [8]:
list(dft_functionals_conversion.values())

['PBE-D2[Ne]',
 'revPBE-D4',
 'vdW-DF',
 'rev-vdW-DF2',
 'PBE0-D4',
 'B3LYP-D2[Ne]',
 'PBE+U-TS/HI',
 'r$^2$SCAN-rVV10',
 'HSE06-D4']

In [9]:
# Getting the lattice parameters for Table S2

# Initialize an empty list to store lattice parameters for each DFT functional
lattice_parameter_xc_dict = {xc_func: {'MgO a': 0, 'MgO Error': 0,  'TiO2_rutile a': 0, 'TiO2_rutile c': 0, 'TiO2_rutile RMSE': 0, 'TiO2_anatase a': 0, 'TiO2_anatase c': 0, 'TiO2_anatase RMSE': 0} for xc_func in ['Experiment'] + list(dft_functionals_conversion.values())}

lattice_parameter_xc_dict['Experiment'] = {'MgO a': 4.216, 'MgO Error': 0, 'TiO2_rutile a': 4.587, 'TiO2_rutile c': 2.954, 'TiO2_rutile RMSE': 0, 'TiO2_anatase a': 3.782, 'TiO2_anatase c': 9.502, 'TiO2_anatase RMSE': 0} 

# Loop through each DFT functional
for surface in ['MgO', 'TiO2_rutile', 'TiO2_anatase']:
    for xc_func in dft_functionals[surface]:
        # Read the CONTCAR file to obtain lattice parameter 'a'
        a = io.read(f'Data/DFT/00-UC/{surface}/{xc_func}/CONTCAR')
        # Add the lattice parameter 'a' to the latpar_functional list
        lattice_parameter_xc_dict[dft_functionals_conversion[xc_func]][f'{surface} a'] = a.get_cell()[0][0]
        if 'TiO2' in surface:
            lattice_parameter_xc_dict[dft_functionals_conversion[xc_func]][f'{surface} c'] = a.get_cell()[2][2]
            lattice_parameter_xc_dict[dft_functionals_conversion[xc_func]][f'{surface} RMSE'] = np.sqrt(np.sum((np.array([lattice_parameter_xc_dict['Experiment'][f'{surface} a'], lattice_parameter_xc_dict['Experiment'][f'{surface} c']]) - np.array([a.get_cell()[0][0], a.get_cell()[2][2]]))**2))
        else:
            lattice_parameter_xc_dict[dft_functionals_conversion[xc_func]][f'{surface} Error'] = np.abs(lattice_parameter_xc_dict['Experiment'][f'{surface} a'] - a.get_cell()[0][0])
        
# Create a DataFrame from the lattice_parameter_xc_dict
lattice_parameter_xc_df = pd.DataFrame(lattice_parameter_xc_dict).T
lattice_parameter_xc_df = lattice_parameter_xc_df.round(3)

lattice_parameter_xc_df.columns = ['$a$', 'Error', '$a$', '$c$', 'RMSE', '$a$', '$c$', 'RMSE']

lattice_parameter_xc_latex_input = lattice_parameter_xc_df.to_latex(escape=False)
lattice_parameter_xc_latex_input = lattice_parameter_xc_latex_input.replace('&  0.000', '&      -')


lattice_parameter_xc_latex_input = "\\begin{table}\n\\caption{\\label{table:lattice_parameters} Lattice parameters for MgO, TiO$_2$ rutile, and TiO$_2$ anatase obtained from DFT calculations and experiment.}\n\\begin{adjustbox}{center}\n" + r"""\begin{tabular}{lrrrrrrrr}
\toprule
\multirow{2}{*}{Functional} & \multicolumn{2}{c}{MgO} & \multicolumn{3}{c}{Rutile TiO\textsubscript{2}}     & \multicolumn{3}{c}{Anatase TiO$_2$} \\       \cmidrule(lr){2-3}  \cmidrule(lr){4-6} \cmidrule(lr){7-9}
""" + '\n'.join(lattice_parameter_xc_latex_input.splitlines()[2:]) + "\\end{adjustbox}\n\\end{table}"


with open('Tables/lattice_parameters.tex', 'w') as f:
    f.write(lattice_parameter_xc_latex_input)


In [10]:
lattice_parameter_xc_df

,$a$,Error,$a$,$c$,RMSE,$a$,$c$,RMSE
Experiment,4.216,0.000,4.587,2.954,0.000,3.782,9.502,0.000
PBE-D2[Ne],4.234,0.018,0.000,0.000,0.000,0.000,0.000,0.000
revPBE-D4,4.220,0.004,4.598,2.958,0.011,3.790,9.549,0.048
vdW-DF,4.273,0.057,4.685,2.995,0.107,3.839,9.767,0.272
rev-vdW-DF2,4.220,0.004,4.618,2.961,0.032,3.798,9.593,0.093
PBE0-D4,4.175,0.041,0.000,0.000,0.000,0.000,0.000,0.000
B3LYP-D2[Ne],4.202,0.014,0.000,0.000,0.000,0.000,0.000,0.000
PBE+U-TS/HI,0.000,0.000,4.647,3.026,0.094,3.859,9.676,0.190
r$^2$SCAN-rVV10,0.000,0.000,4.590,2.957,0.004,3.786,9.531,0.029
HSE06-D4,0.000,0.000,4.575,2.940,0.018,3.769,9.518,0.020


# DFT Analysis

In [11]:
# Getting the DFT adsorption energies for all of the MgO surfaces

eint_dict = {x1:{x:{ y: {z:0 for z in dft_functionals[x1]} for y in dft_functionals[x1]} for x in overall_systems[x1]} for x1 in ['MgO','TiO2_rutile','TiO2_anatase']}
erel_dict = {x1:{x: {z:0 for z in dft_functionals[x1]} for x in overall_systems[x1]} for x1 in ['MgO','TiO2_rutile','TiO2_anatase']}
final_eads_dict = {x1: {x: {z:0 for z in dft_functionals[x1]} for x in overall_systems[x1]} for x1 in ['MgO','TiO2_rutile','TiO2_anatase']}
clus_eint_dict = {x: { y: {z: 0 for z in dft_functionals['MgO'] } for y in dft_functionals['MgO']} for x in ['NO_6','CH3OH_4','H2O_4']}
ml_eint_dict = {x: { y: {z: 0 for z in dft_functionals['MgO'] } for y in dft_functionals['MgO']} for x in ['CH4_ML','C2H6_ML']}

for system in ['MgO','TiO2_rutile','TiO2_anatase']:
    for i in overall_systems[system]:
        molecule = i.split('_')[0]
        if i == 'NO_6':
            molecule = 'N2O2'
        for j in dft_functionals[system]:

            # Calculating the interacion energy for the X functional using the geometry from Y functional
            for k in dft_functionals[system]:

                if '_ML' in i:
                    ad_slab_ene = np.array([0.01]*overall_systems_num_monomers[system][i])
                    slab_ene = np.array([0.01]*overall_systems_num_monomers[system][i])
                    ad_ene = np.array([0.01]*overall_systems_num_monomers[system][i])
                    for k1 in range(overall_systems_num_monomers[system][i]):
                        ad_slab_ene[k1] = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/{k1+1}_AD_SLAB/OUTCAR', code_format='vasp')
                        slab_ene[k1] = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/ALL_SLAB_FS/OUTCAR', code_format='vasp')
                        ad_ene[k1] = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/{k1+1}_AD_FS/OUTCAR', code_format='vasp')
                    
                    dummy_eint_ene = ad_slab_ene - slab_ene - ad_ene
                    # print(np.sum(dummy_eint_ene)*1000/overall_systems_num_monomers[system][i])
                    
                    eint_dict[system][i][j][k] = np.sum(dummy_eint_ene)*1000/overall_systems_num_monomers[system][i]
                    all_ad_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/ALL_AD_FS/OUTCAR', code_format='vasp')
                    ml_eint_dict[i][j][k] = ((all_ad_ene) - np.sum(ad_ene))*1000/overall_systems_num_monomers[system][i]
                    # all_ad_slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/ALL_AD_SLAB/OUTCAR', code_format='vasp') 
                    # all_ad_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/ALL_AD_FS/OUTCAR', code_format='vasp')
                    # all_slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/ALL_SLAB_FS/OUTCAR', code_format='vasp')
                    # ml_eint_dict[i][j][k] = ((all_ad_slab_ene - all_slab_ene - all_ad_ene) - np.sum(dummy_eint_ene))*1000/overall_systems_num_monomers[system][i]

                elif (system == 'MgO') and (i in ['CH3OH_4','H2O_4']):
                    ad_slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/AD_SLAB/OUTCAR', code_format='vasp')
                    slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/SLAB_FS/OUTCAR', code_format='vasp')
                    ad_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/AD_FS/OUTCAR', code_format='vasp')
                    eint_dict[system][i][j][k] = (ad_slab_ene - slab_ene - ad_ene)*1000/overall_systems_num_monomers[system][i]

                    # Calculating the interaction energy of the cluster
                    clus_ene = ad_ene
                    for k1 in range(overall_systems_num_monomers[system][i]):
                        ad_fs_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/AD_FS_{k1+1}/OUTCAR', code_format='vasp')
                        clus_ene = clus_ene - ad_fs_ene
                    clus_eint_dict[i][j][k] = clus_ene*1000/overall_systems_num_monomers[system][i]
                elif (system == 'MgO') and (i == 'NO_6'):
                    ad_slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/AD_SLAB/OUTCAR', code_format='vasp')
                    slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/SLAB_FS/OUTCAR', code_format='vasp')
                    ad_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/AD_FS/OUTCAR', code_format='vasp')
                    eint_dict[system][i][j][k] = (ad_slab_ene - slab_ene - ad_ene)*1000/overall_systems_num_monomers[system][i]                    
                else:
                    ad_slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/AD_SLAB/OUTCAR', code_format='vasp')
                    slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/SLAB_FS/OUTCAR', code_format='vasp')
                    ad_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{k}/AD_FS/OUTCAR', code_format='vasp')
                    eint_dict[system][i][j][k] = (ad_slab_ene - slab_ene - overall_systems_num_monomers[system][i]*ad_ene)*1000/overall_systems_num_monomers[system][i]

            # Calculating the final adsorption energy and the relaxation energy for a functional
            ad_slab_ene = find_energy(f'Data/DFT/03-Mol_Surf/{system}/{i}/{j}/OUTCAR', code_format='vasp')

            if system == 'MgO' and i == 'NO_6':
                ad_ene = find_energy(f'Data/DFT/01-Mol/{system}/{molecule}/{j}/OUTCAR', code_format='vasp')/2
            elif system == 'MgO':
                ad_ene = find_energy(f'Data/DFT/01-Mol/{system}/{molecule}/{j}/OUTCAR', code_format='vasp')
            elif 'TiO2' in system:
                ad_ene = find_energy(f'Data/DFT/01-Mol/TiO2/{molecule}/{j}/OUTCAR', code_format='vasp')

            if (system == 'MgO') and (i in large_systems_mgo):
                ad_slab_ene = find_energy(f'Data/DFT/04-Int_Ene/{system}/{i}/{j}/{j}/AD_SLAB/OUTCAR', code_format='vasp')
                slab_ene = find_energy(f'Data/DFT/02-Surf/{system}/SLAB_LARGE_PV/SP_pv/{j}/OUTCAR', code_format='vasp')
                ad_ene = find_energy(f'Data/DFT/01-Mol/{system}/{i}/{j}/OUTCAR', code_format='vasp')
            elif system == 'MgO': 
                slab_ene = find_energy(f'Data/DFT/02-Surf/{system}/{j}/{j}/OUTCAR', code_format='vasp')
            elif 'TiO2' in system:
                slab_ene = find_energy(f'Data/DFT/02-Surf/{system}/{j}/OUTCAR', code_format='vasp')

            final_eads_dict[system][i][j] = (ad_slab_ene - slab_ene - overall_systems_num_monomers[system][i]*ad_ene)*1000/overall_systems_num_monomers[system][i]

            if (system == 'MgO') and (i in cluster_systems_mgo):
                erel_dict[system][i][j] = final_eads_dict[system][i][j] - ( eint_dict[system][i][j][j] + clus_eint_dict[i][j][j])
            elif (system == 'MgO') and ('_ML' in i):
                erel_dict[system][i][j] = final_eads_dict[system][i][j] - ( eint_dict[system][i][j][j] + ml_eint_dict[i][j][j])
            else:
                erel_dict[system][i][j] = final_eads_dict[system][i][j] - eint_dict[system][i][j][j]



In [12]:
erlx_error_heatmap = {}
mad_erlx_error_heatmap = {}

for mol_surf in ['CO MgO', 'H2O MgO', 'CH4 TiO2_rutile', 'NH3 TiO2_anatase']:
    molecule = mol_surf.split()[0]
    surface = mol_surf.split()[1]

    erlx_error = np.zeros(np.array([6,6])+1)
    max_erlx_error = np.zeros(np.array([6,6])+1)

    for idxj, j in enumerate(dft_functionals[surface]):
        for idxk, k in enumerate(dft_functionals[surface]):
            eads_approx = eint_dict[surface][molecule][j][k] + erel_dict[surface][molecule][k]
            erlx_error[idxj,idxk] = eads_approx - final_eads_dict[surface][molecule][j]
        
    for idxj, j in enumerate(dft_functionals[surface]):
        max_erlx_error[idxj,-1] = np.mean(np.abs(erlx_error[idxj,:-1]))
        max_erlx_error[-1,idxj] = np.mean(np.abs(erlx_error[:-1,idxj]))
            
    erlx_error_heatmap[mol_surf] = np.round(erlx_error,1)
    mad_erlx_error_heatmap[mol_surf] = np.round(max_erlx_error,1)

fig, axs = plt.subplots(2,2,figsize=(8,9.5),dpi=450,constrained_layout=True)
plt.set_cmap('bwr')

system_nice_list = ['CO on MgO (001)', r'H$_2$O on MgO (001)', r'CH$_4$ on TiO$_2$ rutile (110)', r'NH$_3$ on TiO$_2$ anatase (101)']

for index,i in enumerate(['CO MgO', 'H2O MgO', 'CH4 TiO2_rutile', 'NH3 TiO2_anatase']):
    

    divmodi = divmod(index,2)

    if 'MgO' in i:
        xc_nice_list = ['PBE-D2[Ne]','revPBE-D4','vdW-DF','rev-vdW-DF2','PBE0-D4','B3LYP-D2[Ne]','MAD']
    else:
        xc_nice_list = ['PBE-U-TSHI','revPBE-D4','vdW-DF','rev-vdW-DF2',r'r$^2$SCAN-rVV10','HSE06-D4','MAD']

    xc_nice_list_vals = []
    counter = 0
    for indexk, k in  enumerate(dft_functionals[i.split()[1]]):
        if divmodi[1] == 0:
            xc_nice_list_vals += ['{0} ({1:.0f} meV)'.format(xc_nice_list[indexk],final_eads_dict[i.split()[1]][i.split()[0]][k])]
            counter += 1 
        else:
            xc_nice_list_vals += ['({0:.0f} meV)'.format(final_eads_dict[i.split()[1]][i.split()[0]][k])]
            counter += 1 


    data_values = erlx_error_heatmap[i]
    xc_nice_list_vals += ['MAD']

    im = axs[divmodi[0],divmodi[1]].imshow(data_values, vmin=-200,vmax=200)

    # Define custom colormap that transitions from invisible to black
    colors = [(1, 1, 1, 0), (0, 0, 0, 0.7)]  # RGBA tuples: transparent to black
    n_bins = 100  # Discretize into 100 steps
    cmap_custom = LinearSegmentedColormap.from_list('invisible_to_black', colors, N=n_bins)
    # Second color bar for custom colormap
    # cax2 = divider.append_axes("right", size="2%", pad=0.55)  # Increased pad value to shift it further to the right
    im_custom = axs[divmodi[0],divmodi[1]].imshow(mad_erlx_error_heatmap[i], cmap=cmap_custom, alpha=1,vmin=0,vmax=150)  # Use alpha to make it invisible
    # cbar_custom = plt.colorbar(im_custom, cax=cax2)
    # cbar_custom.ax.set_ylabel('Mean absolute deviation', rotation=-90, va="bottom",fontsize=7)
    # cbar_custom.ax.tick_params(labelsize=7)  # Decrease y-tick label size for the second color bar

# # Show all ticks and label them with the respective list entries
    axs[divmodi[0],divmodi[1]].set_xticks(np.arange(len(xc_nice_list)), labels=xc_nice_list)
    axs[divmodi[0],divmodi[1]].set_yticks(np.arange(len(xc_nice_list)), labels=xc_nice_list_vals)

    for i12 in range(len(xc_nice_list)):
        for j in range(len(xc_nice_list)):
            if j == len(xc_nice_list)-1 and i12 == len(xc_nice_list)-1:
                continue
            elif j == len(xc_nice_list)-1 or i12 == len(xc_nice_list)-1:
                text = axs[divmodi[0],divmodi[1]].text(j, i12, mad_erlx_error_heatmap[i][i12,j],
                        ha="center", va="center", color="k")
            else:
                text = axs[divmodi[0],divmodi[1]].text(j, i12, data_values[i12, j],
                        ha="center", va="center", color="k")
    axs[divmodi[0],divmodi[1]].set_title(system_nice_list[index])

    # cbar = axs[divmodi[0],divmodi[1]].figure.colorbar(im, ax=axs[divmodi[0],divmodi[1]],fraction=0.046, pad=0.04)
    # cbar.ax.set_ylabel(r'$E_\textrm{ads}^\textrm{rel.}[\textrm{XC1||XC2}]$ - $E_\textrm{ads}^\textrm{rel.}[\textrm{XC1||XC1}]$', rotation=-90, va="bottom")



    axs[divmodi[0],divmodi[1]].set_xticklabels(xc_nice_list, rotation=45, ha="right",rotation_mode="anchor")
    if divmodi[0] == 1:
        axs[divmodi[0],divmodi[1]].set_xlabel(r'XC Functional (XC2) for generating \textbf{geometries}')


axs[0,0].set_ylabel(r'XC Functional (XC1) for \textbf{calculating} $E_\textrm{ads}^\textrm{approx}$')
axs[1,0].set_ylabel(r'XC Functional (XC1) for \textbf{calculating} $E_\textrm{ads}^\textrm{approx}$')


cbar = fig.colorbar(im, ax=axs, orientation='horizontal', fraction=0.02)
cbar.ax.set_xlabel(r'$E_\textrm{ads}^\textrm{approx}[\textrm{XC1//XC2}]$ - $E_\textrm{ads}^\textrm{True}[\textrm{XC1//XC1}]$ (meV)')

cbar_custom = fig.colorbar(im_custom, ax=axs, orientation='horizontal', fraction=0.02)
cbar_custom.ax.set_xlabel('Mean Absolute Deviation (meV)')

plt.savefig('Fig_SI_Relaxation_Error.png')



In [13]:
# Getting the 

dft_xc_eint_dict = {x1: {z:50 for z in ['01_PBE-D30', '02_PBE-MBDFI', '03_rev-vdW-DF2', '04_R2SCAN-rVV10', '05_PBE0-TSHI','06_HSE06-D4','07_RPA','08_RPA-rSE']} for x1 in ['MgO CH4', 'MgO C2H6', 'MgO CO', 'MgO CO2_2', 'MgO CO2_3', 'MgO H2O','MgO N2O_1','MgO N2O_2', 'MgO NH3','TiO2_rutile CH4','TiO2_rutile CH3OH','TiO2_rutile CO2_1','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']}

for mol_surface in ['MgO CH4', 'MgO C2H6', 'MgO CO', 'MgO CO2_2', 'MgO CO2_3', 'MgO H2O','MgO N2O_1','MgO N2O_2', 'MgO NH3','TiO2_rutile CH4','TiO2_rutile CH3OH','TiO2_rutile CO2_1','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']:
    molecule = mol_surface.split()[-1]
    surface  = mol_surface.split()[0]
    for functional in ['01_PBE-D30', '02_PBE-MBDFI', '03_rev-vdW-DF2', '04_R2SCAN-rVV10', '05_PBE0-TSHI','06_HSE06-D4']:
        # if '06' in functional: 
        #     continue

        # if '02' in functional and molecule == 'CO':
        #     continue
        try:
            ad_slab_ene = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/{functional}/AD_SLAB/OUTCAR', code_format='vasp')
            slab_ene = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/{functional}/SLAB_FS/OUTCAR', code_format='vasp')
            ad_ene = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/{functional}/AD_FS/OUTCAR', code_format='vasp')

            if '06' in functional:
                ad_slab_ene_d4 = np.loadtxt(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/{functional}/AD_SLAB/EDISP_D4') 
                slab_ene_d4 = np.loadtxt(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/{functional}/SLAB_FS/EDISP_D4') 
                ad_ene_d4 = np.loadtxt(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/{functional}/AD_FS/EDISP_D4') 
                dft_xc_eint_dict[mol_surface][functional] = (ad_slab_ene - slab_ene - ad_ene)*1000 + (ad_slab_ene_d4 - slab_ene_d4 - ad_ene_d4)*1000*Hartree
            else:
                dft_xc_eint_dict[mol_surface][functional] = (ad_slab_ene - slab_ene - ad_ene)*1000

        except:
            dft_xc_eint_dict[mol_surface][functional] = 0

    # Do RPA analysis if MgO

    if surface == 'MgO' and molecule in ['CH4','C2H6','CO','CO2_2','CO2_3','H2O','N2O_1','NH3']:
        ad_slab_ene_exx = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/AD_SLAB/OUT.EXX.run.encut.550.k.2', code_format='vasp')
        slab_ene_exx = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/SLAB_FS/OUT.EXX.run.encut.550.k.2', code_format='vasp')
        ad_ene_exx = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/AD_FS/OUT.EXX.run.encut.550.k.2', code_format='vasp')

        ad_slab_ene_exx_rse = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/AD_SLAB/OUT.rSE.run.encut.550.k.2', code_format='vasp')
        slab_ene_exx_rse = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/SLAB_FS/OUT.rSE.run.encut.550.k.2', code_format='vasp')
        ad_ene_exx_rse = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/AD_FS/OUT.rSE.run.encut.550.k.2', code_format='vasp')


        ad_slab_ene_rpac = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/AD_SLAB/OUT.RPA.run.encut.550.k.2', code_format='vasp_rpa')
        slab_ene_rpac = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/SLAB_FS/OUT.RPA.run.encut.550.k.2', code_format='vasp_rpa')
        ad_ene_rpac = find_energy(f'Data/DFT/04-Int_Ene/DFT_Compare/{surface}/{molecule}/07-RPA/AD_FS/OUT.RPA.run.encut.550.k.2', code_format='vasp_rpa')



        dft_xc_eint_dict[mol_surface]['07_RPA'] = (ad_slab_ene_exx - slab_ene_exx - ad_ene_exx)*1000 + (ad_slab_ene_rpac - slab_ene_rpac - ad_ene_rpac)*1000


        dft_xc_eint_dict[mol_surface]['08_RPA-rSE'] = (ad_slab_ene_exx_rse - slab_ene_exx_rse - ad_ene_exx_rse)*1000 + (ad_slab_ene_rpac - slab_ene_rpac - ad_ene_rpac)*1000


In [14]:
erel_error_dict = {x1:{x: {z:0 for z in dft_functionals[x1]} for x in overall_systems[x1]} for x1 in ['MgO','TiO2_rutile','TiO2_anatase']}

final_surface_molecule_labels = [f'{x1}_{x}' for x1 in ['MgO','TiO2_rutile','TiO2_anatase'] for x in overall_systems[x1]]

adsorbate_config = {
    'MgO':{'NO_1': 'Vertical-Hollow',
    'NO_2': 'Vertical-Mg',
    'NO_3': 'Bent-Bridge',
    'NO_4': 'Bent-Mg',
    'NO_5': 'Bent-O',
    'NO_6': 'Dimer',
    'N2O_1': 'Horizontal',
    'N2O_2': 'Vertical',
    'CO2_2': 'Physisorbed',
    'CO2_3': 'Chemisorbed',
    'CH3OH_1': 'Monomer',
    'CH3OH_2': 'Dimer',
    'CH3OH_4': 'Tetramer',
    'CH4_ML': 'Monolayer',
    'C2H6_ML': 'Monolayer',
    'H2O_4': 'Tetramer'},
    'TiO2_rutile': {
        'CO2_1': 'Parallel',
        'CO2_2': 'Tilted'
    },
    'TiO2_anatase': {}
}

final_surface_nice_molecule_labels = []

erel_df_dict = {f'{x}': {} for x in final_surface_molecule_labels}

for system in ['MgO','TiO2_rutile','TiO2_anatase']:
    if 'MgO' in i:
        xc_nice_list = ['PBE-D2[Ne]','revPBE-D4','vdW-DF','rev-vdW-DF2','PBE0-D4','B3LYP-D2[Ne]']
    else:
        xc_nice_list = ['PBE-U-TSHI','revPBE-D4','vdW-DF','rev-vdW-DF2',r'r$^2$SCAN-rVV10','HSE06-D4']

    # Getting the crystal system and molecule name
    if 'TiO2_rutile' in system:
        system_label = r'\ce{TiO2} rutile (110)'
    elif 'TiO2_anatase' in system:
        system_label = r'\ce{TiO2} anatase (101)'
    elif 'MgO' in system:
        system_label = r'\ce{MgO} (001)'


    for i in overall_systems[system]:
        molecule_label = r'\ce{' + i.split('_')[0] + '}'

        if i in adsorbate_config[system]:
            final_surface_nice_molecule_labels += [r"\rotatebox{90}{" + f"{adsorbate_config[system][i]} {molecule_label} on {system_label}" + r"}"]
        else:
            final_surface_nice_molecule_labels += [r"\rotatebox{90}{" + f"{molecule_label} on {system_label}" + r"}"]
            # cc_eint_df_dict[f'{surface}_{molecule}'][quantity + 'Error'] = final_cc_eint_list[surface][molecule][quantity][1]
        
        # This loop goes over the various DFT functionals as geometries
        for idxj, j in enumerate(dft_functionals[system]):
            # Get the adsorption energy and compare against the final adsorption energy
            cumulative_err = []
            # Calculating the interacion energy for the X functional using the geometry from Y functional
            for idxk, k in enumerate(dft_functionals[system]):
                true_ads_ene = final_eads_dict[system][i][k]

                if (system == 'MgO') and (i in cluster_systems_mgo):
                    dummy_ads_ene = eint_dict[system][i][k][j] + erel_dict[system][i][j] + clus_eint_dict[i][k][j]
                elif (system == 'MgO') and ('_ML'in i):
                    dummy_ads_ene = eint_dict[system][i][k][j] + erel_dict[system][i][j] + ml_eint_dict[i][k][j]
                else:
                    dummy_ads_ene = eint_dict[system][i][k][j] + erel_dict[system][i][j]

                if j == '02_revPBE-D4':
                    erel_df_dict[f'{system}_{i}'][r'$E_\textrm{ads}^\textrm{approx}$' + f' [{xc_nice_list[idxk]}' + r'//revPBE-D4]'] = dummy_ads_ene
                    erel_df_dict[f'{system}_{i}'][r'$E_\textrm{ads}^\textrm{true}$' + f' [{xc_nice_list[idxk]}]'] = true_ads_ene
                    erel_df_dict[f'{system}_{i}'][r'$E_\textrm{ads}^\textrm{approx}$' + f' [{xc_nice_list[idxk]}' + r'//revPBE-D4] Error'] = dummy_ads_ene - true_ads_ene

                cumulative_err += [(true_ads_ene - dummy_ads_ene)**2]


            # print(system,i,j,np.sqrt(cumulative_err))

            erel_error_dict[system][i][j] = 2*np.sqrt(np.sum(cumulative_err))/5
            if j == '02_revPBE-D4':
                erel_df_dict[f'{system}_{i}'][r'$E_\textrm{rlx}$ [revPBE-D4]'] = erel_dict[system][i][j]
                erel_df_dict[f'{system}_{i}'][f'Error'] = 2*np.sqrt(np.sum(cumulative_err))/5


In [15]:
mgo_erel_df_dict = {x: erel_df_dict[x] for x in erel_df_dict if 'MgO' in x}
tio2_erel_df_dict = {x: erel_df_dict[x] for x in erel_df_dict if 'TiO2' in x}



df = pd.DataFrame.from_dict(mgo_erel_df_dict)
df = df.round(1)
df.columns = [x for x in final_surface_nice_molecule_labels if 'MgO' in x]
# df.index = final_surface_molecule_nice_labels

with pd.option_context("max_colwidth", 1000):
    mgo_erlx_latex_input = r"\begin{turnpage}" + """\n\\begin{table}[]\n\\adjustbox{max width=1.5\\textwidth}{%\n""" + df.to_latex(escape=False) + "}\n\caption{\label{table:erlx_mgo}" + """$E_\\textrm{rlx}$ for the 6 functionals for the MgO (001) surface} """ + "\n\\end{table}\n\\end{turnpage}"

with open('Data/mgo_erlx.tex','w') as f:
    f.write(mgo_erlx_latex_input)


df = pd.DataFrame.from_dict(tio2_erel_df_dict)
df = df.round(1)
df.columns = [x for x in final_surface_nice_molecule_labels if 'TiO2' in x]
# df.index = final_surface_molecule_nice_labels

with pd.option_context("max_colwidth", 1000):
    tio2_erlx_latex_input = """\\begin{table}[]\n\\adjustbox{max width=1.5\\textwidth}{%\n""" + df.to_latex(escape=False) + "}\n\caption{\label{table:erlx_tio2}" + """$E_\\textrm{rlx}$ for the 6 functionals for the \ce{TiO2} rutile (110) and anatase (101) surfaces} """ + "\n\\end{table}"

with open('Data/tio2_erlx.tex','w') as f:
    f.write(tio2_erlx_latex_input)
# final_eint_str = final_eint_str.replace('&    0.0 ±  0.0','&    -')

# Thermal Corrections

In [16]:
# Getting the vibrational energies for these systems

ethermal = {x1: { x: {y: {z: 0  for z in ['ZPE', 'Eth', 'RT', 'Ecor']} for y in dft_functionals[x1][:4]} for x in overall_systems[x1]} for x1 in ['MgO','TiO2_rutile','TiO2_anatase']}

ethermal_df_dict = {f'{x}': {} for x in final_surface_molecule_labels}


# Define a list experimental parameters with temperature and degrees of freedom values of the molecule
mol_temp_and_dof_dict = {
    'MgO': { # [ Temperature, degrees of freedom = 3N-6 generalor 3N-5 if linear ] 
    'CO': [61,1], #Temperature given as average between studies by Dohnalek et al. and Wichtendahl et al.
    'H2O': [100,3],
    'CH3OH': [296,12],
    'CO2_2': [130,4],
    'CO2_3': [230,4],
    'NO': [84,1],
    'N2O2': [84,3*4-6],
    'CH4' : [47, 3*5-6],
    'C2H6': [75, 3*8-6],
    'C6H6': [175,3*12-6],
    'H2O_4': [235,3*3*4 - 6], 
    'CH3OH_4': [296,3*6*4 -6],
    'NH3': [120,3*4-6],
    'N2O': [77,3*3-5],
    'CH4_ML' : [47, 3*5*4-6],
    'C2H6_ML' : [75, 3*8*4-6]},
    'TiO2_rutile': {
        'CH4': [85,3*5-6],
        'H2O': [303, 3*3-6],
        'CO2': [177,3*3-5],
        'CH3OH': [370,3*6-6]
    },
    'TiO2_anatase': {
        'NH3': [360,3*4-6],
        'H2O': [257, 3]
    }
}


# Initialize a nested dictionary 'thermal_ene_contributions' to store various thermal energy contributions for the 6 studied DFT functional
# Iterate over each DFT functional

for system in ['MgO','TiO2_rutile','TiO2_anatase']:
    if 'MgO' in i:
        xc_nice_list = ['PBE-D2[Ne]','revPBE-D4','vdW-DF','rev-vdW-DF2','PBE0-D4','B3LYP-D2[Ne]']
    else:
        xc_nice_list = ['PBE-U-TSHI','revPBE-D4','vdW-DF','rev-vdW-DF2',r'r$^2$SCAN-rVV10','HSE06-D4']

    for idxi, i in enumerate(overall_systems[system]):
        if system == 'MgO' and 'CO2' in i:
            molecule = i
        else:
            molecule = i.split('_')[0]

        ecor_list = []
        for idxj, j in enumerate(dft_functionals[system][:4]):    # Read vibrational frequencies from OUTCAR files
            # print(system,i,j)
            a, b = read_vib_freq(f'Data/DFT/05-Vib/{system}/{i}/{j}/AD_SLAB/OUTCAR'.format(i)) # Adsorbate on Slab
            c, d = read_vib_freq(f'Data/DFT/05-Vib/{system}/{i}/{j}/AD/OUTCAR'.format(i)) # AD
            # Calculate thermal and zero-point corrections as well as RT term for adsorbate and adsorbate on slab
            ene_ad_slab, eth_ad_slab, zpe_ad_slab, kT = get_quasi_rrho(a, b, mol_temp_and_dof_dict[system][molecule][0])
            ene_ad, eth_ad, zpe_ad, kT = get_quasi_rrho(c, d, mol_temp_and_dof_dict[system][molecule][0])

            if i == 'CO2_3':
                e, f = read_vib_freq(f'Data/DFT/05-Vib/{system}/{i}/{j}/SLAB/OUTCAR'.format(i)) 
                ene_slab, eth_slab, zpe_slab, kT = get_quasi_rrho(e, f, mol_temp_and_dof_dict[system][molecule][0])
                # Store thermal energy contributions in the nested dictionary 'thermal_ene_contributions'
                ethermal[system][i][j]['ZPE'] = zpe_ad_slab - zpe_ad - zpe_slab
                ethermal[system][i][j]['Eth'] = eth_ad_slab - eth_ad - eth_slab
                ethermal[system][i][j]['Ecor'] = ene_ad_slab - ene_ad - ene_slab - kT
                ethermal[system][i][j]['RT'] = kT
            elif i in ['H2O_4','CH3OH_4','NO_6','CH4_ML','C2H6_ML']:
                # Store thermal energy contributions in the nested dictionary 'thermal_ene_contributions'
                ethermal[system][i][j]['ZPE'] = (zpe_ad_slab - zpe_ad*overall_systems_num_monomers[system][i])/overall_systems_num_monomers[system][i]
                ethermal[system][i][j]['Eth'] = (eth_ad_slab - eth_ad*overall_systems_num_monomers[system][i])/overall_systems_num_monomers[system][i]
                ethermal[system][i][j]['Ecor'] = (ene_ad_slab - ene_ad*overall_systems_num_monomers[system][i])/overall_systems_num_monomers[system][i] - kT
                ethermal[system][i][j]['RT'] = kT       
            else:
                # Store thermal energy contributions in the nested dictionary 'thermal_ene_contributions'
                ethermal[system][i][j]['ZPE'] = zpe_ad_slab - zpe_ad
                ethermal[system][i][j]['Eth'] = eth_ad_slab - eth_ad
                ethermal[system][i][j]['Ecor'] = ene_ad_slab - ene_ad - kT
                ethermal[system][i][j]['RT'] = kT

            ecor_list += [ethermal[system][i][j]['Ecor']]
            if idxj == 0:
                ethermal_df_dict[f'{system}_{i}'][f'Temperature'] = mol_temp_and_dof_dict[system][molecule][0]
                ethermal_df_dict[f'{system}_{i}'][f'kT'] =  kT

            ethermal_df_dict[f'{system}_{i}'][f'ZPE [XC {idxj+1}]'] = ethermal[system][i][j]['ZPE']
            ethermal_df_dict[f'{system}_{i}'][f'Eth [XC {idxj+1}]'] = ethermal[system][i][j]['Eth']
            ethermal_df_dict[f'{system}_{i}'][f'Ecor [XC {idxj+1}]'] = ethermal[system][i][j]['Ecor']
        ethermal_df_dict[f'{system}_{i}'][f'Final Ecor'] = np.mean(ecor_list)
        ethermal_df_dict[f'{system}_{i}'][f'Ecor error'] = np.std(ecor_list)



In [17]:
ethermal['MgO']['NO_6']

{'CH4': {'01_PBE-D2-Ne': {'ZPE': 28.91694126886432,
   'Eth': -5.210135728329217,
   'RT': 4.0501452584920905,
   'Ecor': 19.656660282042697},
  '02_revPBE-D4': {'ZPE': 23.298873279238023,
   'Eth': -4.156241679227686,
   'RT': 4.0501452584920905,
   'Ecor': 15.092486341518327},
  '03_vdW-DF': {'ZPE': 9.528063397365486,
   'Eth': -2.365058429444746,
   'RT': 4.0501452584920905,
   'Ecor': 3.1128597094285517},
  '04_rev-vdW-DF2': {'ZPE': 15.249076939617908,
   'Eth': -3.588744267036992,
   'RT': 4.0501452584920905,
   'Ecor': 7.610187414088035}},
 'C2H6': {'01_PBE-D2-Ne': {'ZPE': 11.84988879220441,
   'Eth': -2.2907260551116764,
   'RT': 6.4629977529129095,
   'Ecor': 3.096164984180038},
  '02_revPBE-D4': {'ZPE': 12.73401204745005,
   'Eth': -1.9359924732540215,
   'RT': 6.4629977529129095,
   'Ecor': 4.33502182128345},
  '03_vdW-DF': {'ZPE': -0.48434476561237716,
   'Eth': -0.12308335017703698,
   'RT': 6.4629977529129095,
   'Ecor': -7.0704258687027535},
  '04_rev-vdW-DF2': {'ZPE': 3.

In [18]:



df = pd.DataFrame.from_dict(ethermal_df_dict)
df = df.round(1)
df.columns = final_surface_nice_molecule_labels
# df.index = final_surface_molecule_nice_labels

with pd.option_context("max_colwidth", 1000):
    ethermal_latex_input = r"\begin{turnpage}" + """\n\\begin{table}[]\n\\adjustbox{max width=1.5\\textwidth}{%\n""" + df.to_latex(escape=False) + "}\n\caption{\label{table:ethermal}" + """$E_\\textrm{thermal}$ for the 4 functionals for the MgO (001) surface} """ + "\n\\end{table}\n\\end{turnpage}"

with open('Data/ethermal.tex','w') as f:
    f.write(ethermal_latex_input)

In [19]:
# Comparing vibrational energies for 

ethermal_2l_effect = {adsorbate: {'Ecor AD': 0, 'Ecor AD+SLAB': 0 } for adsorbate in ['CO', 'CO2_2','CO2_3', 'H2O']}


for adsorbate in ['CO', 'CO2_2','CO2_3', 'H2O']:
    ecor_list = []
    for idx_xc_func, xc_func in enumerate(dft_functionals['MgO'][:4]):
        if adsorbate == 'CO2_3':
            a, b = read_vib_freq(f'Data/DFT/05-Vib/MgO/{adsorbate}_AD/{xc_func}/AD_SLAB/OUTCAR'.format(i)) # Adsorbate on Slab
            c, d = read_vib_freq(f'Data/DFT/05-Vib/MgO/{adsorbate}_AD/{xc_func}/AD/OUTCAR'.format(i)) # AD
            ene_ad_slab, eth_ad_slab, zpe_ad_slab, kT = get_quasi_rrho(a, b, mol_temp_and_dof_dict['MgO'][adsorbate][0])
            ene_ad, eth_ad, zpe_ad, kT = get_quasi_rrho(c, d, mol_temp_and_dof_dict['MgO'][adsorbate][0])
            ecor_list += [ene_ad_slab - ene_ad - kT]
        else:
            a, b = read_vib_freq(f'Data/DFT/05-Vib/MgO/2L_Test/{adsorbate}/{xc_func}/AD_SLAB/OUTCAR'.format(i)) # Adsorbate on Slab
            c, d = read_vib_freq(f'Data/DFT/05-Vib/MgO/{adsorbate}/{xc_func}/AD/OUTCAR'.format(i)) # AD
            e, f = read_vib_freq(f'Data/DFT/05-Vib/MgO/2L_Test/{adsorbate}/{xc_func}/SLAB/OUTCAR'.format(i)) 

            # Calculate thermal and zero-point corrections as well as RT term for adsorbate and adsorbate on slab
            ene_ad_slab, eth_ad_slab, zpe_ad_slab, kT = get_quasi_rrho(a, b, mol_temp_and_dof_dict['MgO'][adsorbate][0])
            ene_ad, eth_ad, zpe_ad, kT = get_quasi_rrho(c, d, mol_temp_and_dof_dict['MgO'][adsorbate][0])
            ene_slab, eth_slab, zpe_slab, kT = get_quasi_rrho(e, f, mol_temp_and_dof_dict['MgO'][adsorbate][0])
            # Store thermal energy contributions in the nested dictionary 'thermal_ene_contributions'
            ecor_list += [ene_ad_slab - ene_ad - ene_slab - kT]

    if adsorbate == 'CO2_3':
        ethermal_2l_effect[adsorbate]['Ecor AD+SLAB'] = f"{ethermal_df_dict[f'MgO_{adsorbate}'][f'Final Ecor']:.1f} " + r"${\pm}$" + f" {ethermal_df_dict[f'MgO_{adsorbate}'][f'Ecor error']:.1f}"
        ethermal_2l_effect[adsorbate]['Ecor AD'] = f"{np.mean(ecor_list):.1f} " + r"${\pm}$" + f" {np.std(ecor_list):.1f}"
                # ethermal_2l_effect[adsorbate]['Diff'] = ethermal_2l_effect[adsorbate]['Ecor AD+SLAB'] - ethermal_2l_effect[adsorbate]['Ecor AD']        


    else:
        ethermal_2l_effect[adsorbate]['Ecor AD+SLAB'] = f"{np.mean(ecor_list):.1f} " + r"${\pm}$" + f" {np.std(ecor_list):.1f}"
        ethermal_2l_effect[adsorbate]['Ecor AD'] = f"{ethermal_df_dict[f'MgO_{adsorbate}'][f'Final Ecor']:.1f} " + r"${\pm}$" + f" {ethermal_df_dict[f'MgO_{adsorbate}'][f'Ecor error']:.1f}"
        # ethermal_2l_effect[adsorbate]['Diff'] = ethermal_2l_effect[adsorbate]['Ecor AD+SLAB'] - ethermal_2l_effect[adsorbate]['Ecor AD']
    

In [20]:
ethermal_2l_effect

{'CO': {'Ecor AD': '18.9 ${\\pm}$ 1.5', 'Ecor AD+SLAB': '19.0 ${\\pm}$ 0.5'},
 'CO2_2': {'Ecor AD': '-10.0 ${\\pm}$ 1.3',
  'Ecor AD+SLAB': '-8.9 ${\\pm}$ 1.2'},
 'CO2_3': {'Ecor AD': '18.3 ${\\pm}$ 1.2',
  'Ecor AD+SLAB': '31.3 ${\\pm}$ 1.6'},
 'H2O': {'Ecor AD': '59.0 ${\\pm}$ 0.6', 'Ecor AD+SLAB': '61.8 ${\\pm}$ 1.9'}}

In [21]:
# Make a dataframe for the 2L effect

df = pd.DataFrame.from_dict(ethermal_2l_effect)
df.columns = ['CO', 'Physisorbed \\ce{CO2}','Chemisorbed \\ce{CO2}', '\\ce{H2O}']

df = df.transpose()

with pd.option_context("max_colwidth", 1000):
    ethermal_latex_input = """\\begin{table}[]\n""" + "\\caption{\\label{table:ethermal_layer_effect} $E_\\textrm{thermal}$ comparing freezing entire surface (AD) and including surface (AD+SLAB) in the thermal treatment}\n" + df.to_latex(escape=False,column_format='lrr') + "\n\\end{table}"

with open('Tables/ethermal_slab_effect.tex','w') as f:
    f.write(ethermal_latex_input)

# Lateral interaction corrections

In [22]:
eint_ml_cluster_cc_corr = {x:0 for x in ['CH4_ML','C2H6_ML','CH3OH_2','CH3OH_4','H2O_4','NO_6']}

system = 'MgO'

# Start by calculating the correction for the clustered molecules
for i in ['H2O_4','CH3OH_4']:
    dummy_ene_list = []
    for j in range(1,overall_systems_num_monomers['MgO'][i]+2):
        ene_hf = []
        ene_ccsdt = []
        for k in ['TZ','QZ']:
            ene_hf += [find_energy(f'Data/cWFT/CCSDT/{system}/{i}/Cluster_Corr/{j}_FNOCC/{k}/mrcc.out', code_format='mrcc',typ='hf')*Hartree*1000]
            ene_ccsdt += [find_energy(f'Data/cWFT/CCSDT/{system}/{i}/Cluster_Corr/{j}_FNOCC/{k}/mrcc.out', code_format='mrcc',typ='fnoccsdt')*Hartree*1000]

        dummy = extrapolate.get_cbs(ene_hf[0],ene_ccsdt[0],ene_hf[1],ene_ccsdt[1],X=3,Y=4,family='acc',output=False)
        dummy_ene_list += [dummy[-1]]

    eint_ml_cluster_cc_corr[i] = (dummy_ene_list[-1] - np.sum(dummy_ene_list[:overall_systems_num_monomers['MgO'][i]]))/overall_systems_num_monomers['MgO'][i] - clus_eint_dict[i]['02_revPBE-D4']['02_revPBE-D4']


# Now let's do the monolayer calculations.
for i in ['CH4_ML','C2H6_ML']:
    lat_int_fnocc = 0
    lat_int_revpbed4 = 0
    if i == 'CH4_ML':
        num_dimer = 68
    elif i == 'C2H6_ML':
        num_dimer = 72
    for j in range(1,num_dimer+1):
        ene_hf = []
        ene_ccsdt = []
        for k in ['TZ','QZ']:
            a = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_FNOCC/{j}/{k}/0/mrcc.out',code_format='mrcc', typ='hf')
            b = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_FNOCC/{j}/{k}/1/mrcc.out',code_format='mrcc', typ='hf')
            c = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_FNOCC/{j}/{k}/2/mrcc.out',code_format='mrcc', typ='hf')
            ene_hf += [(a-b-c)*Hartree*1000/2]

            a = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_FNOCC/{j}/{k}/0/mrcc.out',code_format='mrcc', typ='fnoccsdt')
            b = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_FNOCC/{j}/{k}/1/mrcc.out',code_format='mrcc', typ='fnoccsdt')
            c = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_FNOCC/{j}/{k}/2/mrcc.out',code_format='mrcc', typ='fnoccsdt')
            ene_ccsdt += [(a-b-c)*Hartree*1000/2]

        dummy1 = extrapolate.get_cbs(ene_hf[0],ene_ccsdt[0],ene_hf[1],ene_ccsdt[1],X=3 , Y=4, family='acc',output=False)
        lat_int_fnocc += dummy1[-1]

        # Computing the revPBE-D4 lattice energy
        a = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_revPBE-D4/{j}/0/orca.out',code_format='orca', typ='dft')
        b = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_revPBE-D4/{j}/1/orca.out',code_format='orca', typ='dft')
        c = find_energy(f'Data/cWFT/CCSDT/{system}/{i}/MBE_Corr/{j}_revPBE-D4/{j}/2/orca.out',code_format='orca', typ='dft')
        lat_int_revpbed4 += (a-b-c)*Hartree*1000/2
    eint_ml_cluster_cc_corr[i] = lat_int_fnocc - lat_int_revpbed4

In [23]:
system = 'MgO'

geom_cc_corr = {x: 0 for x in ['CO2_3','CO2_3_B3LYP','NO_6', 'NO_6_B3LYP']}

for i in ['CO2_3','CO2_3_B3LYP','NO_6', 'NO_6_B3LYP']:
    dummy_ene_list = []
    for j in ['FS','RS']:
        ene_hf = []
        ene_ccsdt = []
        for k in ['TZ','QZ']:
            if 'CO2' in i:
                ene_hf  += [find_energy(f'Data/cWFT/CCSDT/{system}/{i}/Conform_Corr/{j}/{k}/mrcc.out',code_format='mrcc', typ='hf')]
                ene_ccsdt  += [find_energy(f'Data/cWFT/CCSDT/{system}/{i}/Conform_Corr/{j}/{k}/mrcc.out',code_format='mrcc', typ='ccsdt')]
            else:
                ene_hf  += [find_energy(f'Data/cWFT/CCSDT/{system}/{i}/Conform_Corr/{k}/{j}/mrcc.out',code_format='mrcc', typ='hf')]
                ene_ccsdt  += [find_energy(f'Data/cWFT/CCSDT/{system}/{i}/Conform_Corr/{k}/{j}/mrcc.out',code_format='mrcc', typ='ccsdt')]                
        dummy1 = extrapolate.get_cbs(ene_hf[0],ene_ccsdt[0],ene_hf[1],ene_ccsdt[1],X=3 , Y=4, family='acc',output=False)
        dummy_ene_list += [dummy1[-1]]

    if i == 'CO2_3':
        ad_fs_ene = find_energy(f'Data/DFT/04-Int_Ene/MgO/CO2_3/02_revPBE-D4/02_revPBE-D4/AD_FS/OUTCAR',code_format='vasp')
        ad_rs_ene = find_energy(f'Data/DFT/01-Mol/MgO/CO2/02_revPBE-D4/OUTCAR',code_format='vasp')
    elif i == 'CO2_3_B3LYP':
        ad_fs_ene = find_energy(f'Data/DFT/04-Int_Ene/MgO/CO2_3/06_B3LYP-D2-Ne/06_B3LYP-D2-Ne/AD_FS/OUTCAR',code_format='vasp')
        ad_rs_ene = find_energy(f'Data/DFT/01-Mol/MgO/CO2/06_B3LYP-D2-Ne/OUTCAR',code_format='vasp')
    elif i == 'NO_6':
        ad_fs_ene = find_energy(f'Data/DFT/04-Int_Ene/MgO/NO_6/02_revPBE-D4/02_revPBE-D4/AD_FS/OUTCAR',code_format='vasp')
        ad_rs_ene = find_energy(f'Data/DFT/01-Mol/MgO/N2O2/02_revPBE-D4/OUTCAR',code_format='vasp')        
    elif i == 'NO_6_B3LYP':
        ad_fs_ene = find_energy(f'Data/DFT/04-Int_Ene/MgO/NO_6/06_B3LYP-D2-Ne/06_B3LYP-D2-Ne/AD_FS/OUTCAR',code_format='vasp')
        ad_rs_ene = find_energy(f'Data/DFT/01-Mol/MgO/N2O2/06_B3LYP-D2-Ne/OUTCAR',code_format='vasp')     
    dummy_cc_ene = (dummy_ene_list[-1] - dummy_ene_list[0])*Hartree*1000
    dummy_dft_ene = (ad_rs_ene - ad_fs_ene)*1000

    if 'NO_6'  in i:
        geom_cc_corr[i] = (dummy_cc_ene - dummy_dft_ene)/2
    else:
        geom_cc_corr[i] = (dummy_cc_ene - dummy_dft_ene)





# CCSD(T) Eint

In [24]:
cluster_size_dict = {y: {x: [] for x in overall_systems[y]} for y in ['MgO','TiO2_rutile','TiO2_anatase']}

for system in ['MgO','TiO2_rutile','TiO2_anatase']:
    if system == 'MgO':
        max_size = 8
    elif system == 'TiO2_rutile':
        max_size = 9
    elif system == 'TiO2_anatase':
        max_size = 10

    for i in overall_systems[system]:
        molecule = i.split('_')[0]
        for j in range(1,max_size+1):
            num_atoms = 0
            with open(f'Data/cWFT/MP2/{system}/{i}/{j}/DZ/SLAB_CP/orca.inp','r') as file:
                process_lines = False
                # Loop through each line in the file
                for line in file:
                    # Check if the line contains the string "coords"
                    if "coords" in line:
                        # Set the flag to True to start processing lines
                        process_lines = True
                        continue  # Skip the line with "coords" itself

                    # If the flag is True, process the line
                    if process_lines:
                        # Do something with the line (e.g., print it)
                        if ('Mg   ' in line) or ('O   ' in line) or ('Ti   ' in line):
                            num_atoms += 1
            cluster_size_dict[system][i] += [num_atoms]  # Stripping to remove leading/trailing whitespace


In [25]:
# Obtaining data for Table S9 of adsorption energies at MP2 and CCSD(T) levels with increasing cluster size.
# Define of levels of theories 'lot_list' and initialize dictionaries to store adsorption energy results

lot_list = ['MP2_DZ', 'MP2_TZ', 'MP2_QZ','MP2_DZ_smallcore', 'MP2_TZ_smallcore','MP2_QZ_smallcore', 'MP2_TZQZ', 'MP2_DZTZ', 'MP2_DZTZ_smallcore','MP2_TZQZ_smallcore', 'LCCSDT_DZTZ', 'LMP2_DZTZ', 'CCCSDT_DZTZ', 'CMP2_DZTZ']
ene_list = {y: {x: {z:[] for z in lot_list} for x in overall_systems[y]} for y in ['MgO','TiO2_rutile','TiO2_anatase']}

for system in ['MgO','TiO2_anatase','TiO2_rutile']: #['MgO','TiO2_rutile','TiO2_anatase']:
    if system == 'MgO':
        max_size = 7
    elif system == 'TiO2_rutile':
        max_size = 8
    elif system == 'TiO2_anatase':
        max_size = 9
    
    for i in overall_systems[system]:

        molecule = i.split('_')[0]
        tot_atom_list = cluster_size_dict[system][i]

        if system == 'MgO' and '_ML' in i:
            num_monomers = 1
        else:
            num_monomers = overall_systems_num_monomers[system][i]


        for j in range(1,max_size+1):
            # Start by doing the DZ calculation
            # Calculate adsorption energy from difference of AD_SLAB, SLAB_CP, and AD_CP, where CP means counterpoise corrected (i.e. ghost basis sets have been placed)
            ene_list[system][i]['MP2_DZ'] += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/DZ', code_format='orca',typ='dft')*Hartree*1000/num_monomers]       


            if j < max_size-1:
                ene_list[system][i]['MP2_TZ'] += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/TZ', code_format='orca',typ='dft')*Hartree*1000/num_monomers]
                ene_hf = []
                ene_mp2 = []
                for k in ['DZ','TZ']:
                    ene_hf += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}', code_format='orca',typ='hf_lmp2')*Hartree*1000]
                    ene_mp2 += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}', code_format='orca',typ='ri-mp2')*Hartree*1000]

                dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                ene_list[system][i]['MP2_DZTZ'] += [dummy[-1]/num_monomers]

            if j < max_size-3:
                ene_list[system][i]['MP2_QZ'] += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/QZ', code_format='orca',typ='dft')*Hartree*1000/num_monomers]
                ene_hf = []
                ene_mp2 = []
                for k in ['TZ','QZ']:
                    ene_hf += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}', code_format='orca',typ='hf_lmp2')*Hartree*1000]
                    ene_mp2 += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}', code_format='orca',typ='ri-mp2')*Hartree*1000]

                dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='mixcc',output=False)
                ene_list[system][i]['MP2_TZQZ'] += [dummy[-1]/num_monomers]

    
            if system == 'MgO' and j < 4:
                if (i in ['NO_1','NO_2','NO_3','NO_4','NO_5']):
                    # Add the core data
                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_list[system][i][f'MP2_{k}_smallcore'] += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}_Mg_Hecore', code_format='orca',typ='dft')*Hartree*1000/num_monomers]
                        ene_hf += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}_Mg_Hecore', code_format='orca',typ='hf_lmp2')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}_Mg_Hecore', code_format='orca',typ='ri-mp2')*Hartree*1000]

                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    ene_list[system][i]['MP2_DZTZ_smallcore'] += [dummy[-1]/num_monomers]  
                else:
                    # Perform basis set extrapolation for LNO-CCSD(T) DZTZ
                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lccsdt')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    ene_list[system][i]['LCCSDT_DZTZ'] += [dummy[-1]/num_monomers]
                    # Also add LMP2 DZTZ numbers
                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lmp2')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    ene_list[system][i]['LMP2_DZTZ'] += [dummy[-1]/num_monomers]

            elif 'TiO2' in system and j < 5:
                # Add the core data
                ene_hf = []
                ene_mp2 = []
                for k in ['TZ','QZ']:
                    ene_list[system][i][f'MP2_{k}_smallcore'] += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}_Ti_Necore', code_format='orca',typ='dft')*Hartree*1000/num_monomers]
                    ene_hf += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}_Ti_Necore', code_format='orca',typ='hf_lmp2')*Hartree*1000]
                    ene_mp2 += [get_eads(f'Data/cWFT/MP2/{system}/{i}/{j}/{k}_Ti_Necore', code_format='orca',typ='ri-mp2')*Hartree*1000]
                dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='mixcc',output=False)
                ene_list[system][i]['MP2_TZQZ_smallcore'] += [dummy[-1]/num_monomers]

                # Perform basis set extrapolation for LNO-CCSD(T) DZTZ
                ene_hf = []
                ene_mp2 = []
                for k in ['DZ','TZ']:
                    ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                    ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lccsdt')*Hartree*1000]
                dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                ene_list[system][i]['LCCSDT_DZTZ'] += [dummy[-1]/num_monomers]
                # Also add LMP2 DZTZ numbers
                ene_hf = []
                ene_mp2 = []
                for k in ['DZ','TZ']:
                    ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                    ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lmp2')*Hartree*1000]
                dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                ene_list[system][i]['LMP2_DZTZ'] += [dummy[-1]/num_monomers]

            if j < 2:
                if (i in ['NO_1','NO_2','NO_3','NO_4','NO_5']):
                    # We get the local CCSD(T) but correct with FNO-CCSD(T) on the DZ basis set
                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}_CC', code_format='orca',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}_CC', code_format='orca',typ='lccsdt')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    ene_list[system][i]['LCCSDT_DZTZ'] += [(dummy[-1])]

                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}_MP2', code_format='orca_mp2',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}_MP2', code_format='orca_mp2',typ='lmp2')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    ene_list[system][i]['LMP2_DZTZ'] += [(dummy[-1])]


                elif system == 'MgO' and (i in large_systems_mgo):
                    # We get the local CCSD(T) but correct with FNO-CCSD(T) on the DZ basis set
                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lccsdt')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    
                    k = 'DZ'

                    ene_dz_diff = (get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Canonical/{k}', code_format='mrcc',typ='fnoccsdt_tot') - get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lccsdt_tot'))*Hartree*1000

                    ene_list[system][i]['CCCSDT_DZTZ'] += [(ene_dz_diff + dummy[-1])/num_monomers]

                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lmp2')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    
                    k = 'DZ'
                    ene_dz_diff = (get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Canonical/{k}', code_format='mrcc',typ='fnoccsdt_mp2_tot') - get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Local/{k}', code_format='mrcc',typ='lccsdt_lmp2_tot'))*Hartree*1000

                    ene_list[system][i]['CMP2_DZTZ'] += [(ene_dz_diff + dummy[-1])/num_monomers]                    

                else:
                    # Perform basis set extrapolation for canonical CCSD(T) DZTZ
                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Canonical/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Canonical/{k}', code_format='mrcc',typ='ccsdt')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    ene_list[system][i]['CCCSDT_DZTZ'] += [dummy[-1]/num_monomers]
                    ene_hf = []
                    ene_mp2 = []
                    for k in ['DZ','TZ']:
                        ene_hf += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Canonical/{k}', code_format='mrcc',typ='hf')*Hartree*1000]
                        ene_mp2 += [get_eads(f'Data/cWFT/CCSDT/{system}/{i}/{j}_Canonical/{k}', code_format='mrcc',typ='mp2')*Hartree*1000]
                    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
                    ene_list[system][i]['CMP2_DZTZ'] += [dummy[-1]/num_monomers]


In [26]:
def fit_eint2(x_data, y_data):
    x_transformed_data = np.array([1/x for x in x_data])

    # Degree of the polynomial
    degree = 2  # You can change the degree based on your requirement

    # Perform the polynomial fit
    coefficients = np.polyfit(x_transformed_data, y_data, degree) #, w=[0.2]*2 + [1.0]*(len(x_data)-2)) # -1))

    # Generate fitted values
    y_fitted = np.polyval(coefficients, x_transformed_data)

    # Print the coefficients of the polynomial
    return coefficients[0], coefficients[1], coefficients[2]

In [27]:
def fit_eint1(x_data, y_data):
    x_transformed_data = np.array([1/x for x in x_data])

    # Degree of the polynomial
    degree = 1  # You can change the degree based on your requirement

    # Perform the polynomial fit
    coefficients = np.polyfit(x_transformed_data, y_data, degree)


    # Generate fitted values
    y_fitted = np.polyval(coefficients, x_transformed_data)

    # Print the coefficients of the polynomial
    return coefficients[0], coefficients[0], coefficients[1]

In [28]:
from scipy.optimize import curve_fit

def fit_eint(x_data, y_data):
    def model(x, A, gamma, B):
        return - A / x**gamma + B

    # Get the negative of the y_data
    y_data = -(np.array(y_data) - 1000)

    # Check if any y_data is negative
    if any(y_data < 0):
        raise ValueError("y_data contains negative values")

    # Make initial guesses for the A, gamma, B parameters, we assume gamma=1 and B is the last y value
    A_initial = - x_data[-2]*(y_data[-2] - y_data[-1])
    if A_initial < 0:
        A_initial = abs(A_initial)

    # Perform the curve fitting
    initial_guess = [A_initial, 1, y_data[-1]]  # You may need to adjust this initial guess
    bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])  # Constrain the solution to positive values
    params, covariance = curve_fit(model, x_data, y_data, p0=initial_guess,bounds=bounds)

    A_fit, gamma_fit, B_fit = params

    return A_fit, gamma_fit, -(B_fit - 1000)

In [29]:
# Obtaining the extrapolated data which 

lot_list = ['MP2_DZ', 'MP2_TZ', 'MP2_QZ', 'MP2_TZQZ', 'MP2_DZTZ', 'MP2_DZTZ_lc', 'LCCSDT_DZTZ', 'LMP2_DZTZ', 'CCCSDT_DZTZ', 'CMP2_DZTZ']
ene_extrap_list = {y: {x: {z:[] for z in ['MP2_DZ','MP2_DZTZ']} for x in overall_systems[y]} for y in ['MgO','TiO2_rutile','TiO2_anatase']}

for system in ['MgO','TiO2_anatase','TiO2_rutile']: #,'TiO2_anatase','TiO2_rutile']: #['MgO','TiO2_rutile','TiO2_anatase']:
    if system == 'MgO':
        max_size = 7
        min_size = 0
    elif system == 'TiO2_rutile':
        max_size = 8
        min_size = 2
    elif system == 'TiO2_anatase':
        max_size = 9
        min_size = 2

    for i in overall_systems[system]:
        # print(i)
        try:
            # molecule = i.split('_')[0]
            # if system == 'MgO' and i == 'CO2_3':
            #     system_tot_atom_list = cluster_size_dict[system][i][1:max_size]
            #     system_eint_dz_list = ene_list[system][i]['MP2_DZ'][1:max_size]
            #     system_eint_dztz_list = ene_list[system][i]['MP2_DZTZ'][1:max_size-2]
            # else:
            system_tot_atom_list = cluster_size_dict[system][i][min_size:max_size]
            system_eint_dz_list = ene_list[system][i]['MP2_DZ'][min_size:max_size]
            system_eint_dztz_list = ene_list[system][i]['MP2_DZTZ'][min_size:max_size-2]

            # Regularise the fit by making the first value a bit different from the rest
            # system_eint_dz_list[0] = system_eint_dz_list[0] - (system_eint_dz_list[-1] - system_eint_dz_list[0])*2
            # system_eint_dztz_list[0] = system_eint_dztz_list[0] - (system_eint_dztz_list[-1] - system_eint_dztz_list[0])*2
            # First doing the DZ calculation
            for j in range(1,max_size-min_size+1):    
                if j < 4:
                    ene_extrap_list[system][i]['MP2_DZ'] += [0]
                # Extrapolate if we have more than 3 clusters
                else:
                    # gamma_best = find_co_gamma(system_eint_dz_list[:j],tot_atom_list=system_tot_atom_list[:j])
                    # slope, emp2_infty, r, p, se = linregress([1/(x**(gamma_best)) for x in system_tot_atom_list[:j]], [float(x) for x in system_eint_dz_list[:j]])
                    A_opt, gamma, eint_bulklim = fit_eint1(system_tot_atom_list[:j], system_eint_dz_list[:j])
                    ene_extrap_list[system][i]['MP2_DZ'] += [eint_bulklim]
                    # print(system_eint_dz_list[:j], system_tot_atom_list[:j], gamma_best,emp2_infty)

            for j in range(1,max_size-2-min_size+1):    
                if j < 4:
                    ene_extrap_list[system][i]['MP2_DZTZ'] += [0]
                # Extrapolate if we have more than 3 clusters
                else:
                    # gamma_best = find_co_gamma(system_eint_dz_list[:j],tot_atom_list=system_tot_atom_list[:j])
                    # # gamma_best = 0.3
                    # slope, emp2_infty, r, p, se = linregress([1/(x**(gamma_best)) for x in system_tot_atom_list[:j]], [float(x) for x in system_eint_dztz_list[:j]])
                    A_opt, gamma, eint_bulklim = fit_eint1(system_tot_atom_list[:j], system_eint_dztz_list[:j])
                    ene_extrap_list[system][i]['MP2_DZTZ'] += [eint_bulklim]
                    # print(system_eint_dztz_list[:j], gamma_best,emp2_infty)
        except:
            print(f'{i} {j} dont work', system_tot_atom_list[:j], system_eint_dztz_list[:j])


In [30]:
ene_extrap_list['MgO']['CH3OH_4']

{'MP2_DZ': [0,
  0,
  0,
  -398.6172411747905,
  -398.71054839276127,
  -400.94229022627,
  -402.6730967760484],
 'MP2_DZTZ': [0, 0, 0, -468.685288434571, -468.9638549080172]}

In [31]:
ene_list['MgO']['CH3OH_4']['MP2_DZ']

[-319.7058832214984,
 -323.10083579088405,
 -370.03682872254103,
 -373.1913655404686,
 -378.17316876321735,
 -385.7704074238453,
 -389.1343103406241]

In [32]:
# Creating the final CCSD(T) adsorption energies for all of the systems

final_cc_eint_list = {y: {x: {z:[0,0] for z in ['MP2_bulk','deltaCC','deltaBasis','deltaCore','Mol_Int','Mol_Int_Corr','Mol_Conform_Corr','final', 'final_overall']} for x in overall_systems[y]} for y in ['MgO','TiO2_rutile','TiO2_anatase']}


for system in ['MgO','TiO2_anatase','TiO2_rutile']: #['MgO','TiO2_rutile','TiO2_anatase']:
    for i in overall_systems[system]:
        # Let's first get MP2_DZTZ and its error here

        final_cc_eint_list[system][i]['MP2_bulk'][0] = ene_extrap_list[system][i]['MP2_DZTZ'][-1]
        final_cc_eint_list[system][i]['MP2_bulk'][1] = abs(ene_extrap_list[system][i]['MP2_DZ'][-1] - ene_extrap_list[system][i]['MP2_DZ'][len(ene_extrap_list[system][i]['MP2_DZTZ'])-1])

        # Let's now make the deltaCC correction
        if 'NO' in i:
            final_cc_eint_list[system][i]['deltaCC'] = [ ene_list[system][i]['LCCSDT_DZTZ'][0] -  ene_list[system][i]['LMP2_DZTZ'][0],0]
        else:
            final_cc_eint_list[system][i]['deltaCC'][0] = ene_list[system][i]['CCCSDT_DZTZ'][-1] -  ene_list[system][i]['CMP2_DZTZ'][-1]
            final_cc_eint_list[system][i]['deltaCC'][1] = 2*np.std(np.array(ene_list[system][i]['LCCSDT_DZTZ'][-2:]) -  np.array(ene_list[system][i]['LMP2_DZTZ'][-2:]))

        if (system == 'MgO') and (i in ['CH4_ML','C2H6_ML']):
            final_cc_eint_list[system][i]['Mol_Int'][0] = ml_eint_dict[i]['02_revPBE-D4']['02_revPBE-D4']
            final_cc_eint_list[system][i]['Mol_Int_Corr'][0] = eint_ml_cluster_cc_corr[i]
        elif (system == 'MgO') and (i in ['H2O_4','CH3OH_4']):
            final_cc_eint_list[system][i]['Mol_Int'][0] = clus_eint_dict[i]['02_revPBE-D4']['02_revPBE-D4']
            final_cc_eint_list[system][i]['Mol_Int_Corr'][0] = eint_ml_cluster_cc_corr[i]
        elif (system == 'MgO') and (i in ['CO2_3']):
            final_cc_eint_list[system][i]['Mol_Conform_Corr'][0] = -geom_cc_corr[i]
        elif (system == 'MgO') and (i in ['NO_6']):
            final_cc_eint_list[system][i]['Mol_Conform_Corr'][0] = -geom_cc_corr[i]
            final_cc_eint_list[system][i]['Mol_Int'][0] = -45

        # Let's make the basis set correction
        final_cc_eint_list[system][i]['deltaBasis'][0] = np.mean(np.array(ene_list[system][i]['MP2_TZQZ'][-3:]) -  np.array(ene_list[system][i]['MP2_DZTZ'][len(ene_list[system][i]['MP2_TZQZ'])-3:len(ene_list[system][i]['MP2_TZQZ'])]))
        final_cc_eint_list[system][i]['deltaBasis'][1] = 2*np.std(np.array(ene_list[system][i]['MP2_TZQZ'][-3:]) -  np.array(ene_list[system][i]['MP2_DZTZ'][len(ene_list[system][i]['MP2_TZQZ'])-3:len(ene_list[system][i]['MP2_TZQZ'])]))

        if 'TiO2' in system:
            final_cc_eint_list[system][i]['deltaCore'][0] = ene_list[system][i]['MP2_TZQZ_smallcore'][2] -  ene_list[system][i]['MP2_TZQZ'][2]

        dummy_sum2 = 0
        for j in ['MP2_bulk','deltaCC','deltaBasis','deltaCore','Mol_Int','Mol_Int_Corr','Mol_Conform_Corr']:
            final_cc_eint_list[system][i]['final_overall'][0] += final_cc_eint_list[system][i][j][0] 
            dummy_sum2 += final_cc_eint_list[system][i][j][1]**2

        final_cc_eint_list[system][i]['final_overall'][1] = np.sqrt(dummy_sum2)

        dummy_sum2 = 0
        for j in ['MP2_bulk','deltaCC','deltaBasis','deltaCore']:
            final_cc_eint_list[system][i]['final'][0] += final_cc_eint_list[system][i][j][0] 
            dummy_sum2 += final_cc_eint_list[system][i][j][1]**2

        final_cc_eint_list[system][i]['final'][1] = np.sqrt(dummy_sum2)

In [33]:
si_table_eint_str = ""

system_info_dict = {
     'MgO':{'num_clus': 7, 'basis_labels': ['MP2_DZ', 'MP2_DZ Extrap', 'MP2_DZTZ', 'MP2_DZTZ Extrap','MP2_TZQZ','LMP2_DZTZ', 'LCCSDT_DZTZ'], 'basis_nice_labels': ['MP2 awCVDZ', 'MP2 Extrap. awCVDZ', 'MP2 CBS(awCVDZ/awCVTZ)', 'MP2 Extrap. CBS(awCVDZ/awCVTZ)','MP2 CBS(awCVTZ/awCVQZ)', 'LMP2 CBS(awCVDZ/awCVTZ)', 'LNO-CCSD(T) CBS(awCVDZ/awCVTZ)']},
     'TiO2_rutile':{'num_clus': 6, 'basis_labels': ['MP2_DZ', 'MP2_DZ Extrap', 'MP2_DZTZ', 'MP2_DZTZ Extrap', 'MP2_TZQZ','MP2_TZQZ_smallcore','LMP2_DZTZ', 'LCCSDT_DZTZ'], 'basis_nice_labels': ['MP2 aVDZ', 'MP2 Extrap. aVDZ', 'MP2 CBS(aVDZ/aVTZ)', 'MP2 Extrap. (aVDZ/aVTZ)', 'MP2 CBS(aVTZ/aVQZ)', 'MP2 CBS(awCVTZ/awCVQZ)','LMP2 CBS(aVDZ/aVTZ)', 'LNO-CCSD(T) CBS(aVDZ/aVTZ)']},
     'TiO2_anatase':{'num_clus': 7, 'basis_labels': ['MP2_DZ', 'MP2_DZ Extrap', 'MP2_DZTZ', 'MP2_DZTZ Extrap', 'MP2_TZQZ','MP2_TZQZ_smallcore','LMP2_DZTZ', 'LCCSDT_DZTZ'], 'basis_nice_labels': ['MP2 aVDZ', 'MP2 Extrap. aVDZ', 'MP2 CBS(aVDZ/aVTZ)', 'MP2 Extrap. (aVDZ/aVTZ)', 'MP2 CBS(aVTZ/aVQZ)', 'MP2 CBS(awCVTZ/awCVQZ)','LMP2 CBS(aVDZ/aVTZ)', 'LNO-CCSD(T) CBS(aVDZ/aVTZ)']}
}

adsorbate_config = {
    'MgO':{'NO_1': 'Vertical-Hollow',
    'NO_2': 'Vertical-Mg',
    'NO_3': 'Bent-Bridge',
    'NO_4': 'Bent-Mg',
    'NO_5': 'Bent-O',
    'NO_6': 'Dimer',
    'N2O_1': 'Horizontal',
    'N2O_2': 'Vertical',
    'CO2_2': 'Physisorbed',
    'CO2_3': 'Chemisorbed',
    'CH3OH_1': 'Monomer',
    'CH3OH_2': 'Dimer',
    'CH3OH_4': 'Tetramer',
    'CH4_ML': 'Monolayer',
    'C2H6_ML': 'Monolayer',
    'H2O_4': 'Tetramer'},
    'TiO2_rutile': {
        'CO2_1': 'Parallel',
        'CO2_2': 'Tilted'
    },
    'TiO2_anatase': {}
}


for system in ['MgO','TiO2_rutile','TiO2_anatase']:
    if system == 'MgO':
        max_size = 7
        min_size = 0
    elif system == 'TiO2_rutile':
        max_size = 8
        min_size = 2
    elif system == 'TiO2_anatase':
        max_size = 9
        min_size = 2
    for molecule in overall_systems[system]:
        # Initialize an empty dictionary 'trunc_ene_list' to store extrapolate interaction energies for truncating below 8 clusters
        trunc_ene_list = {}

        # Loop through a list of electronic structure method names ('MP2_DZ', 'MP2_TZ', etc.)
        for i in system_info_dict[system]['basis_labels']:
            num_clus = system_info_dict[system]['num_clus']
            if 'Extrap' in i:
                system_eint_list = ene_extrap_list[system][molecule][i.replace(' Extrap','')]

            else:
                system_eint_list = ene_list[system][molecule][i][min_size:max_size]


            # Check if the length of the energy list 'ene_list[i]' is less than num_clus
            if len(system_eint_list) < num_clus:
                # Calculate the difference in length needed to make it num_clus
                dff_len = num_clus - len(system_eint_list)
                # Create a truncated energy list by adding zeros to meet the length of num_clus
                trunc_ene_list[i] = system_eint_list + [0] * dff_len
            else:
                # If the energy list already has at least num_clus values, keep it as is
                trunc_ene_list[i] = system_eint_list
        # Create a DataFrame 'df' using the truncated energy lists
        df = pd.DataFrame(trunc_ene_list)

        # Rename the columns of the DataFrame
        df.columns = [name + ']' for name in system_info_dict[system]['basis_nice_labels']]

        # Create a new column '# of atoms' and populate it with the first 8 elements of 'tot_atom_list'
        df['# of atoms'] = cluster_size_dict[system][molecule][min_size:max_size]

        # Rearrange the columns in the DataFrame and round the values to integers
        df = df[['# of atoms'] + [name + ']' for name in system_info_dict[system]['basis_nice_labels']]]
        df = df.round().astype(int)

        # Identify the type of SKZCAM clusters
        SKZCAM_cluster_type = 'none'

        if cluster_size_dict[system][molecule][min_size] == 6:
            SKZCAM_cluster_type = 'Mg-centered MgO (001)'
        elif cluster_size_dict[system][molecule][min_size] == 17:
            SKZCAM_cluster_type = 'O-centered MgO (001)'
        elif cluster_size_dict[system][molecule][min_size] == 10:
            SKZCAM_cluster_type = 'Hollow-centered MgO (001)'
        elif cluster_size_dict[system][molecule][min_size] == 34:
            SKZCAM_cluster_type = r'\ce{TiO2} rutile (110)'
        elif cluster_size_dict[system][molecule][min_size] == 29:
            SKZCAM_cluster_type = r'\ce{TiO2} anatase (101)'




        # Increment the index of the DataFrame by 1
        df.index += 1
        df.index.name = 'Cluster #'
        df = df.transpose()

        df_latex_string = df.to_latex()
        df_latex_string = df_latex_string.replace('&    0','&    -')
        df_latex_string = df_latex_string.replace('&   0','&   -')
        df_latex_string = df_latex_string.replace('&  0','&   -')
        df_latex_string = df_latex_string.replace('&     0','&      -')


        df_latex_string = "\n".join(
            [r'$E_\textrm{int}$[' + line if 'MP2' in line else line for line in df_latex_string.splitlines()]
        )

        df_latex_string = "\n".join(
            [r'$E_\textrm{int}$[' + line if 'CCSD(T)' in line else line for line in df_latex_string.splitlines()]
        )


        df_latex_string = "\n".join(
            [r'$E_\textrm{int}^\textrm{bulk}$[MP2 ' + line[29:] if 'Extrap.' in line else line for line in df_latex_string.splitlines()]
        )

        si_table_eint_str += """\\begin{table}[]\n"""

        # Getting the crystal system and molecule name
        if 'TiO2_rutile' in system:
            system_label = r'\ce{TiO2} rutile (110)'
        elif 'TiO2_anatase' in system:
            system_label = r'\ce{TiO2} anatase (101)'
        elif 'MgO' in system:
            system_label = r'\ce{MgO} (001)'


        molecule_label = r'\ce{' + molecule.split('_')[0] + '}'

        if molecule in adsorbate_config[system]:
            si_table_eint_str += "\caption{\label{" + f"table:eint_{system}_{molecule}" + "}" + """$E_\\textrm{int}$ (in meV) of the SKZCAM clusters for """ + f"{adsorbate_config[system][molecule].lower()} {molecule_label} on {system_label}." + f" The clusters correspond to the `{SKZCAM_cluster_type}' type visualized in Figure" + r"~\ref{fig:skzcam_clusters}" +  """.}\n"""
        else:
            si_table_eint_str += "\caption{\label{" + f"table:eint_{system}_{molecule}" + "}" + """$E_\\textrm{int}$ (in meV) of the SKZCAM clusters for """ + f"{molecule_label} on {system_label}." + f" The clusters correspond to the `{SKZCAM_cluster_type}' type visualized in Figure" + r"~\ref{fig:skzcam_clusters}" +  """.}\n"""
        si_table_eint_str += df_latex_string

        si_table_eint_str += "\\end{table}\n"

with open("Data/skzcam_eint_table.tex", "w") as text_file:
    text_file.write(si_table_eint_str)

In [34]:
final_surface_molecule_labels = [f'{surface}_{molecule}' for surface in final_cc_eint_list for molecule in final_cc_eint_list[surface]]
final_surface_molecule_nice_labels = []

cc_eint_df_dict = {surface_molecule: {} for surface_molecule in final_surface_molecule_labels}

for surface in final_cc_eint_list:
    # Getting the crystal system and molecule name
    if 'TiO2_rutile' in system:
        system_label = r'\ce{TiO2} rutile (110)'
    elif 'TiO2_anatase' in system:
        system_label = r'\ce{TiO2} anatase (101)'
    elif 'MgO' in system:
        system_label = r'\ce{MgO} (001)'
    for molecule in final_cc_eint_list[surface]:
        molecule_label = r'\ce{' + molecule.split('_')[0] + '}'

        for quantity in ['MP2_bulk','deltaCC','deltaBasis','deltaCore','final']:
            error = f'{final_cc_eint_list[surface][molecule][quantity][1]:4.1f}'.rjust(4)
            cc_eint_df_dict[f'{surface}_{molecule}'][quantity] = f'{final_cc_eint_list[surface][molecule][quantity][0]:4.1f} ± {error}'
        if molecule in adsorbate_config[surface]:
            final_surface_molecule_nice_labels += [f"{adsorbate_config[surface][molecule]} {molecule_label} on {system_label}"]
        else:
            final_surface_molecule_nice_labels += [f"{molecule_label} on {system_label}"]
            # cc_eint_df_dict[f'{surface}_{molecule}'][quantity + 'Error'] = final_cc_eint_list[surface][molecule][quantity][1]

df = pd.DataFrame(cc_eint_df_dict).transpose()
df = df.round(1)
df.columns = ['MP2bulk','deltaCC','deltaBasis','deltaCore','final']
df.index = final_surface_molecule_nice_labels


final_eint_str = df.to_latex(escape=False)
final_eint_str = final_eint_str.replace('&    0.0 ±  0.0','&    -')
preamble_str = r"""\begin{table}
\caption{\label{table:final_eint} The contributions to $E_\textrm{int}$ (in meV) for the molecule-surface systems studied within this work.}
\begin{tabular}{lrrrrr}
\toprule
System &         $E_\textrm{int}^\textrm{bulk MP2}$ &        $\Delta$CC &    $\Delta_\textrm{Basis}$ &     $\Delta_\textrm{Core}$ &           Final $E_\textrm{int}$ \\
\midrule
"""


final_eint_str = preamble_str + '\n'.join(final_eint_str.splitlines()[4:]) + '\n\\end{table}'

with open("Data/final_eint_table.tex", "w") as text_file:
    text_file.write(final_eint_str)

In [35]:
# Table comparing the LNO-CCSD(T) estimate against the CCSD(T) estimate for all of the systems

final_surface_molecule_labels = [f'{surface}_{molecule}' for surface in final_cc_eint_list for molecule in final_cc_eint_list[surface]]
final_surface_molecule_nice_labels = []

deltacc_df_dict = {surface_molecule: {} for surface_molecule in final_surface_molecule_labels}

mad_deltacc_dict = {x: [] for x in ['Canonical MP2','LMP2','LMP2 Error','Canonical CCSD(T)','LNO-CCSD(T)','LNO-CCSD(T) Error','Canonical DeltaCC','DeltaCC','DeltaCC Error']}

for surface in final_cc_eint_list:
    # Getting the crystal system and molecule name
    if 'TiO2_rutile' in surface:
        system_label = r'\ce{TiO2} rutile (110)'
    elif 'TiO2_anatase' in surface:
        system_label = r'\ce{TiO2} anatase (101)'
    elif 'MgO' in surface:
        system_label = r'\ce{MgO} (001)'
    for molecule in final_cc_eint_list[surface]:
        molecule_label = r'\ce{' + molecule.split('_')[0] + '}'
        try:
            eint_ene_dict = {
                'Canonical MP2': ene_list[surface][molecule]['CMP2_DZTZ'][0],
                'LMP2': ene_list[surface][molecule]['LMP2_DZTZ'][0],
                'LMP2 Error': abs(ene_list[surface][molecule]['LMP2_DZTZ'][0] - ene_list[surface][molecule]['CMP2_DZTZ'][0]),
                'Canonical CCSD(T)': ene_list[surface][molecule]['CCCSDT_DZTZ'][0],
                'LNO-CCSD(T)': ene_list[surface][molecule]['LCCSDT_DZTZ'][0],
                'LNO-CCSD(T) Error': abs(ene_list[surface][molecule]['LCCSDT_DZTZ'][0] - ene_list[surface][molecule]['CCCSDT_DZTZ'][0]),
                'Canonical DeltaCC': ene_list[surface][molecule]['CCCSDT_DZTZ'][0] - ene_list[surface][molecule]['CMP2_DZTZ'][0],
                'DeltaCC': ene_list[surface][molecule]['LCCSDT_DZTZ'][0] - ene_list[surface][molecule]['LMP2_DZTZ'][0],
                'DeltaCC Error': abs((ene_list[surface][molecule]['LCCSDT_DZTZ'][0] - ene_list[surface][molecule]['LMP2_DZTZ'][0])- (ene_list[surface][molecule]['CCCSDT_DZTZ'][0] - ene_list[surface][molecule]['CMP2_DZTZ'][0]))
            }
            if molecule != 'CO2_3':
                mad_deltacc_dict['LMP2 Error'] += [eint_ene_dict['LMP2 Error']]
                mad_deltacc_dict['LNO-CCSD(T) Error'] += [eint_ene_dict['LNO-CCSD(T) Error']]
                mad_deltacc_dict['DeltaCC Error'] += [eint_ene_dict['DeltaCC Error']]
                mad_deltacc_dict['Canonical DeltaCC'] += [0]
                mad_deltacc_dict['DeltaCC'] += [0]
                mad_deltacc_dict['Canonical MP2'] += [0]
                mad_deltacc_dict['LMP2'] += [0]
                mad_deltacc_dict['Canonical CCSD(T)'] += [0]
                mad_deltacc_dict['LNO-CCSD(T)'] += [0]
        except:
            print(f'{surface} {molecule} dont work')
            eint_ene_dict = {x:0 for x in ['Canonical MP2','LMP2','LMP2 Error','Canonical CCSD(T)','LNO-CCSD(T)','LNO-CCSD(T) Error','Canonical DeltaCC','DeltaCC','DeltaCC Error']}

        deltacc_df_dict[f'{surface}_{molecule}'] = eint_ene_dict
        if molecule in adsorbate_config[surface]:
            final_surface_molecule_nice_labels += [f"{adsorbate_config[surface][molecule]} {molecule_label} on {system_label}"]
        else:
            final_surface_molecule_nice_labels += [f"{molecule_label} on {system_label}"]
            # cc_eint_df_dict[f'{surface}_{molecule}'][quantity + 'Error'] = final_cc_eint_list[surface][molecule][quantity][1]

df = pd.DataFrame(deltacc_df_dict).transpose()
# Add a row to the DataFrame with the mean absolute deviation (MAD) of the errors
df.loc['MAD'] = [np.mean(mad_deltacc_dict[x]) for x in mad_deltacc_dict] # = df.abs().mean()
df.columns = [r'\rotatebox{90}{' + x + '}' for x in ['C-MP2','L-MP2','L-MP2 Err.','C-CCSD(T)','LNO-CCSD(T)','LNO-CCSD(T) Err.',r'C-$\Delta$CC',r'L-$\Delta$CC',r'L-$\Delta$CC Err.']]
df = df.round(1)

df.index = final_surface_molecule_nice_labels + ['MAD']

deltacc_local_errors = df.to_latex(escape=False)

preamble_str = r"""\begin{table}
\caption{\label{table:deltacc_lno_errors} Comparing canonical (C-)MP2, canonical CCSD(T) and canonical $\Delta$CC against their local variants (i.e., LMP2, LNO-CCSD(T), L-$\Delta$CC). This was compared for the first cluster generated by the SKZCAM protocol for MgO in Fig. XX, while it corresponds to cluster ${-}2$ for the \ce{TiO2} surfaces.}
\centerline{
\scalebox{1}{
"""


deltacc_local_errors = preamble_str + '\n'.join(deltacc_local_errors.splitlines()) + '\n}}\n\\end{table}'

with open("Data/deltacc_lno_errors_table.tex", "w") as text_file:
    text_file.write(deltacc_local_errors)

MgO NO_1 dont work
MgO NO_2 dont work
MgO NO_3 dont work
MgO NO_4 dont work
MgO NO_5 dont work


In [36]:
# Table comparing the LNO-CCSD(T) estimate against the CCSD(T) estimate for all of the systems

final_surface_molecule_labels = [f'{surface}_{molecule}' for surface in final_cc_eint_list for molecule in final_cc_eint_list[surface]]
final_surface_molecule_nice_labels = []

deltacc_size_df_dict = {surface_molecule: {} for surface_molecule in final_surface_molecule_labels}

for surface in final_cc_eint_list:
    # Getting the crystal system and molecule name
    if 'TiO2_rutile' in surface:
        system_label = r'\ce{TiO2} rutile (110)'
    elif 'TiO2_anatase' in surface:
        system_label = r'\ce{TiO2} anatase (101)'
    elif 'MgO' in surface:
        system_label = r'\ce{MgO} (001)'
    for molecule in final_cc_eint_list[surface]:
        molecule_label = r'\ce{' + molecule.split('_')[0] + '}'
        try:
            if surface == 'MgO':
                deltacc_ene_dict = {
                    '1': ene_list[surface][molecule]['LCCSDT_DZTZ'][0] - ene_list[surface][molecule]['LMP2_DZTZ'][0],
                    '2': ene_list[surface][molecule]['LCCSDT_DZTZ'][1] - ene_list[surface][molecule]['LMP2_DZTZ'][1],
                    '3': ene_list[surface][molecule]['LCCSDT_DZTZ'][2] - ene_list[surface][molecule]['LMP2_DZTZ'][2],
                }
                deltacc_ene_dict['Max Deviation'] = np.max([abs(deltacc_ene_dict['1'] - deltacc_ene_dict['2']),abs(deltacc_ene_dict['1'] - deltacc_ene_dict['3'])])
            elif 'TiO2' in surface:
                deltacc_ene_dict = {
                    '1': ene_list[surface][molecule]['LCCSDT_DZTZ'][2] - ene_list[surface][molecule]['LMP2_DZTZ'][2],
                    '2': ene_list[surface][molecule]['LCCSDT_DZTZ'][3] - ene_list[surface][molecule]['LMP2_DZTZ'][3],
                    '3': 0
                }
                deltacc_ene_dict['Max Deviation'] = abs(deltacc_ene_dict['1'] - deltacc_ene_dict['2'])

        except:
            print(f'{surface} {molecule} dont work')
            deltacc_ene_dict = {x:0 for x in ['1','2','3','Max Deviation']}

        deltacc_size_df_dict[f'{surface}_{molecule}'] = deltacc_ene_dict
        if molecule in adsorbate_config[surface]:
            final_surface_molecule_nice_labels += [f"{adsorbate_config[surface][molecule]} {molecule_label} on {system_label}"]
        else:
            final_surface_molecule_nice_labels += [f"{molecule_label} on {system_label}"]

df = pd.DataFrame(deltacc_size_df_dict).transpose()
# Add a row to the DataFrame with the mean absolute deviation (MAD) of the errors
# df.loc['MAD'] = [np.mean(mad_deltacc_dict[x]) for x in mad_deltacc_dict] # = df.abs().mean()
df.columns = ['1','2','3','Max Deviation']
df = df.round(1)

df.index = final_surface_molecule_nice_labels # + ['MAD']

deltacc_size_errors = df.to_latex(escape=False)

preamble_str = r"""\begin{table}
\caption{\label{table:deltacc_size_errors} Comparison of L-$\Delta$CC for the series of clusters by the SKZCAM protocol.}
\centerline{
\scalebox{1}{
"""


deltacc_size_errors = preamble_str + '\n'.join(deltacc_size_errors.splitlines()) + '\n}}\n\\end{table}'

with open("Data/deltacc_size_errors_table.tex", "w") as text_file:
    text_file.write(deltacc_size_errors)

MgO NO_1 dont work
MgO NO_2 dont work
MgO NO_3 dont work
MgO NO_4 dont work
MgO NO_5 dont work


In [37]:
final_eads = {y: {x: [0,0] for x in overall_systems[y]} for y in ['MgO','TiO2_rutile','TiO2_anatase']}

for system in ['MgO','TiO2_anatase','TiO2_rutile']: #['MgO','TiO2_rutile','TiO2_anatase']:
    for i in overall_systems[system]:
        final_eads[system][i][0] = final_cc_eint_list[system][i]['final_overall'][0] + erel_dict[system][i]['02_revPBE-D4'] 
        final_eads[system][i][1] = np.sqrt(final_cc_eint_list[system][i]['final_overall'][1]**2 + erel_error_dict[system][i]['02_revPBE-D4']**2)

        print(system,i,final_eads[system][i][0],final_eads[system][i][1])

MgO CH4 -124.43605385931204 8.92903615556795
MgO C2H6 -179.58005840613006 15.455863705854894
MgO CO -201.86034658849144 9.925243448074927
MgO N2O_1 -257.099856920388 9.613638488220069
MgO N2O_2 -159.9768033748861 21.12463558783191
MgO C6H6 -406.2249062264371 45.732657203417
MgO H2O -592.9997440786382 33.078178412713264
MgO NH3 -571.5383624485349 26.176586786225183
MgO CO2_2 -298.1091608273723 13.735967612060255
MgO CO2_3 -790.6240242074668 248.71443641946465
MgO NO_1 83.11858673404049 40.71972650348138
MgO NO_2 -27.470168502128637 19.485835811669215
MgO NO_3 13.737419554339475 21.176848897289258
MgO NO_4 -94.16783755341508 11.981701350373674
MgO NO_5 50.16132678104979 20.216312261721068
MgO NO_6 -219.3624703899469 15.058820452656223
MgO CH3OH_1 -648.2983981350724 38.69755282230082
MgO CH3OH_2 -466.7981087637148 21.905581444275427
MgO CH3OH_4 -786.4968371756587 24.966749129588628
MgO H2O_4 -698.5369482357885 19.250406862702892
MgO CH4_ML -154.21011908117836 12.90932681952774
MgO C2H6_ML

In [38]:
fig, axs = plt.subplots(figsize=(2.4, 2),dpi=1200,constrained_layout=True)


# Bar graph of different functionals in funct_line: Loop over the functionals and plot the MAD
axs.barh([1-0.2],ene_list['MgO']['CO']['MP2_DZTZ'][0],0.4,color=color_dict['blue'],label='MP2')
axs.barh([1+0.3],ene_list['MgO']['CO']['CCCSDT_DZTZ'][0] ,0.4,color=color_dict['yellow'],label='CCSD(T)') #edgecolor='black',alpha=1)

axs.barh([2.5-0.2],ene_extrap_list['MgO']['CO']['MP2_DZTZ'][-1],0.4,color=color_dict['blue'],alpha=1)
axs.barh([2.5+0.3],ene_extrap_list['MgO']['CO']['MP2_DZTZ'][-1] + ene_list['MgO']['CO']['CCCSDT_DZTZ'][0]  - ene_list['MgO']['CO']['MP2_DZTZ'][0],0.4,color=color_dict['yellow'],edgecolor='black',alpha=1, label='SKZCAM')

# axs.bar([2],[-10],width=0.4,color='none',edgecolor='black',alpha=1, label=r'+$\Delta$CC')


axs.set_yticks([1.05,2.55])
axs.set_yticklabels([]) #['Smallest Cluster','Bulk Limit'])

axs.set_xlabel(r'$E_\textrm{int}$ (meV)')
axs.legend(fontsize=7)
# axs[2].set_ylim([10,0.5])
axs.set_ylim([3.3,0.3])
axs.set_xlim([-140,-220])

# axs.xaxis.set_ticks_position('top')
# axs.xaxis.set_label_position('top')

plt.savefig('SI_deltacc.png')

# Getting the final Hads

In [39]:
# Getting the final Hads values for the various systems

# final_cc_eint_list = {y: {x: {z:[0,0] for z in ['MP2_bulk','deltaCC','deltaBasis','deltaCore','final']} for x in overall_systems[y]} for y in ['MgO','TiO2_rutile','TiO2_anatase']}

final_hads = {y: {x: [0,0] for x in overall_systems[y]} for y in ['MgO','TiO2_rutile','TiO2_anatase']}

for system in ['MgO','TiO2_anatase','TiO2_rutile']: #['MgO','TiO2_rutile','TiO2_anatase']:
    for i in overall_systems[system]:
        final_hads[system][i][0] = final_cc_eint_list[system][i]['final_overall'][0] + erel_dict[system][i]['02_revPBE-D4'] + ethermal[system][i]['02_revPBE-D4']['Ecor']
        final_hads[system][i][1] = np.sqrt(final_cc_eint_list[system][i]['final_overall'][1]**2 + erel_error_dict[system][i]['02_revPBE-D4']**2)

        print(system,i,final_hads[system][i][0],final_hads[system][i][1])


MgO CH4 -109.34356751779372 8.92903615556795
MgO C2H6 -175.24503658484662 15.455863705854894
MgO CO -184.22442223073887 9.925243448074927
MgO N2O_1 -262.35466262209513 9.613638488220069
MgO N2O_2 -161.54615535543988 21.12463558783191
MgO C6H6 -409.5752577202924 45.732657203417
MgO H2O -533.3564856437537 33.078178412713264
MgO NH3 -522.3702333072004 26.176586786225183
MgO CO2_2 -307.10876290262877 13.735967612060255
MgO CO2_3 -757.9952996506869 248.71443641946465
MgO NO_1 92.53405111017814 40.71972650348138
MgO NO_2 -24.274498606794662 19.485835811669215
MgO NO_3 25.619931977822578 21.176848897289258
MgO NO_4 -91.56933795698515 11.981701350373674
MgO NO_5 50.19166969522321 20.216312261721068
MgO NO_6 -172.86234682329615 15.058820452656223
MgO CH3OH_1 -631.70463506925 38.69755282230082
MgO CH3OH_2 -467.03082128775617 21.905581444275427
MgO CH3OH_4 -747.681125679233 24.966749129588628
MgO H2O_4 -625.2974946514004 19.250406862702892
MgO CH4_ML -134.63439631134347 12.90932681952774
MgO C2H6

# Experiments

In [40]:
# Compile a list of the experimental adsorption enthalpies
mol_hads = {
    'MgO': { # [ Temperature, degrees of freedom = 3N-6 generalor 3N-5 if linear ] 
    'CO': [(-16.9 + -17.6)/2*kjmol_to_meV, 1.6*2.3*ethermal['MgO']['CO']['02_revPBE-D4']['RT'],'Campbell and Sellers (C&S) Hads with nu = 13.85'], #Temperature given as average between studies by Dohnalek et al. and Wichtendahl et al.
    'H2O': [-50.2*kjmol_to_meV,11.7*kjmol_to_meV, 'Sauer estimate at 100K'],  
    'CH3OH': [-90.5*kjmol_to_meV, 1.6*2.3*ethermal['MgO']['CH3OH_4']['02_revPBE-D4']['RT'],'C&S Hads with nu = 15.3'], 
    'CO2_2': [-9.4*kcalmol_to_meV + 0.5*ethermal['MgO']['CO2_2']['02_revPBE-D4']['RT'] - 2.3*ethermal['MgO']['CO2_2']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['MgO']['CO2_2']['02_revPBE-D4']['RT'], 'Eact Meixner. Convert nu = 13 to nu = 14'], # Meixner gives a case of physisorption for CO2 on MgO to be 
    'CO2_3': [(-51.7 + -67.7)/2*kjmol_to_meV + 0.5*ethermal['MgO']['CO2_3']['02_revPBE-D4']['RT'] - 2.3*ethermal['MgO']['CO2_3']['02_revPBE-D4']['RT'],np.sqrt(1.6*2.3*ethermal['MgO']['CO2_3']['02_revPBE-D4']['RT']**2 + (8*kjmol_to_meV)**2), 'Eact half of alpha and beta peaks. Convert nu = 13 to nu = 14'], # Meixner gives a case of physisorption for CO2 on MgO to be 
    'NO': [-21.3*kjmol_to_meV, 1.6*2.3*ethermal['MgO']['NO_1']['02_revPBE-D4']['RT'],'C&S Hads with nu = 14.0'],
    'N2O2': [-24.1*kjmol_to_meV, 1.6*2.3*ethermal['MgO']['NO_1']['02_revPBE-D4']['RT'],'C&S Hads with nu = 14.0'],
    'CH4' : [-11.1*kjmol_to_meV + 0.5*ethermal['MgO']['CH4']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['MgO']['CH4']['02_revPBE-D4']['RT'],  'Dohnalek original E0 with nu=13.1'],
    'C2H6': [-21.3*kjmol_to_meV + 0.5*ethermal['MgO']['C2H6']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['MgO']['C2H6']['02_revPBE-D4']['RT'], 'Dohnalek original E0 with nu=14.9'],
    'CH4_ML': [-12.63*kjmol_to_meV + 0.5*ethermal['MgO']['CH4']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['MgO']['CH4']['02_revPBE-D4']['RT'] , 'Dohnalek original E0 with nu=13.1']  ,
    'C2H6_ML': [-22.76*kjmol_to_meV + 0.5*ethermal['MgO']['C2H6']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['MgO']['C2H6']['02_revPBE-D4']['RT'], 'Dohnalek original E0 with nu=14.9'],
    'C6H6': [((-51.3+-42.8)/2*kjmol_to_meV),1.6*2.3*ethermal['MgO']['C6H6']['02_revPBE-D4']['RT'],'C&S Hads with nu = 15.1'],
    'H2O_4': [-67.9*kjmol_to_meV,1.6*2.3*ethermal['MgO']['H2O_4']['02_revPBE-D4']['RT'], 'C&S Hads with nu = 14.5'],  
    'CH3OH_4': [(-90.5-83.7)/2*kjmol_to_meV,1.6*2.3*ethermal['MgO']['CH3OH_4']['02_revPBE-D4']['RT'], 'C&S Hads with nu = 15.3'], 
    'NH3': [-(13.4+15)/2*kcalmol_to_meV + 0.5*ethermal['MgO']['NH3']['02_revPBE-D4']['RT'] ,2*1.6*2.3*ethermal['MgO']['NH3']['02_revPBE-D4']['RT'],'Meixner study Eact nu=13 and nu=15'],
    'N2O': [-230 + 0.5*ethermal['MgO']['N2O_1']['02_revPBE-D4']['RT'] - 2.3*ethermal['MgO']['N2O_1']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['MgO']['N2O_1']['02_revPBE-D4']['RT'],'Eact with nu=13 corrected to nu = 14']},
    'TiO2_rutile': {
        'CH4': [-25*kjmol_to_meV + 0.5*ethermal['TiO2_rutile']['CH4']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['TiO2_rutile']['CH4']['02_revPBE-D4']['RT'],'Dohnalek Eact with nu = 7.9 x 10^14'],
        'H2O': [-(92.6)*kjmol_to_meV,1.6*2.3*ethermal['TiO2_rutile']['H2O']['02_revPBE-D4']['RT'],'C&S Hads with nu = 14.7'],
        'CO2': [-48.3*kjmol_to_meV,1.6*2.3*ethermal['TiO2_rutile']['CO2_2']['02_revPBE-D4']['RT'],'C&S Hads with nu = 13.6'],
        'CH3OH': [-117*kjmol_to_meV,1.6*2.3*ethermal['TiO2_rutile']['CH3OH']['02_revPBE-D4']['RT'],'C&S Hads with nu = 15.3']
    },
    'TiO2_anatase': {
        'NH3': [-26*kcalmol_to_meV + 0.5*ethermal['TiO2_anatase']['NH3']['02_revPBE-D4']['RT'] - 2.3*ethermal['TiO2_anatase']['NH3']['02_revPBE-D4']['RT'],1.6*2.3*ethermal['TiO2_anatase']['NH3']['02_revPBE-D4']['RT'],'Eact with nu = 13'],
        'H2O': [-(76.9)*kjmol_to_meV,1.6*2.3*ethermal['TiO2_anatase']['H2O']['02_revPBE-D4']['RT'],'C&S Hads with nu = 14.6']
    }
}

In [156]:
# Convert 'expt_hads.csv' csv file to pandas DataFrame

expt_hads_df = pd.read_csv('expt_data.csv',dtype=object)
expt_hads_df = expt_hads_df.reset_index(drop=True)
# def convert_df_to_latex_input(
#     df,
#     start_input = '\\begin{table}\n',
#     end_input = '\n\\end{table}',
#     label = "tab:default",
#     caption = "This is a table",
#     replace_input = {},
#     df_latex_skip = 0,
#     adjustbox = 0,
#     scalebox = False,
#     multiindex_sep = "",
#     filename = "./table.tex",
#     column_format = None
# ):


# Convert to latex table
convert_df_to_latex_input(
df = expt_hads_df,
start_input = '\\begin{turnpage}\n\\begin{table}\n',
end_input = '\n\\end{table}\n\\end{turnpage}',
label = "tab:expt_hads",
caption = "Experimental adsorption enthalpies for various systems.",
replace_input = {
    "+-": r"$\pm$"
},
adjustbox = 1.6,
column_format = "lrrrrrrp{12cm}",
index = False,
filename = 'Tables/expt_hads.tex')

expt_and_autoskzcam_hads_df = expt_hads_df.copy()

expt_hads_df.columns = ['Surface','Adsorbate','Temperature','log(ν)', 'Eact', 'Hads', 'Error', 'Details']
expt_hads_df = expt_hads_df.replace({
    r'\\ce\{|\}|\(001\)|\(110\)|\(101\)|\(\)': '',
    r'\$\\to\$': '->',
    r'\+-': '±',
    r'MgO': 'MgO',
    r'TiO2 rutile': 'rutile',
    r'TiO2 anatase': 'anatase',
}, regex=True)

expt_hads_df = expt_hads_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [157]:
final_hads_compare = {
    'MgO': {
        'CH4': final_hads['MgO']['CH4'],
        'C2H6': final_hads['MgO']['C2H6'],
        'CO': final_hads['MgO']['CO'],
        'N2O': final_hads['MgO']['N2O_1'],
        'C6H6': final_hads['MgO']['C6H6'],
        'H2O': final_hads['MgO']['H2O'],
        'NH3': final_hads['MgO']['NH3'],
        "`Physisorbed' CO2": final_hads['MgO']['CO2_2'],
        'Chemisorbed CO2': final_hads['MgO']['CO2_3'],
        'Cluster NO': final_hads['MgO']['NO_6'],
        'Cluster H2O': final_hads['MgO']['H2O_4'],
        'Cluster CH3OH': final_hads['MgO']['CH3OH_4'],
        'Monolayer CH4': final_hads['MgO']['CH4_ML'],
        'Monolayer C2H6': final_hads['MgO']['C2H6_ML'],
    },
    'TiO2 rutile': {
        'CH4': final_hads['TiO2_rutile']['CH4'],
        'H2O': final_hads['TiO2_rutile']['H2O'],
        'CO2': final_hads['TiO2_rutile']['CO2_2'],
        'CH3OH': final_hads['TiO2_rutile']['CH3OH']
    },
    'TiO2 anatase': {
        'NH3': final_hads['TiO2_anatase']['NH3'],
        'H2O': final_hads['TiO2_anatase']['H2O']
    }
}

expt_hads_list = []
autoskzcam_hads_list = []
delta_min_list = []

for df_idx in expt_hads_df.index:
    row = expt_hads_df.loc[df_idx]
    expt_hads_list += [f"{row['Hads']} ± {row['Error']}"]
    # Create a Delta_min value for the minimum difference between the experimental and calculated adsorption enthalpies
    expt_hads_lower_bound = int(row['Hads']) - int(row['Error'])
    expt_hads_upper_bound = int(row['Hads']) + int(row['Error'])
    autoskzcam_hads_lower_bound = final_hads_compare[row['Surface']][row['Adsorbate']][0] - final_hads_compare[row['Surface']][row['Adsorbate']][1]
    autoskzcam_hads_upper_bound = final_hads_compare[row['Surface']][row['Adsorbate']][0] + final_hads_compare[row['Surface']][row['Adsorbate']][1]
    if max(expt_hads_lower_bound, autoskzcam_hads_lower_bound) <= min(expt_hads_upper_bound, autoskzcam_hads_upper_bound):
        delta_min = 0
    else:
        delta_min = min(abs(expt_hads_upper_bound - autoskzcam_hads_lower_bound), abs(autoskzcam_hads_upper_bound - expt_hads_lower_bound))
    delta_min_list += [round(delta_min)]

    autoskzcam_hads_list += [f"{final_hads_compare[row['Surface']][row['Adsorbate']][0]:.0f} ± {final_hads_compare[row['Surface']][row['Adsorbate']][1]:.0f}"]

# Make a new DataFrame with the experimental and calculated adsorption enthalpies using the same Surface and Adsorbate labels
# Only keep the surface and adsorbate columns
expt_and_autoskzcam_hads_df = expt_and_autoskzcam_hads_df[['Surface','Adsorbate']]
expt_and_autoskzcam_hads_df['Expt. Hads'] = expt_hads_list
expt_and_autoskzcam_hads_df['autoSKZCAM Hads'] = autoskzcam_hads_list
expt_and_autoskzcam_hads_df['$\Delta_{\textrm{min}}$'] = delta_min_list

# Convert to latex table
convert_df_to_latex_input(
df = expt_and_autoskzcam_hads_df,
start_input = '\\begin{table}\n',
end_input = '\n\\end{table}',
label = "tab:expt_autoskzcam_hads",
caption = "Experimental and autoSKZCAM adsorption enthalpies for various systems.",
replace_input = {
    "+-": r"$\pm$"
},
index = False,
filename = 'Tables/expt_and_autoskzcam_hads.tex')


In [122]:
expt_hads_df['Surface']

0              MgO
1              MgO
2              MgO
3              MgO
4              MgO
5              MgO
6              MgO
7              MgO
8              MgO
9              MgO
10             MgO
11             MgO
12             MgO
13             MgO
14     TiO2 rutile
15     TiO2 rutile
16     TiO2 rutile
17     TiO2 rutile
18    TiO2 anatase
19    TiO2 anatase
Name: Surface, dtype: object

In [97]:
expt_hads_df

,Surface,Adsorbate,Temperature,log(ν),Eact,Hads,Error,Details
0,MgO,CH4,47,13.1 ± 2.0,-115,-113,19,Dilute limit $E_\textrm{act$ estimate by Tait ...
1,MgO,C2H6,75,14.9 ± 2.0,-221,-218,30,Dilute limit $E_\textrm{act$ estimate by Tait ...
2,MgO,CO,61,13.8 ± 1.6,-,-179,19,Average of the $H_\textrm{ads$ re-analysis by ...
3,MgO,N2O,77,13.0 -> 14.0 ± 2.0,-230,-242,31,$E_\textrm{act$ measured by Lian \etal{~\cite{...
4,MgO,C6H6,162,15.1 ± 1.6,-,-488,68,Average taken of $H_\textrm{ads$ re-analyzed b...
5,MgO,H2O,203,-,-,-520,121,$H_\textrm{ads$ from Ferry \etal{~\cite{alessi...
6,MgO,NH3,160,13.0 -> 14.0 ± 2.0,-581,-606,63,$E_\textrm{act$ measurement from Arthur \etal{...
7,MgO,`Physisorbed' CO2,120,13.0 -> 14.0 ± 2.0,-408,-426,48,$E_\textrm{act$ measurement from Meixner \etal...
8,MgO,Chemisorbed CO2,230,13.0 -> 14.0 ± 2.0,-619,-654,123,$E_\textrm{act$ measurement from Chakradhar an...
9,MgO,Monolayer CH4,47,13.1 ± 2.0,-131,-129,19,Monolayer $E_\textrm{act$ estimate by Tait \et...


In [47]:
final_hads

{'MgO': {'CH4': [-109.34356751779372, 8.92903615556795],
  'C2H6': [-175.24503658484662, 15.455863705854894],
  'CO': [-184.22442223073887, 9.925243448074927],
  'N2O_1': [-262.35466262209513, 9.613638488220069],
  'N2O_2': [-161.54615535543988, 21.12463558783191],
  'C6H6': [-409.5752577202924, 45.732657203417],
  'H2O': [-533.3564856437537, 33.078178412713264],
  'NH3': [-522.3702333072004, 26.176586786225183],
  'CO2_2': [-307.10876290262877, 13.735967612060255],
  'CO2_3': [-757.9952996506869, 248.71443641946465],
  'NO_1': [92.53405111017814, 40.71972650348138],
  'NO_2': [-24.274498606794662, 19.485835811669215],
  'NO_3': [25.619931977822578, 21.176848897289258],
  'NO_4': [-91.56933795698515, 11.981701350373674],
  'NO_5': [50.19166969522321, 20.216312261721068],
  'NO_6': [-172.86234682329615, 15.058820452656223],
  'CH3OH_1': [-631.70463506925, 38.69755282230082],
  'CH3OH_2': [-467.03082128775617, 21.905581444275427],
  'CH3OH_4': [-747.681125679233, 24.966749129588628],
  '

In [45]:
expt_hads_df[r'$H_\textrm{ads}$ (meV)']

0      -113
1      -218
2      -179
3      -242
4      -488
5      -520
6      -606
7      -426
8      -654
9      -129
10     -233
11     -235
12     -704
13     -903
14     -245
15     -501
16     -930
17    -1213
18     -797
19    -1162
Name: $H_\textrm{ads}$ (meV), dtype: object

In [43]:
expt_hads

NameError: name 'expt_hads' is not defined

In [52]:
final_hads

{'MgO': {'CH4': [-109.34356751779372, 8.92903615556795],
  'C2H6': [-175.24503658484662, 15.455863705854894],
  'CO': [-184.22442223073887, 9.925243448074927],
  'N2O_1': [-262.35466262209513, 9.613638488220069],
  'N2O_2': [-161.54615535543988, 21.12463558783191],
  'C6H6': [-409.5752577202924, 45.732657203417],
  'H2O': [-533.3564856437537, 33.078178412713264],
  'NH3': [-522.3702333072004, 26.176586786225183],
  'CO2_2': [-307.10876290262877, 13.735967612060255],
  'CO2_3': [-757.9952996506869, 248.71443641946465],
  'NO_1': [92.53405111017814, 40.71972650348138],
  'NO_2': [-24.274498606794662, 19.485835811669215],
  'NO_3': [25.619931977822578, 21.176848897289258],
  'NO_4': [-91.56933795698515, 11.981701350373674],
  'NO_5': [50.19166969522321, 20.216312261721068],
  'NO_6': [-172.86234682329615, 15.058820452656223],
  'CH3OH_1': [-631.70463506925, 38.69755282230082],
  'CH3OH_2': [-467.03082128775617, 21.905581444275427],
  'CH3OH_4': [-747.681125679233, 24.966749129588628],
  '

In [51]:
mol_temp_and_dof_dict

{'MgO': {'CO': [61, 1],
  'H2O': [100, 3],
  'CH3OH': [296, 12],
  'CO2_2': [130, 4],
  'CO2_3': [230, 4],
  'NO': [84, 1],
  'N2O2': [84, 6],
  'CH4': [47, 9],
  'C2H6': [75, 18],
  'C6H6': [175, 30],
  'H2O_4': [235, 30],
  'CH3OH_4': [296, 66],
  'NH3': [120, 6],
  'N2O': [77, 4],
  'CH4_ML': [47, 54],
  'C2H6_ML': [75, 90]},
 'TiO2_rutile': {'CH4': [85, 9],
  'H2O': [303, 3],
  'CO2': [177, 4],
  'CH3OH': [370, 12]},
 'TiO2_anatase': {'NH3': [360, 6], 'H2O': [257, 3]}}

In [50]:
ethermal['MgO']['N2O_1']['02_revPBE-D4']['RT']

6.635344359657254

In [40]:
exp_ene_list = {
    'CH4' : mol_hads['MgO']['CH4'],
    'C2H6': mol_hads['MgO']['C2H6'],
    'CO': mol_hads['MgO']['CO'],
    'N2O': mol_hads['MgO']['N2O'],
    'C6H6': mol_hads['MgO']['C6H6'],
    'H2O': mol_hads['MgO']['H2O'],
    'NH3': mol_hads['MgO']['NH3'],
    'CO2': mol_hads['MgO']['CO2_3'], 
    'NO': mol_hads['MgO']['NO'],
    'H2O_clus': mol_hads['MgO']['H2O_4'] ,
    'CH3OH_clus': mol_hads['MgO']['CH3OH_4'],
    'CH4_ml': mol_hads['MgO']['CH4_ML'],
    'C2H6_ml': mol_hads['MgO']['C2H6_ML'],
    'rutile_CH4': mol_hads['TiO2_rutile']['CH4'][:2],
    'rutile_CH3OH': mol_hads['TiO2_rutile']['CH3OH'][:2],
    'rutile_CO2': mol_hads['TiO2_rutile']['CO2'][:2],
    'rutile_H2O': mol_hads['TiO2_rutile']['H2O'][:2],
    'anatase_H2O': mol_hads['TiO2_anatase']['H2O'][:2],
    'anatase_NH3': mol_hads['TiO2_anatase']['NH3'][:2]
}

skzcam_ene_list = {
    'CH4' : final_hads['MgO']['CH4'],
    'C2H6': final_hads['MgO']['C2H6'],
    'CO': final_hads['MgO']['CO'],
    'N2O': final_hads['MgO']['N2O_1'],
    'C6H6': final_hads['MgO']['C6H6'],
    'H2O': final_hads['MgO']['H2O'],
    'NH3': final_hads['MgO']['NH3'],
    'CO2': final_hads['MgO']['CO2_3'], 
    'NO': final_hads['MgO']['NO_6'],
    'H2O_clus': final_hads['MgO']['H2O_4'] ,
    'CH3OH_clus': final_hads['MgO']['CH3OH_4'],
    'CH4_ml': final_hads['MgO']['CH4_ML'],
    'C2H6_ml': final_hads['MgO']['C2H6_ML'],
    'rutile_CH4': final_hads['TiO2_rutile']['CH4'],
    'rutile_CH3OH': final_hads['TiO2_rutile']['CH3OH'],
    'rutile_CO2': final_hads['TiO2_rutile']['CO2_2'],
    'rutile_H2O': final_hads['TiO2_rutile']['H2O'],
    'anatase_H2O': final_hads['TiO2_anatase']['H2O'],
    'anatase_NH3': final_hads['TiO2_anatase']['NH3']
}

# Figure 2 - All molecules

In [41]:
fig, axs = plt.subplots(1,2,figsize=(6.69,2),dpi=1200, sharey=True, constrained_layout=True,gridspec_kw={'width_ratios':(1.85,1)})

xticklens = [1,1.8,2.98,3.87,4.73,5.625,6.335,6.95]



axs[0].grid(axis = "y",ls='--', lw=.5)


axs[0].errorbar(np.array(xticklens[:8])-0.1, [skzcam_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [skzcam_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],color=color_dict['blue'],markerfacecolor='none',capsize=2,fmt="o",alpha=1,label='SKZCAM')

axs[0].errorbar(np.array(xticklens[:8])+0.1, [exp_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [exp_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="o",alpha=1,color=color_dict['red'],markerfacecolor='none',label='Experiment')


axs[0].errorbar(np.array(xticklens[:8])-0.1, [skzcam_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [skzcam_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],color=color_dict['blue'],markerfacecolor='none',capsize=2,fmt="o",alpha=1)

axs[0].errorbar(np.array(xticklens[:8])+0.1, [exp_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [exp_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="o",alpha=1,color=color_dict['red'],markerfacecolor='none')

xticklens1 = [14.25,15.75]

axs[1].errorbar(np.array(xticklens1)-0.1, [skzcam_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [skzcam_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],color=color_dict['blue'],capsize=2,fmt="o",markerfacecolor='none', label='autoSKZCAM')

axs[1].errorbar(np.array(xticklens1)+0.1, [exp_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [exp_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="o",color=color_dict['red'],markerfacecolor='none', label='Experiment')



# axs[0].errorbar(np.array(xticklens[-3:])-0.1, [skzcam_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [skzcam_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],color=color_dict['blue'],capsize=2,fmt="o",alpha=1.0,markerfacecolor='none')

# axs[0].errorbar(np.array(xticklens[-3:])+0.1, [exp_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [exp_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],capsize=2,fmt="o",alpha=1.0,color=color_dict['red'],markerfacecolor='none')

# axs.errorbar([14,15], [skzcam_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [skzcam_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],color=color_dict['blue'],markerfacecolor='none',capsize=2,fmt="o",alpha=1)

# axs.errorbar([14,15], [exp_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [exp_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="o",alpha=1,color=color_dict['red'],markerfacecolor='none')

# CO on MgO

axs[0].bar([xticklens[2]-0.3, xticklens[5]-0.3], height = [89--379,-481--608] , color=color_dict['yellow'], bottom = [-379 + ethermal['MgO']['CO']['02_revPBE-D4']['Ecor'],-608 + ethermal['MgO']['H2O']['02_revPBE-D4']['Ecor']], width=0.15,alpha=0.7, label = 'cWFT literature')
axs[0].bar([xticklens[7]+0.3], height = [770--680] , color=color_dict['grey'], bottom = [-680+ ethermal['MgO']['CO2_2']['02_revPBE-D4']['Ecor']], width=0.15,alpha=0.7, label = 'DFT literature')
axs[0].annotate('770', xy=(xticklens[7]+0.3, 0), xytext=(xticklens[7]+0.3, -200),fontsize=7,horizontalalignment="center",arrowprops=dict(arrowstyle="->"))


axs[1].bar([xticklens[2]-0.3, xticklens[5]-0.3], height = [89--379,-481--608] , color=color_dict['yellow'], bottom = [-379 + ethermal['MgO']['CO']['02_revPBE-D4']['Ecor'],-608 + ethermal['MgO']['H2O']['02_revPBE-D4']['Ecor']], width=0.15,alpha=0.7, label = 'cWFT literature')
axs[1].bar([xticklens[7]+0.3], height = [770--680] , color=color_dict['grey'], bottom = [-680+ ethermal['MgO']['CO2_2']['02_revPBE-D4']['Ecor']], width=0.15,alpha=0.7, label = 'DFT literature')
axs[0].annotate('770', xy=(xticklens[7]+0.3, 0), xytext=(xticklens[7]+0.3, -200),fontsize=7,horizontalalignment="center",arrowprops=dict(arrowstyle="->"))

# axs.plot([xticklens[2]-0.2,xticklens[2]-0.2], [89,-379],color=color_dict['grey'],alpha=0.7,linewidth=5)


# axs.plot([xticklens[5]-0.2,xticklens[5]-0.2], [-481,-608],color=color_dict['grey'],alpha=0.7,linewidth=5)


axs[0].set_xticks(xticklens)
axs[0].set_xticklabels([r'CH\textsubscript{4}', r'C\textsubscript{2}H\textsubscript{6}','CO',r'N\textsubscript{2}O', r'C\textsubscript{6}H\textsubscript{6}',  r'H\textsubscript{2}O', r'NH\textsubscript{3}', r'CO\textsubscript{2}']) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

axs[0].set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (meV)')
# ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
# axs.set_xlabel(r'Molecule on MgO (001)')
axs[0].set_ylim([-1000,0])
axs[1].legend(frameon=True,loc='lower left')
axs[0].set_xlim([0.5,7.5])

axs[1].set_xticks(xticklens1)
axs[1].set_xticklabels([r'CH\textsubscript{4}',r'C\textsubscript{2}H\textsubscript{6}']) #[r'H\textsubscript{2}O', 

# axs[1].set_ylim([-700,0])


axs[1].grid(axis = "y",ls='--', lw=.5)

# axs[0].legend(frameon=False,loc='upper left')

axs[1].set_xlim([13.5,16.5])


axs[0].xaxis.tick_top()
axs[0].xaxis.set_label_position('top') 
axs[1].xaxis.tick_top()
axs[1].xaxis.set_label_position('top') 

plt.savefig('Fig_02a.png')

In [42]:
fig, axs = plt.subplots(1,2,figsize=(6.69,2.59),dpi=1200, sharey=True, constrained_layout=True,gridspec_kw={'width_ratios':(1.55,1)})

# ax2 = axs.twinx()

# def convert_ax2(axs):
#     y1, y2 = axs.get_ylim()
#     ax2.set_ylim(y1/43.3634, y2/43.3634)
#     ax2.figure.canvas.draw()    

# axs.callbacks.connect("ylim_changed", convert_ax2)  

xticklens = [7.98,9.05,10.15]

axs[1].errorbar(np.array(xticklens[-3:])-0.1, [skzcam_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [skzcam_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],color=color_dict['blue'],capsize=2,fmt="o",alpha=1.0,markerfacecolor='none')

axs[1].errorbar(np.array(xticklens[-3:])+0.1, [exp_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [exp_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],capsize=2,fmt="o",alpha=1.0,color=color_dict['red'],markerfacecolor='none')


axs[1].set_xticks(xticklens)
axs[1].set_xticklabels(['NO',r'H\textsubscript{2}O',r'CH\textsubscript{3}OH']) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

axs[0].set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (meV)')
# ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
# axs[1].legend(frameon=False,loc='lower left')


xticklens = [0.9,2,2.9,3.95,5.6,6.32]

axs[0].errorbar( np.array(xticklens[:4])-0.1, [skzcam_ene_list[x][0] for x in ['rutile_CH4','rutile_CO2','rutile_H2O', 'rutile_CH3OH']],   [skzcam_ene_list[x][1] for x in ['rutile_CH4','rutile_CO2','rutile_H2O', 'rutile_CH3OH']],color=color_dict['blue'],capsize=2,fmt="o",label='SKZCAM',markerfacecolor='none')

axs[0].errorbar( np.array(xticklens[:4])+0.1, [exp_ene_list[x][0] for x in ['rutile_CH4','rutile_CO2','rutile_H2O', 'rutile_CH3OH']],   [exp_ene_list[x][1] for x in ['rutile_CH4','rutile_CO2','rutile_H2O', 'rutile_CH3OH']],capsize=2,fmt="o",color=color_dict['red'],label='Experiment',markerfacecolor='none')


axs[0].errorbar(np.array(xticklens[-2:])-0.1, [skzcam_ene_list[x][0] for x in ['anatase_H2O','anatase_NH3']],   [skzcam_ene_list[x][1] for x in ['anatase_H2O','anatase_NH3']],color=color_dict['blue'],capsize=2,fmt="o",markerfacecolor='none')

axs[0].errorbar(np.array(xticklens[-2:])+0.1, [exp_ene_list[x][0] for x in ['anatase_H2O','anatase_NH3']],   [exp_ene_list[x][1] for x in ['anatase_H2O','anatase_NH3']],capsize=2,fmt="o",color=color_dict['red'],markerfacecolor='none')



axs[0].bar([xticklens[2]-0.3], height = [-954- -1270] , color=color_dict['yellow'], bottom = [-954 + ethermal['TiO2_rutile']['H2O']['02_revPBE-D4']['Ecor']], width=0.15, label = 'cWFT literature')
axs[0].bar([xticklens[2]+0.3], height = [-230 - -930] , color=color_dict['grey'], bottom = [-840 + ethermal['TiO2_rutile']['H2O']['02_revPBE-D4']['Ecor']], width=0.15, label = 'DFT literature')
# axs.annotate('770', xy=(xticklens[7]+0.2, 0), xytext=(xticklens[7]+0.2, -200),fontsize=7,horizontalalignment="center",arrowprops=dict(arrowstyle="->"))




axs[0].set_xticks(xticklens)
axs[0].set_xticklabels([r'CH\textsubscript{4}',r'CO\textsubscript{2}', r'H\textsubscript{2}O', r'CH\textsubscript{3}OH', ] + [r'H\textsubscript{2}O', r'NH\textsubscript{3}']) #[r'H\textsubscript{2}O', 
# axs[0].set_xticklabels([r'rutile CH\textsubscript{4}', r'rutile CH\textsubscript{3}OH', r'rutile CO\textsubscript{2}', r'rutile H\textsubscript{2}O'] + [r'anatase H\textsubscript{2}O', r'anatase NH\textsubscript{3}'],rotation = 90) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

# axs[0].set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (meV)')
# ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
# axs[0].set_xlabel(r'Molecule on TiO$_2$ ')
# axs[1].set_ylim([-700,0])

axs[0].set_ylim([-1400,0])

axs[1].grid(axis = "y",ls='--', lw=.5)

axs[0].grid(axis = "y",ls='--', lw=.5)

# axs[0].legend(frameon=False,loc='upper left')

axs[1].set_xlim([7.5,11])
axs[0].set_xlim([0.5,7])


axs[1].xaxis.tick_top()
axs[0].xaxis.tick_top()
axs[1].xaxis.set_label_position('top') 
axs[0].xaxis.set_label_position('top') 


plt.savefig('Fig_02b.png')

# Figure 4 DFT benchmark

In [43]:
xc_compare_system_list = ['MgO CH4', 'MgO C2H6', 'MgO CO', 'MgO CO2_2', 'MgO CO2_3', 'MgO H2O','MgO N2O_1','MgO N2O_2', 'MgO NH3','TiO2_rutile CH4','TiO2_rutile CH3OH','TiO2_rutile CO2_1','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']
xc_compare_dft_list = list(reversed(['01_PBE-D30', '02_PBE-MBDFI', '03_rev-vdW-DF2', '04_R2SCAN-rVV10', '05_PBE0-TSHI','06_HSE06-D4','07_RPA','08_RPA-rSE']))
xc_compare_system_list_small = ['MgO CH4', 'MgO C2H6', 'MgO CO', 'MgO CO2_2', 'MgO H2O','MgO N2O_1', 'MgO NH3','TiO2_rutile CH4','TiO2_rutile CH3OH','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']


dft_xc_eint_error_dict = {x: {'MgO': 0,'TiO2': 0,'Total': 0} for x in xc_compare_dft_list} # + ['07_RPA','08_RPA-rSE']}
dft_xc_eint_system_error_dict = {x: {y:0 for y in xc_compare_system_list} for x in xc_compare_dft_list} # + ['07_RPA','08_RPA-rSE']}


for functional in xc_compare_dft_list:
    if functional in ['07_RPA','08_RPA-rSE']:
        mgo_sum = []
        total_sum = []
        for mol_surface in xc_compare_system_list_small:
            molecule = mol_surface.split()[-1]
            surface  = mol_surface.split()[0]

            if surface == 'MgO':
                dft_xc_eint_system_error_dict[functional][mol_surface] = dft_xc_eint_dict[mol_surface][functional] -  final_cc_eint_list[surface][molecule]['final'][0]
                mgo_sum += [dft_xc_eint_dict[mol_surface][functional] -  final_cc_eint_list[surface][molecule]['final'][0]]

            # total_sum += [dft_xc_eint_dict[mol_surface][functional] -  final_cc_eint_list[surface][molecule]['final'][0]]

        dft_xc_eint_error_dict[functional]['MgO'] = np.mean(np.abs(mgo_sum))
    else:
        mgo_sum = []
        tio2_sum = []
        total_sum = []
        for mol_surface in xc_compare_system_list_small:
            molecule = mol_surface.split()[-1]
            surface  = mol_surface.split()[0]
            dft_xc_eint_system_error_dict[functional][mol_surface] = dft_xc_eint_dict[mol_surface][functional] -  final_cc_eint_list[surface][molecule]['final'][0]
            if surface == 'MgO':
                mgo_sum += [dft_xc_eint_dict[mol_surface][functional] -  final_cc_eint_list[surface][molecule]['final'][0]]

            elif 'TiO2' in surface:
                tio2_sum += [dft_xc_eint_dict[mol_surface][functional] -  final_cc_eint_list[surface][molecule]['final'][0]]
            total_sum += [dft_xc_eint_dict[mol_surface][functional] -  final_cc_eint_list[surface][molecule]['final'][0]]

        dft_xc_eint_error_dict[functional]['MgO'] = np.mean(np.abs(mgo_sum))
        dft_xc_eint_error_dict[functional]['TiO2'] = np.mean(np.abs(tio2_sum))
        dft_xc_eint_error_dict[functional]['Total'] = np.mean(np.abs(total_sum))


In [44]:



x_index_labels = ['MgO CH4',
 'MgO C2H6',
 'MgO CO',
 'MgO CO2_2',
 'MgO H2O',
 'MgO N2O_1',
 'MgO NH3',
 'MgO MAD',
 'TiO2_rutile CH4',
 'TiO2_rutile CH3OH',
 'TiO2_rutile CO2_2',
 'TiO2_rutile H2O',
 'TiO2_anatase H2O',
 'TiO2_anatase NH3',
 'TiO2 MAD',
 'Total MAD']

xc_compare_dft_list1 = list(reversed(['01_PBE-D30', '02_PBE-MBDFI', '03_rev-vdW-DF2','04_M06-L', '04_R2SCAN-rVV10', '05_PBE0-TSHI','06_HSE06-D4','07_RPA','08_RPA-rSE']))

ene_heatmap = np.zeros((len(xc_compare_dft_list1),len(xc_compare_system_list_small) +3),dtype=int)
ene_heatmap_mads = np.zeros((len(xc_compare_dft_list1),len(xc_compare_system_list_small) +3),dtype=int)


for index1, i in enumerate(xc_compare_dft_list1):
    for index2, j in enumerate(x_index_labels):
        if 'MAD' in j and 'M06-L' in i:
            ene_heatmap[index1,index2] = 0 #int(round(dft_xc_eint_error_dict[i][j.split()[0]]))
            ene_heatmap_mads[index1,index2] = int(round(dft_xc_eint_error_dict['03_rev-vdW-DF2'][j.split()[0]]))
        elif 'MAD' in j:
            ene_heatmap[index1,index2] = 0 #int(round(dft_xc_eint_error_dict[i][j.split()[0]]))
            ene_heatmap_mads[index1,index2] = int(round(dft_xc_eint_error_dict[i][j.split()[0]]))
        elif 'M06-L' in i:
            ene_heatmap[index1,index2] = int(round(dft_xc_eint_system_error_dict['03_rev-vdW-DF2'][j]))
        else:
            ene_heatmap[index1,index2] = int(round(dft_xc_eint_system_error_dict[i][j]))

In [45]:

fig, axs = plt.subplots(figsize=(6.69,3.5),dpi=1200,tight_layout=True)
plt.set_cmap('bwr')


# print(index,i)
xc_nice_list = list(reversed(['PBE-D3','PBE-MBD/HI','rev-vdW-DF2','M06-L', r'r$^2$SCAN-rVV10',r'PBE0-TS/HI','HSE06-D4','RPA','RPA+rSE']))

xc_compare_system_nice_list =  ['CH4', r'C$_2$H$_6$', 'CO', r'CO$_2$', r'H$_2$O',r'N$_2$O', r'NH$_3$','MAD',r'r-CH$_4',r'r-CH$_3$OH',r'r-CO$_2$',r'r-H$_2$O',r'a-H$_2$O',r'a-NH$_3$',r'MAD',r'\textbf{Total MAD}'] # + ['CH4']*(len(xc_compare_system_list_small))


im = axs.imshow(ene_heatmap, vmin=-300,vmax=300)


axs.set_xticks(np.arange(len(xc_compare_system_list_small) +3), labels=xc_compare_system_nice_list,fontsize=8)
axs.set_yticks(np.arange(len(xc_compare_dft_list1)), labels=xc_nice_list,fontsize=8)

for i12 in range(len(xc_nice_list)):
    for j in range(len(xc_compare_system_list_small) +3):
        if 'RPA' in xc_nice_list[i12] and 'MgO' not in x_index_labels[j]:
            continue
        if 'MAD' in xc_compare_system_nice_list[j]:
            text = axs.text(j, i12, ene_heatmap_mads[i12, j],
                    ha="center", va="center", color="k",fontsize=8)
        else:
            text = axs.text(j, i12, ene_heatmap[i12, j],
                        ha="center", va="center", color="k",fontsize=8)

# Create a divider for the existing axes to place the color bars
divider = make_axes_locatable(axs)

# First color bar
cax1 = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax1)
cbar.ax.set_ylabel(r'$E_\textrm{int}[\textrm{Method}]$ - $E_\textrm{int}[\textrm{SKZCAM}]$ (meV)', rotation=-90, va="bottom", fontsize=7)
cbar.ax.tick_params(labelsize=7)  # Decrease y-tick label size for the second color bar


# Define custom colormap that transitions from invisible to black
colors = [(1, 1, 1, 0), (0, 0, 0, 0.7)]  # RGBA tuples: transparent to black
n_bins = 100  # Discretize into 100 steps
cmap_custom = LinearSegmentedColormap.from_list('invisible_to_black', colors, N=n_bins)

# Second color bar for custom colormap
cax2 = divider.append_axes("right", size="2%", pad=0.55)  # Increased pad value to shift it further to the right
im_custom = axs.imshow(ene_heatmap_mads, cmap=cmap_custom, alpha=1,vmin=0,vmax=200)  # Use alpha to make it invisible
cbar_custom = plt.colorbar(im_custom, cax=cax2)
cbar_custom.ax.set_ylabel('Mean absolute deviation', rotation=-90, va="bottom",fontsize=7)
cbar_custom.ax.tick_params(labelsize=7)  # Decrease y-tick label size for the second color bar

# axs.hlines(y=np.arange(0, 10)+0.5, xmin=np.full(10, 0)-0.5, xmax=np.full(10, 10)-0.5, color="black")
# axs.vlines(x=np.arange(0, 8)+0.5, ymin=np.full(10, 0)-0.5, ymax=np.full(8, 10)-0.5, color="black")

# Adding vertical lines to separate columns
for i in range(len(xc_compare_system_list_small) + 2):
    if i > 7:
        axs.plot([i+0.5,i+0.5],[1.5,8.5], color='white', linewidth=0.5)
    else:
        axs.axvline(x=i + 0.5, color='white', linewidth=0.5)

# Adding horizontal lines to separate rows
for i in range(len(xc_nice_list) - 1):
    if i < 2:
        axs.plot([-0.5,7.5],[i+0.5,i+0.5], color='white', linewidth=0.5)
    else:
        axs.axhline(y=i + 0.5, color='white', linewidth=0.5)

axs.axvline(x=7.5, color='black', linewidth=0.75)
axs.plot([14.5,14.5],[1.5,8.5], color='black', linewidth=0.75)
axs.plot([7.5,15.5],[1.5,1.5], color='black', linewidth=0.75)

# axs.xaxis.tick_top()

axs.add_patch(Rectangle((7.5, -0.5), 8, 2, fill=False, hatch='////'))
plt.setp(axs.get_xticklabels(), rotation=90)

plt.savefig('Fig_04.png')


In [46]:
# fig, axs = plt.subplots(1,2,figsize=(6.69,3),dpi=1200, constrained_layout=True, gridspec_kw={'width_ratios': [3, 1]})


# # axs[0].errorbar([1,2,3,4,5,6,7], [skzcam_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O',  'H2O', 'NH3', 'CO2']],   [skzcam_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'H2O', 'NH3', 'CO2']],color=color_dict['black'],capsize=2,fmt="o",alpha=1.0,label='SKZCAM')

# # axs[0].errorbar([1,2,3,4,5,6,7,8,9,10,11,12,13], [final_cc_eint_list['MgO'][x]['final'][0] for x in ['CH4', 'C2H6','CO','N2O_1', 'H2O', 'NH3']] + [-250] + [final_cc_eint_list['TiO2_rutile'][x]['final'][0] for x in ['CH4', 'CH3OH','CO2_2','H2O']] + [final_cc_eint_list['TiO2_anatase'][x]['final'][0] for x in ['H2O','NH3']],   [43]*13,color=color_dict['black'],capsize=2,fmt="o",alpha=1.0,label='SKZCAM')

# axs[0].plot([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['01_PBE-D30'][x] for x in xc_compare_system_list_small],color=color_dict['red'],linestyle='--',marker="x",alpha=0.5,label='PBE-D3(0)')

# axs[0].plot([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['02_PBE-MBDFI'][x] for x in xc_compare_system_list_small], color=color_dict['orange'],linestyle='--',marker="x",alpha=0.5,label='PBE-MBD/FI')

# axs[0].errorbar([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['03_rev-vdW-DF2'][x] for x in xc_compare_system_list_small], color=color_dict['maroon'],linestyle='--',marker="x",alpha=0.5,label='rev-vdW-DF2')

# axs[0].errorbar([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['04_R2SCAN-rVV10'][x] for x in xc_compare_system_list_small],  color=color_dict['teal'],linestyle='--',marker="x",alpha=0.5,label=r'r\textsuperscript{2}SCAN-rVV10')

# axs[0].errorbar([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['05_PBE0-TSHI'][x] for x in xc_compare_system_list_small], color=color_dict['yellow'],linestyle='--',marker="x",alpha=0.5,label=r'PBE0-TSHI')

# axs[0].errorbar([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['06_HSE06-D4'][x] for x in xc_compare_system_list_small], color=color_dict['cyan'],linestyle='--',marker="x",alpha=0.5,label=r'HSE06-D4')

# axs[0].errorbar([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['07_RPA'][x] for x in xc_compare_system_list_small],color=color_dict['olive'],linestyle='--',marker="x",alpha=0.5,label='RPA')

# axs[0].errorbar([1,2,3,4,5,6,7,8,9,10,11,12,13], [dft_xc_eint_system_error_dict['08_RPA-rSE'][x] for x in xc_compare_system_list_small], color=color_dict['blue'],linestyle='--',marker="x",alpha=0.5,label='RPA+rSE')

# axs[0].set_xticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
# axs[0].set_xticklabels([r'CH\textsubscript{4}', r'C\textsubscript{2}H\textsubscript{6}','CO',r'N\textsubscript{2}O', r'H\textsubscript{2}O', r'NH\textsubscript{3}', r'CO\textsubscript{2}', r'CH\textsubscript{4}', r'CH\textsubscript{3}OH', r'CO\textsubscript{2}', r'H\textsubscript{2}O'] + [r'H\textsubscript{2}O', r'NH\textsubscript{3}'] , rotation = 90) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

# axs[0].set_ylabel(r'Adsorption energy $E_\textrm{ads}$ (meV)')
# # ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
# axs[0].set_xlabel(r'Molecule on MgO (001)')
# axs[0].set_ylim([-400,300])
# axs[0].legend(frameon=False,loc='lower left',fontsize=8)

# # axs[1].errorbar([1,2,3,4], [skzcam_ene_list[x][0] for x in ['rutile_CH4', 'rutile_CH3OH','rutile_CO2','rutile_H2O']],   [skzcam_ene_list[x][1] for x in ['rutile_CH4', 'rutile_CH3OH','rutile_CO2','rutile_H2O']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5,label='SKZCAM')

# axs[1].errorbar([1,2,3,4], [final_cc_eint_list['TiO2_rutile'][x]['final'][0] for x in ['CH4', 'CH3OH','CO2_2','H2O']], [43]*4 ,color=color_dict['black'],capsize=2,fmt="o",alpha=1,label='SKZCAM')

# axs[1].errorbar([6,7], [final_cc_eint_list['TiO2_anatase'][x]['final'][0] for x in ['H2O','NH3']],   [43]*2,color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5)

# axs[1].errorbar([1,2,3,4,6,7], [dft_xc_eint_dict[x]['01_PBE-D30'] for x in ['TiO2_rutile CH4', 'TiO2_rutile CH3OH','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']],   [0]*6,color=color_dict['red'],capsize=2,fmt="x",alpha=0.5,label='PBE-D3(0)')

# axs[1].errorbar([1,2,3,4,6,7], [dft_xc_eint_dict[x]['02_PBE-MBDFI'] for x in ['TiO2_rutile CH4', 'TiO2_rutile CH3OH','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']],   [0]*6,color=color_dict['orange'],capsize=2,fmt="x",alpha=0.5,label='PBE-MBD/FI')

# axs[1].errorbar([1,2,3,4,6,7], [dft_xc_eint_dict[x]['03_rev-vdW-DF2'] for x in ['TiO2_rutile CH4', 'TiO2_rutile CH3OH','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']],   [0]*6,color=color_dict['maroon'],capsize=2,fmt="x",alpha=0.5,label='rev-vdW-DF2')

# axs[1].errorbar([1,2,3,4,6,7], [dft_xc_eint_dict[x]['04_R2SCAN-rVV10'] for x in ['TiO2_rutile CH4', 'TiO2_rutile CH3OH','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']],   [0]*6,color=color_dict['teal'],capsize=2,fmt="x",alpha=0.5,label=r'r\textsuperscript{2}SCAN-rVV10')

# axs[1].errorbar([1,2,3,4,6,7], [dft_xc_eint_dict[x]['05_PBE0-TSHI'] for x in ['TiO2_rutile CH4', 'TiO2_rutile CH3OH','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']],   [0]*6,color=color_dict['yellow'],capsize=2,fmt="x",alpha=0.5,label='PBE0-TSHI')

# axs[1].errorbar([1,2,3,4,6,7], [dft_xc_eint_dict[x]['06_HSE06-D4'] for x in ['TiO2_rutile CH4', 'TiO2_rutile CH3OH','TiO2_rutile CO2_2','TiO2_rutile H2O','TiO2_anatase H2O','TiO2_anatase NH3']],   [0]*6,color=color_dict['cyan'],capsize=2,fmt="x",alpha=0.5,label=r'HSE06-D4')





# axs[1].set_xticks([1,2,3,4,6,7])
# axs[1].set_xticklabels([r'CH\textsubscript{4}', r'CH\textsubscript{3}OH', r'CO\textsubscript{2}', r'H\textsubscript{2}O'] + [r'H\textsubscript{2}O', r'NH\textsubscript{3}'],rotation = 90) #[r'H\textsubscript{2}O', 
# # axs[1].set_xticklabels([r'rutile CH\textsubscript{4}', r'rutile CH\textsubscript{3}OH', r'rutile CO\textsubscript{2}', r'rutile H\textsubscript{2}O'] + [r'anatase H\textsubscript{2}O', r'anatase NH\textsubscript{3}'],rotation = 90) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

# axs[1].set_ylabel(r'Adsorption energy $E_\textrm{ads}$ (meV)')
# # ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
# axs[1].set_xlabel(r'Molecule on TiO$_2$ ')
# axs[1].set_ylim([-200,200])
# # axs[1].legend(frameon=False,loc='upper left')

# axs[0].xaxis.tick_top()
# axs[1].xaxis.tick_top()
# axs[0].xaxis.set_label_position('top') 
# axs[1].xaxis.set_label_position('top') 


# plt.savefig('Fig_03.png')

# Figure 4 Atomistic Insights

In [148]:
mgo_system_func_dependence = {x: [] for x in ['NO_1','NO_2','NO_3','NO_4','NO_5','NO_6','CO2_2','CO2_3','CH3OH_1','CH3OH_2','CH3OH_4']}

adsorbate_config = {
    'NO_1': 'Vertical-Hollow',
    'NO_2': 'Vertical-Mg',
    'NO_3': 'Bent-Bridge',
    'NO_4': 'Bent-Mg',
    'NO_5': 'Bent-O',
    'NO_6': 'Dimer',
    'CO2_2': 'Physisorbed',
    'CO2_3': 'Chemisorbed',
    'CH3OH_1': 'Monomer',
    'CH3OH_2': 'Dimer',
    'CH3OH_4': 'Tetramer',
    'H2O': 'Monomer',
    'H2O_4': 'Cluster',
    'N2O_1': 'Parallel',
    'N2O_2': 'Tilted',
    'TiO2_rutile CO2_1': 'Parallel',
    'TiO2_rutile CO2_2': 'Tilted'
}

for index,i in enumerate(adsorbate_config):
    data_dummy = []

    if 'TiO2' in i:
        molecule_name = i.split()[1]
        for j in dft_functionals['TiO2_rutile']:
            data_dummy += [final_eads_dict['TiO2_rutile'][molecule_name][j]]
    else:
        for j in dft_functionals['MgO']:
            data_dummy += [final_eads_dict['MgO'][i][j]] # + ethermal['MgO'][i]['02_revPBE-D4']['Ecor']]
        
    mgo_system_func_dependence[i] = data_dummy

In [144]:
# Combine CH3OH and H2O clusters

fig, axs = plt.subplots(1,2,figsize=(6.69,3.6),dpi=1200, sharey=True, constrained_layout=True, gridspec_kw={'width_ratios': [3,2.1]})

x_list0 = np.arange(1,4)*2

for idx_xc_func, xc_func in enumerate(dft_functionals['MgO']):
    axs[0].bar(x_list0 + 0.2*idx_xc_func, [mgo_system_func_dependence['CH3OH_1'][idx_xc_func], mgo_system_func_dependence['CH3OH_2'][idx_xc_func],mgo_system_func_dependence['CH3OH_4'][idx_xc_func]], label=convert_to_nice_labels['xc_functionals'][xc_func],width=0.2)

axs[0].bar(x_list0 - 0.2, [final_hads['MgO']['CH3OH_1'][0],final_hads['MgO']['CH3OH_2'][0],final_hads['MgO']['CH3OH_4'][0]], label='autoSKZCAM',width=0.2, color='gray', edgecolor='black',alpha=0.7)

axs[0].plot([0,9], [mol_hads['MgO']['CH3OH_4'][0], mol_hads['MgO']['CH3OH_4'][0]], color='k', linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs[0].fill_between([0,9], mol_hads['MgO']['CH3OH_4'][0] - mol_hads['MgO']['CH3OH_4'][1], mol_hads['MgO']['CH3OH_4'][0] + mol_hads['MgO']['CH3OH_4'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')

axs[0].set_xticks(x_list0 + 0.4)

axs[0].set_xticklabels([],rotation=90,ha='center')
axs[0].set_ylabel(r'$H_\textrm{ads}$ (meV)')
# Set xlabels to the top
axs[0].xaxis.tick_top()
# axs[0].legend(frameon=False,ncol=2,loc='lower left',fontsize=8)
axs[0].set_ylim(-1000,-400)
axs[0].set_xlim([1.5,7.3])

x_list1 = np.arange(1,3)*2

for idx_xc_func, xc_func in enumerate(dft_functionals['MgO']):
    axs[1].bar(x_list1 + 0.2*idx_xc_func, [mgo_system_func_dependence['H2O'][idx_xc_func], mgo_system_func_dependence['H2O_4'][idx_xc_func]], label=convert_to_nice_labels['xc_functionals'][xc_func],width=0.2)

axs[1].bar(x_list1 - 0.2, [final_hads['MgO']['H2O'][0],final_hads['MgO']['H2O_4'][0]], label='autoSKZCAM',width=0.2, color='gray', edgecolor='black',alpha=0.7)

axs[1].plot([0,9], [mol_hads['MgO']['H2O_4'][0], mol_hads['MgO']['H2O_4'][0]], color='k', linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs[1].fill_between([0,9], mol_hads['MgO']['H2O_4'][0] - mol_hads['MgO']['H2O_4'][1], mol_hads['MgO']['H2O_4'][0] + mol_hads['MgO']['H2O_4'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')

axs[1].set_xticks(x_list1 + 0.4)

axs[1].set_xticklabels([],rotation=90,ha='center')
# Set xlabels to the top
axs[1].xaxis.tick_top()
axs[1].legend(frameon=False,ncol=2,loc='lower left',fontsize=8)
axs[1].set_ylim(-1000,-400)
axs[1].set_xlim([1.5, 5.3])




plt.savefig('Fig_SI_CH3OH_SKZCAM_and_XC_Hads.png')


array([1, 2, 3])

In [ ]:
# We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.



fig, axs = plt.subplots(figsize=(3.365,3),dpi=1200, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})

# axs0_ticklabels = []
# for i, struct in enumerate([2,3]):
#     co2_xc_list = np.array(mgo_system_func_dependence[f'CO2_{struct}']) - mol_hads['MgO'][f'CO2_3'][0]
#     co2_skzcam = final_hads['MgO'][f'CO2_{struct}'][0] -  mol_hads['MgO'][f'CO2_3'][0]

#     axs[0].bar(i*2 + 0.25, np.ptp(co2_xc_list),bottom = np.min(co2_xc_list),width=0.5, color=color_dict['blue'], alpha=  0.7)
#     axs[0].plot([i*2-0.75,i*2-0.25], [co2_skzcam]*2, '-',color=color_dict['red'],linewidth=1)
#     axs[0].fill_between([i*2-0.75,i*2-0.25], co2_skzcam -mol_hads['MgO'][f'CO2_3'][1], co2_skzcam + mol_hads['MgO'][f'CO2_{struct}'][1], color=color_dict['red'],alpha=0.1, edgecolor='none')

#     axs0_ticklabels += [adsorbate_config[f'CO2_{struct}']]


# axs[0].fill_between([-1,3], -mol_hads['MgO']['CO2_3'][1], mol_hads['MgO']['CO2_3'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')
# axs[0].plot([-2, 3], [0,0], '-',color=color_dict['black'],linewidth=0.7)


axs1_ticklabels = []

for i, struct in enumerate([6,4,2,3,5,1]): #range(1,7):


    no_xc_list = np.array(mgo_system_func_dependence[f'NO_{struct}']) - mol_hads['MgO']['NO'][0]
    no_skzcam = final_hads['MgO'][f'NO_{struct}'][0] -  mol_hads['MgO']['NO'][0]

    if struct == 6:

        axs.bar(i*2 + 0.25, np.ptp(no_xc_list),bottom = np.min(no_xc_list),width=0.5, color=color_dict['blue'],alpha=0.7, label = 'DFT range')
        axs.plot([i*2-0.75,i*2-0.25], [no_skzcam]*2, '-',color=color_dict['red'],linewidth=1)
        axs.fill_between([i*2-0.75,i*2-0.25], no_skzcam -mol_hads['MgO']['NO'][1], no_skzcam + mol_hads['MgO']['NO'][1], color=color_dict['red'],alpha=0.1, edgecolor='none',label='autoSKZCAM')

    else:
        axs.bar(i*2 + 0.25, np.ptp(no_xc_list),bottom = np.min(no_xc_list),width=0.5, color=color_dict['blue'],alpha=0.7)
        axs.plot([i*2-0.75,i*2-0.25], [no_skzcam]*2, '-',color=color_dict['red'],linewidth=1)

        axs.fill_between([i*2-0.75,i*2-0.25], no_skzcam -mol_hads['MgO']['NO'][1], no_skzcam + mol_hads['MgO']['NO'][1], color=color_dict['red'],alpha=0.1, edgecolor='none')

    axs1_ticklabels += [adsorbate_config[f'NO_{struct}']]


axs.fill_between([-1,13], -mol_hads['MgO']['NO'][1], mol_hads['MgO']['NO'][1], color=color_dict['black'],alpha=0.1, edgecolor='none', label='Expt. Error')
axs.plot([-2, 14], [0,0], '-',color=color_dict['black'],linewidth=0.7)

axs.set_xticks(np.arange(0,11,2))
axs.set_xticklabels(axs1_ticklabels,rotation=90,ha='center')

axs.legend(frameon=False, fontsize=8)
axs.set_ylabel(r'Deviation from Expt. $H_\textrm{ads}$ (meV)')

# axs[0].set_xlim([-1,2.75])
axs.set_xlim([-1,11])

axs.set_ylim([-300,450])


plt.savefig('Fig_03.png')

In [58]:
mol_hads['MgO']['CH3OH_4'][0]

[-902.7278799573779, 93.86685589723966, 'C&S Hads with nu = 15.3']

In [93]:
# Making the CH3OH configurations

fig, axs = plt.subplots(figsize=(6.69,3.5),dpi=1200, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})

# Considering that 
x_list = np.array([1,2-0.37,3-0.19,4-0.185,5-0.23,6+0.03,7+0.03,8])

axs.bar(x_list, [-521, -631, -715, -717, -749, -760, -765, -830], color=color_dict['red'], width=0.3, alpha=0.9,label='revPBE-D4',facecolor='red',edgecolor='black')
axs.errorbar(x_list[[0,1,4]], [final_hads['MgO']['CH3OH_1'][0], final_hads['MgO']['CH3OH_2'][0], final_hads['MgO']['CH3OH_4'][0]], yerr =  [final_hads['MgO']['CH3OH_1'][1], final_hads['MgO']['CH3OH_2'][1], final_hads['MgO']['CH3OH_4'][1]], color=color_dict['blue'], alpha=0.7,label='autoSKZCAM', ls='none', marker='o', capsize=2)


axs.plot([0,9], [mol_hads['MgO']['CH3OH_4'][0], mol_hads['MgO']['CH3OH_4'][0]], color='k', linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs.fill_between([0,9], mol_hads['MgO']['CH3OH_4'][0] - mol_hads['MgO']['CH3OH_4'][1], mol_hads['MgO']['CH3OH_4'][0] + mol_hads['MgO']['CH3OH_4'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')

axs.set_xlim([0.5,8.5])
axs.set_ylim(-1050,-450)
# Set xticklabels to the top
axs.xaxis.tick_top()
axs.set_xticks(x_list)
axs.set_xticklabels([])
axs.legend(fontsize=8)

axs.set_ylabel(r'$H_\textrm{ads}$ (meV)')
plt.savefig('Fig_SI_CH3OH_configurations_hads.png')


In [95]:
mgo_system_func_dependence['CH3OH_1']

[-598.9649899999705,
 -624.7368300000069,
 -575.9006500000226,
 -663.1328599999904,
 -658.9240999999788,
 -551.9974199999851]

In [160]:
mol_hads['TiO2_rutile']['CO2']

[-500.59422045856894, 0.0, 'C&S Hads with nu = 13.6']

In [161]:
# We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.

adsorbate_config = {
    'NO_1': 'Vertical-Hollow',
    'NO_2': 'Vertical-Mg',
    'NO_3': 'Bent-Bridge',
    'NO_4': 'Bent-Mg',
    'NO_5': 'Bent-O',
    'NO_6': 'Dimer',
    'CO2_2': 'Physisorbed',
    'CO2_3': 'Chemisorbed',
    'CH3OH_1': 'Monomer',
    'CH3OH_2': 'Dimer',
    'CH3OH_4': 'Tetramer'
}

fig, axs = plt.subplots(figsize=(3.365,3.5),dpi=600, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})


# Plot the CO2 configurations

# axs.bar(-2-0.25, final_hads['TiO2_rutile']['CO2_1'][0],width=0.5, color=color_dict['red'])
# axs.bar(-2+0.25, final_hads['TiO2_rutile']['CO2_2'][0],width=0.5, color=color_dict['blue'])


axs.errorbar(list(range(7)), [final_hads['TiO2_rutile']['CO2_1'][0]] + [final_eads_dict['TiO2_rutile']['CO2_1'][x] + ethermal['TiO2_rutile']['CO2_1']['02_revPBE-D4']['Ecor'] for x in dft_functionals['TiO2_rutile']], yerr = [final_hads['TiO2_rutile']['CO2_1'][1]] + [0]*6, color=color_dict['red'], alpha=0.7, ls='--', marker='o', capsize=2, label='Parallel')
axs.errorbar(list(range(7)), [final_hads['TiO2_rutile']['CO2_2'][0]] + [final_eads_dict['TiO2_rutile']['CO2_2'][x] + ethermal['TiO2_rutile']['CO2_2']['02_revPBE-D4']['Ecor'] for x in dft_functionals['TiO2_rutile']], yerr =  [final_hads['TiO2_rutile']['CO2_2'][1]] + [0]*6, color=color_dict['blue'], alpha=0.7, ls='--', marker='o', capsize=2, label='Tilted')

axs.plot([0,7], [mol_hads['TiO2_rutile']['CO2'][0], mol_hads['TiO2_rutile']['CO2'][0]], color='k', linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs.fill_between([0,7], mol_hads['TiO2_rutile']['CO2'][0] - mol_hads['TiO2_rutile']['CO2'][1], mol_hads['TiO2_rutile']['CO2'][0] + mol_hads['TiO2_rutile']['CO2'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')

# for idx_xc_func, xc_func in enumerate(dft_functionals['TiO2_rutile']):
#     axs.bar(idx_xc_func*2-0.25, final_eads_dict['TiO2_rutile']['CO2_1'][xc_func] + ethermal['TiO2_rutile']['CO2_1']['02_revPBE-D4']['Ecor'],width=0.5, color=color_dict['red'],label="Parallel" if idx_xc_func == 0 else "")
#     axs.bar(idx_xc_func*2+0.25, final_eads_dict['TiO2_rutile']['CO2_2'][xc_func] + ethermal['TiO2_rutile']['CO2_2']['02_revPBE-D4']['Ecor'],width=0.5, color=color_dict['blue'],label="Tilted" if idx_xc_func == 0 else "")



# axs.fill_between([-3,11], mol_hads['TiO2_rutile']['CO2_1'][0]-mol_hads['TiO2_rutile']['CO2_1'][1], mol_hads['TiO2_rutile']['CO2_1'][0] + mol_hads['TiO2_rutile']['CO2_1'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')
# axs.plot([-3,11], [mol_hads['TiO2_rutile']['CO2_1'][0],mol_hads['TiO2_rutile']['CO2_1'][0]], '-',color=color_dict['black'],linewidth=0.7, label=r"Chemisorbed Expt $H_\textrm{ads}$ XX \textit{et al.}")

# axs.fill_between([-3,11], mol_hads['TiO2_rutile']['CO2_2'][0]-mol_hads['TiO2_rutile']['CO2_2'][1], mol_hads['TiO2_rutile']['CO2_2'][0] + mol_hads['TiO2_rutile']['CO2_2'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')
# axs.plot([-3, 11], [mol_hads['TiO2_rutile']['CO2_2'][0],mol_hads['TiO2_rutile']['CO2_2'][0]], '--',color=color_dict['black'],linewidth=0.7, label=r"`Physisorbed' Expt $H_\textrm{ads}$ Meixner \textit{et al.}")


axs.set_xticks(list(range(7)))
axs.set_xticklabels([r'\textbf{SKZCAM}'] + [convert_to_nice_labels['xc_functionals'][x] for x in dft_functionals['TiO2_rutile']],rotation=90,ha='center')

axs.legend(frameon=False, fontsize=8)
axs.set_ylabel(r'$H_\textrm{ads}$ (meV)')

# axs.set_xlim([-3,11])

axs.set_ylim([-1000,0])


plt.savefig('Fig_SI_rutile_CO2_configuration_hads.png')

In [168]:
# We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.

fig, axs = plt.subplots(figsize=(3.365,3.5),dpi=600, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})


# Plot the CO2 configurations

axs.bar(-2-0.25, final_hads['MgO']['N2O_1'][0],width=0.5, color=color_dict['red'])
axs.bar(-2+0.25, final_hads['MgO']['N2O_2'][0],width=0.5, color=color_dict['blue'])


for idx_xc_func, xc_func in enumerate(dft_functionals['MgO']):
    axs.bar(idx_xc_func*2-0.25, final_eads_dict['MgO']['N2O_1'][xc_func] + ethermal['MgO']['N2O_1']['02_revPBE-D4']['Ecor'],width=0.5, color=color_dict['red']) #,label="Physisorbed" if idx_xc_func == 0 else "")
    axs.bar(idx_xc_func*2+0.25, final_eads_dict['MgO']['N2O_2'][xc_func] + ethermal['MgO']['N2O_2']['02_revPBE-D4']['Ecor'],width=0.5, color=color_dict['blue']) #,label="Chemisorbed" if idx_xc_func == 0 else "")



axs.fill_between([-3,11], mol_hads['MgO']['N2O'][0]-mol_hads['MgO']['N2O'][1], mol_hads['MgO']['N2O'][0] + mol_hads['MgO']['N2O'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')
axs.plot([-3,11], [mol_hads['MgO']['N2O'][0],mol_hads['MgO']['N2O'][0]], '-',color=color_dict['black'],linewidth=0.7, label=r"Expt")



axs.set_xticks(np.arange(-2,11,2))
axs.set_xticklabels([r'\textbf{SKZCAM}'] + [convert_to_nice_labels['xc_functionals'][x] for x in dft_functionals['MgO']],rotation=90,ha='center')

axs.legend(frameon=False, fontsize=8)
axs.set_ylabel(r'$H_\textrm{ads}$ (meV)')

axs.set_xlim([-3,11])

axs.set_ylim([-300,0])


plt.savefig('Fig_SI_N2O_configuration_hads.png')


In [ ]:
# We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.

adsorbate_config = {
    'NO_1': 'Vertical-Hollow',
    'NO_2': 'Vertical-Mg',
    'NO_3': 'Bent-Bridge',
    'NO_4': 'Bent-Mg',
    'NO_5': 'Bent-O',
    'NO_6': 'Dimer',
    'CO2_2': 'Physisorbed',
    'CO2_3': 'Chemisorbed',
    'CH3OH_1': 'Monomer',
    'CH3OH_2': 'Dimer',
    'CH3OH_4': 'Tetramer'
}

fig, axs = plt.subplots(figsize=(3.365,4),dpi=600, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})


# Plot the CO2 configurations

axs.bar(-2-0.25, final_hads['MgO']['CO2_2'][0],width=0.5, color=color_dict['red'])
axs.bar(-2+0.25, final_hads['MgO']['CO2_3'][0],width=0.5, color=color_dict['blue'])


for idx_xc_func, xc_func in enumerate(dft_functionals['MgO']):
    axs.bar(idx_xc_func*2-0.25, final_eads_dict['MgO']['CO2_2'][xc_func] + ethermal['MgO']['CO2_2']['02_revPBE-D4']['Ecor'],width=0.5, color=color_dict['red'],label="Physisorbed" if idx_xc_func == 0 else "")
    axs.bar(idx_xc_func*2+0.25, final_eads_dict['MgO']['CO2_3'][xc_func] + ethermal['MgO']['CO2_3']['02_revPBE-D4']['Ecor'],width=0.5, color=color_dict['blue'],label="Chemisorbed" if idx_xc_func == 0 else "")



axs.fill_between([-3,11], mol_hads['MgO']['CO2_3'][0]-mol_hads['MgO']['CO2_3'][1], mol_hads['MgO']['CO2_3'][0] + mol_hads['MgO']['CO2_3'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')
axs.plot([-3,11], [mol_hads['MgO']['CO2_3'][0],mol_hads['MgO']['CO2_3'][0]], '-',color=color_dict['black'],linewidth=0.7, label=r"Chemisorbed Expt $H_\textrm{ads}$ XX \textit{et al.}")

axs.fill_between([-3,11], mol_hads['MgO']['CO2_2'][0]-mol_hads['MgO']['CO2_2'][1], mol_hads['MgO']['CO2_2'][0] + mol_hads['MgO']['CO2_2'][1], color=color_dict['black'],alpha=0.1, edgecolor='none')
axs.plot([-3, 11], [mol_hads['MgO']['CO2_2'][0],mol_hads['MgO']['CO2_2'][0]], '--',color=color_dict['black'],linewidth=0.7, label=r"`Physisorbed' Expt $H_\textrm{ads}$ Meixner \textit{et al.}")


axs.set_xticks(np.arange(-2,11,2))
axs.set_xticklabels([r'\textbf{SKZCAM}'] + [convert_to_nice_labels['xc_functionals'][x] for x in dft_functionals['MgO']],rotation=90,ha='center')

axs.legend(frameon=False, fontsize=8)
axs.set_ylabel(r'$H_\textrm{ads}$ (meV)')

axs.set_xlim([-3,11])

axs.set_ylim([-1000,0])


plt.savefig('Fig_SI_CO2_configuration_hads.png')

In [ ]:
# We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.
adsorbate_config = {
    'NO_1': 'Vertical-Hollow',
    'NO_2': 'Vertical-Mg',
    'NO_3': 'Bent-Bridge',
    'NO_4': 'Bent-Mg',
    'NO_5': 'Bent-O',
    'NO_6': 'Dimer',
    'CO2_2': 'Physisorbed',
    'CO2_3': 'Chemisorbed',
    'CH3OH_1': 'Monomer',
    'CH3OH_2': 'Dimer',
    'CH3OH_4': 'Tetramer'
}

# CO2, then NO, then CH3OH


fig, axs = plt.subplots(1,2,figsize=(6.69,3),dpi=600, constrained_layout=True,gridspec_kw={'width_ratios':[2,1.12]})

# X = np.arange(7)

X_list =  [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14]]
X_list_flat = [item for row in X_list for item in row]

# Nitric oxide systems
no_1_xc_list = [final_hads['MgO']['NO_1'][0]] + mgo_system_func_dependence['NO_1']
no_2_xc_list = [final_hads['MgO']['NO_2'][0]] + mgo_system_func_dependence['NO_2']
no_3_xc_list = [final_hads['MgO']['NO_3'][0]] + mgo_system_func_dependence['NO_3']
no_4_xc_list = [final_hads['MgO']['NO_4'][0]] + mgo_system_func_dependence['NO_4']
no_5_xc_list = [final_hads['MgO']['NO_5'][0]] + mgo_system_func_dependence['NO_5']
no_6_xc_list = [final_hads['MgO']['NO_6'][0]] + mgo_system_func_dependence['NO_6']

# Dotted line
# axs[0].plot(X_list_flat, sum([[x]*2 for x in no_1_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
# axs[0].plot(X_list_flat, sum([[x]*2 for x in no_2_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
# axs[0].plot(X_list_flat, sum([[x]*2 for x in no_3_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
# axs[0].plot(X_list_flat, sum([[x]*2 for x in no_4_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
# axs[0].plot(X_list_flat, sum([[x]*2 for x in no_5_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
# axs[0].plot(X_list_flat, sum([[x]*2 for x in no_6_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)

# Stacked lines

for index,i in enumerate(X_list):
    axs[0].plot(i, [no_1_xc_list[index]]*2,'-',color = color_dict['purple'])
    axs[0].plot(i, [no_2_xc_list[index]]*2,'-',color = color_dict['cyan'])
    axs[0].plot(i, [no_3_xc_list[index]]*2,'-',color = color_dict['blue'])
    axs[0].plot(i, [no_4_xc_list[index]]*2,'-',color = color_dict['green'])
    axs[0].plot(i, [no_5_xc_list[index]]*2,'-',color = color_dict['magenta'])
    axs[0].plot(i, [no_6_xc_list[index]]*2,'-',color = color_dict['yellow'])


axs[0].plot([-5, 40],[-210]*2,  '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
axs[0].fill_between([-5,40],-210 - mol_hads['MgO']['NO'][1], -210 + mol_hads['MgO']['NO'][1],color='tab:gray',edgecolor=None,alpha=0.2)

axs[0].set_xticks([1.5,3.5,5.5,7.5,9.5,11.5,13.5])

axs[0].tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)


axs[0].set_xticklabels([r'\textbf{SKZCAM}'] + [convert_to_nice_labels['xc_functionals'][x] for x in dft_functionals['MgO']],rotation=90,ha='center')

axs[1].axis('off')




axs[0].set_ylabel('Adsorption Enthalpy (meV)')

axs[0].set_ylim([-400,100])

axs[0].set_xlim([0.25,14.75])


plt.savefig('Fig_SI_NO_configurations_hads.png')

In [79]:
# We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.
adsorbate_config = {
    'NO_1': 'Vertical-Hollow',
    'NO_2': 'Vertical-Mg',
    'NO_3': 'Bent-Bridge',
    'NO_4': 'Bent-Mg',
    'NO_5': 'Bent-O',
    'NO_6': 'Dimer',
    'CO2_2': 'Physisorbed',
    'CO2_3': 'Chemisorbed',
    'CH3OH_1': 'Monomer',
    'CH3OH_2': 'Dimer',
    'CH3OH_4': 'Tetramer'
}

mgo_system_func_dependence = {x: [] for x in ['NO_1','NO_2','NO_3','NO_4','NO_5','NO_6','CO2_2','CO2_3','CH3OH_1','CH3OH_2','CH3OH_4']}

for index,i in enumerate(adsorbate_config):
    data_dummy = []

    for j in dft_functionals['MgO']:
        data_dummy += [final_eads_dict['MgO'][i][j] + ethermal['MgO'][i]['02_revPBE-D4']['Ecor']]
    
    mgo_system_func_dependence[i] = data_dummy

# CO2, then NO, then CH3OH

xc_list = ['01_PBE-D2-Ne','02_revPBE-D4','03_vdW-DF','04_rev-vdW-DF2','05_PBE0-D4','06_B3LYP-D2-Ne']
xc_list_nice = ['PBE-D2','revPBE-D4','vdW-DF','rev-vdW-DF2','PBE0-D4','B3LYP-D2']

fig, axs = plt.subplots(3,2,figsize=(6.69,6.5),dpi=1200, constrained_layout=True,gridspec_kw={'width_ratios':[2,1.12]})

# X = np.arange(7)

X_list =  [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14]]
X_list_flat = [item for row in X_list for item in row]


co2_2_xc_list = mgo_system_func_dependence['CO2_2'] + [final_hads['MgO']['CO2_2'][0]]
co2_3_xc_list = mgo_system_func_dependence['CO2_3'] + [final_hads['MgO']['CO2_3'][0]]

# Dotted line
axs[0,0].plot(X_list_flat, sum([[x]*2 for x in co2_2_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[0,0].plot(X_list_flat, sum([[x]*2 for x in co2_3_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)

# Stacked lines
for index,i in enumerate(X_list):
    axs[0,0].plot(i, [co2_2_xc_list[index]]*2,'-',color = color_dict['red'])
    axs[0,0].plot(i, [co2_3_xc_list[index]]*2,'-',color = color_dict['orange'])

# Nitric oxide systems


no_1_xc_list = mgo_system_func_dependence['NO_1'] + [final_hads['MgO']['NO_1'][0]]
no_2_xc_list = mgo_system_func_dependence['NO_2'] + [final_hads['MgO']['NO_2'][0]]
no_3_xc_list = mgo_system_func_dependence['NO_3'] + [final_hads['MgO']['NO_3'][0]]
no_4_xc_list = mgo_system_func_dependence['NO_4'] + [final_hads['MgO']['NO_4'][0]]
no_5_xc_list = mgo_system_func_dependence['NO_5'] + [final_hads['MgO']['NO_5'][0]]
no_6_xc_list = mgo_system_func_dependence['NO_6'] + [final_hads['MgO']['NO_6'][0]]

# Dotted line
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_1_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_2_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_3_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_4_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_5_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_6_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)

# Stacked lines

for index,i in enumerate(X_list):
    axs[1,0].plot(i, [no_1_xc_list[index]]*2,'-',color = color_dict['purple'])
    axs[1,0].plot(i, [no_2_xc_list[index]]*2,'-',color = color_dict['cyan'])
    axs[1,0].plot(i, [no_3_xc_list[index]]*2,'-',color = color_dict['blue'])
    axs[1,0].plot(i, [no_4_xc_list[index]]*2,'-',color = color_dict['green'])
    axs[1,0].plot(i, [no_5_xc_list[index]]*2,'-',color = color_dict['magenta'])
    axs[1,0].plot(i, [no_6_xc_list[index]]*2,'-',color = color_dict['yellow'])


# Methanol systems

ch3oh_1_xc_list = mgo_system_func_dependence['CH3OH_1'] + [final_hads['MgO']['CH3OH_1'][0]]
ch3oh_2_xc_list = mgo_system_func_dependence['CH3OH_2'] + [final_hads['MgO']['CH3OH_2'][0]]
ch3oh_4_xc_list = mgo_system_func_dependence['CH3OH_4'] + [final_hads['MgO']['CH3OH_4'][0]]

# Dotted line
axs[2,0].plot(X_list_flat, sum([[x]*2 for x in ch3oh_1_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[2,0].plot(X_list_flat, sum([[x]*2 for x in ch3oh_2_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[2,0].plot(X_list_flat, sum([[x]*2 for x in ch3oh_4_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)

# Stacked lines
for index,i in enumerate(X_list):
    axs[2,0].plot(i, [ch3oh_1_xc_list[index]]*2,'-',color = color_dict['maroon'])
    axs[2,0].plot(i, [ch3oh_2_xc_list[index]]*2,'-',color = color_dict['olive'])
    axs[2,0].plot(i, [ch3oh_4_xc_list[index]]*2,'-',color = color_dict['teal'])


axs[0,0].plot( [-5, 20],[mol_hads['MgO']['CO2_3'][0]]*2, '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
axs[0,0].fill_between([-5,40],mol_hads['MgO']['CO2_3'][0] - mol_hads['MgO']['CO2_3'][1], mol_hads['MgO']['CO2_3'][0] + mol_hads['MgO']['CO2_3'][1],color='tab:gray',edgecolor=None,alpha=0.2)

axs[1,0].plot([-5, 40],[-210]*2,  '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
axs[1,0].fill_between([-5,40],-210 - mol_hads['MgO']['NO'][1], -210 + mol_hads['MgO']['NO'][1],color='tab:gray',edgecolor=None,alpha=0.2)

axs[2,0].plot([-5, 40], [mol_hads['MgO']['CH3OH_4'][0]]*2,  '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
axs[2,0].fill_between([-5,40],mol_hads['MgO']['CH3OH_4'][0] - mol_hads['MgO']['CH3OH_4'][1], mol_hads['MgO']['CH3OH_4'][0] + mol_hads['MgO']['CH3OH_4'][1],color='tab:gray',edgecolor=None,alpha=0.2)

axs[0,0].set_xticks([1.5,3.5,5.5,7.5,9.5,11.5,13.5])
axs[1,0].set_xticks([1.5,3.5,5.5,7.5,9.5,11.5,13.5])
axs[2,0].set_xticks([1.5,3.5,5.5,7.5,9.5,11.5,13.5])

axs[0,0].tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
axs[1,0].tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
axs[2,0].tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)


axs[1,0].set_xticklabels([])
axs[2,0].set_xticklabels([])
axs[1,1].set_xticklabels([])
axs[2,1].set_xticklabels([])
axs[0,1].set_xticklabels([])

axs[0,1].axis('off')
axs[1,1].axis('off')
axs[2,1].axis('off')


axs[0,0].set_xticklabels(['SKZCAM'] + xc_list_nice,rotation=45,ha='left')


axs[0,0].set_ylabel('Adsorption Enthalpy (meV)')
axs[1,0].set_ylabel('Adsorption Enthalpy (meV)')
axs[2,0].set_ylabel('Adsorption Enthalpy (meV)')

axs[0,0].set_ylim([-800,-200])
axs[1,0].set_ylim([-400,0])
axs[2,0].set_ylim([-1100,-300])

axs[0,0].set_xlim([0.25,14.75])
axs[1,0].set_xlim([0.25,14.75])
axs[2,0].set_xlim([0.25,14.75])


plt.savefig('Fig_03_original.png')

In [49]:
data_dummy += [final_eads_dict['MgO'][i][j]]

KeyError: 'MgO'

In [52]:
j

'01_PBE-D2-Ne'

In [ ]:
# We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.

# CO2, then NO, then CH3OH

xc_list = ['01_PBE-D2-Ne','02_revPBE-D4','03_vdW-DF','04_rev-vdW-DF2','05_PBE0-D4','06_B3LYP-D2-Ne']
xc_list_nice = ['PBE-D2','revPBE-D4','vdW-DF','rev-vdW-DF2','PBE0-D4','B3LYP-D2']

fig, axs = plt.subplots(3,2,figsize=(6.69,6.5),dpi=1200, constrained_layout=True,gridspec_kw={'width_ratios':[2,1.12]})

# X = np.arange(7)

X_list =  [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14]]
X_list_flat = [item for row in X_list for item in row]


co2_2_xc_list = mgo_system_func_dependence['CO2_2'] + [final_hads['MgO']['CO2_2'][0]]
co2_3_xc_list = mgo_system_func_dependence['CO2_3'] + [final_hads['MgO']['CO2_3'][0]]

# Dotted line
axs[0,0].plot(X_list_flat, sum([[x]*2 for x in co2_2_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[0,0].plot(X_list_flat, sum([[x]*2 for x in co2_3_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)

# Stacked lines
for index,i in enumerate(X_list):
    axs[0,0].plot(i, [co2_2_xc_list[index]]*2,'-',color = color_dict['red'])
    axs[0,0].plot(i, [co2_3_xc_list[index]]*2,'-',color = color_dict['orange'])

# Nitric oxide systems


no_1_xc_list = mgo_system_func_dependence['NO_1'] + [final_hads['MgO']['NO_1'][0]]
no_2_xc_list = mgo_system_func_dependence['NO_2'] + [final_hads['MgO']['NO_2'][0]]
no_3_xc_list = mgo_system_func_dependence['NO_3'] + [final_hads['MgO']['NO_3'][0]]
no_4_xc_list = mgo_system_func_dependence['NO_4'] + [final_hads['MgO']['NO_4'][0]]
no_5_xc_list = mgo_system_func_dependence['NO_5'] + [final_hads['MgO']['NO_5'][0]]
no_6_xc_list = mgo_system_func_dependence['NO_6'] + [final_hads['MgO']['NO_6'][0]]

# Dotted line
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_1_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_2_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_3_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_4_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_5_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[1,0].plot(X_list_flat, sum([[x]*2 for x in no_6_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)

# Stacked lines

for index,i in enumerate(X_list):
    axs[1,0].plot(i, [no_1_xc_list[index]]*2,'-',color = color_dict['purple'])
    axs[1,0].plot(i, [no_2_xc_list[index]]*2,'-',color = color_dict['cyan'])
    axs[1,0].plot(i, [no_3_xc_list[index]]*2,'-',color = color_dict['blue'])
    axs[1,0].plot(i, [no_4_xc_list[index]]*2,'-',color = color_dict['green'])
    axs[1,0].plot(i, [no_5_xc_list[index]]*2,'-',color = color_dict['magenta'])
    axs[1,0].plot(i, [no_6_xc_list[index]]*2,'-',color = color_dict['yellow'])


# Methanol systems

ch3oh_1_xc_list = mgo_system_func_dependence['CH3OH_1'] + [final_hads['MgO']['CH3OH_1'][0]]
ch3oh_2_xc_list = mgo_system_func_dependence['CH3OH_2'] + [final_hads['MgO']['CH3OH_2'][0]]
ch3oh_4_xc_list = mgo_system_func_dependence['CH3OH_4'] + [final_hads['MgO']['CH3OH_4'][0]]

# Dotted line
axs[2,0].plot(X_list_flat, sum([[x]*2 for x in ch3oh_1_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[2,0].plot(X_list_flat, sum([[x]*2 for x in ch3oh_2_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)
axs[2,0].plot(X_list_flat, sum([[x]*2 for x in ch3oh_4_xc_list],[]),'--',color=color_dict['black'],linewidth=0.7)

# Stacked lines
for index,i in enumerate(X_list):
    axs[2,0].plot(i, [ch3oh_1_xc_list[index]]*2,'-',color = color_dict['maroon'])
    axs[2,0].plot(i, [ch3oh_2_xc_list[index]]*2,'-',color = color_dict['olive'])
    axs[2,0].plot(i, [ch3oh_4_xc_list[index]]*2,'-',color = color_dict['teal'])


axs[0,0].plot( [-5, 20],[mol_hads['MgO']['CO2_3'][0]]*2, '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
axs[0,0].fill_between([-5,40],mol_hads['MgO']['CO2_3'][0] - mol_hads['MgO']['CO2_3'][1], mol_hads['MgO']['CO2_3'][0] + mol_hads['MgO']['CO2_3'][1],color='tab:gray',edgecolor=None,alpha=0.2)

axs[1,0].plot([-5, 40],[-210]*2,  '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
axs[1,0].fill_between([-5,40],-210 - mol_hads['MgO']['NO'][1], -210 + mol_hads['MgO']['NO'][1],color='tab:gray',edgecolor=None,alpha=0.2)

axs[2,0].plot([-5, 40], [mol_hads['MgO']['CH3OH_4'][0]]*2,  '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
axs[2,0].fill_between([-5,40],mol_hads['MgO']['CH3OH_4'][0] - mol_hads['MgO']['CH3OH_4'][1], mol_hads['MgO']['CH3OH_4'][0] + mol_hads['MgO']['CH3OH_4'][1],color='tab:gray',edgecolor=None,alpha=0.2)

axs[0,0].set_xticks([1.5,3.5,5.5,7.5,9.5,11.5,13.5])
axs[1,0].set_xticks([1.5,3.5,5.5,7.5,9.5,11.5,13.5])
axs[2,0].set_xticks([1.5,3.5,5.5,7.5,9.5,11.5,13.5])

axs[0,0].tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
axs[1,0].tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
axs[2,0].tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)


axs[1,0].set_xticklabels([])
axs[2,0].set_xticklabels([])
axs[1,1].set_xticklabels([])
axs[2,1].set_xticklabels([])
axs[0,1].set_xticklabels([])

axs[0,1].axis('off')
axs[1,1].axis('off')
axs[2,1].axis('off')


axs[0,0].set_xticklabels(['SKZCAM'] + xc_list_nice,rotation=45,ha='left')


axs[0,0].set_ylabel('Adsorption Enthalpy (meV)')
axs[1,0].set_ylabel('Adsorption Enthalpy (meV)')
axs[2,0].set_ylabel('Adsorption Enthalpy (meV)')

axs[0,0].set_ylim([-800,-200])
axs[1,0].set_ylim([-400,0])
axs[2,0].set_ylim([-1100,-300])

axs[0,0].set_xlim([0.25,14.75])
axs[1,0].set_xlim([0.25,14.75])
axs[2,0].set_xlim([0.25,14.75])


plt.savefig('Fig_04.png')

In [ ]:
# # We need to make a three-way figure to highlight the variations in the adsorption energy for the various systems.

# # CO2, then NO, then CH3OH

# xc_list = ['01_PBE-D2-Ne','02_revPBE-D4','03_vdW-DF','04_rev-vdW-DF2','05_PBE0-D4','06_B3LYP-D2-Ne']
# xc_list_nice = ['PBE-D2','revPBE-D4','vdW-DF','rev-vdW-DF2','PBE0-D4','B3LYP-D2']

# fig, axs = plt.subplots(3,1,figsize=(3.345,6.5),dpi=600, sharey=True, constrained_layout=True)

# X = np.arange(7)

# axs[0].barh(X -0.2, mgo_system_func_dependence['CO2_2'] + [-400] , xerr = [0.0]*6 + [final_hads['MgO']['CO2_2'][1]], color = color_dict['blue'], height= 0.4, label='Physisorbed', ecolor='black', capsize=2)
# axs[0].barh(X + 0.2, mgo_system_func_dependence['CO2_3'] + [final_hads['MgO']['CO2_3'][0]], xerr = [0.0]*6 + [final_hads['MgO']['CO2_3'][1]/3], height= 0.4, color = color_dict['red'], label='Chemisorbed', ecolor='black', capsize=2)
# axs[0].plot([mol_hads['MgO']['CO2_3'][0]]*2, [-5, 20], '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
# # axs[0].fill_between([-5,40],[-615+100, -615+100],[-615-100,-615-100],color='tab:gray',edgecolor=None,alpha=0.2)
# axs[0].fill_betweenx([-5,40],mol_hads['MgO']['CO2_3'][0] - mol_hads['MgO']['CO2_3'][1], mol_hads['MgO']['CO2_3'][0] + mol_hads['MgO']['CO2_3'][1],[mol_hads['MgO']['CO2_3'][0] - mol_hads['MgO']['CO2_3'][1],mol_hads['MgO']['CO2_3'][0] + mol_hads['MgO']['CO2_3'][1]],color='tab:gray',edgecolor=None,alpha=0.2)


# axs[0].set_ylim([-0.5,6.5])
# axs[0].set_xlim([-800,0])

# X = np.arange(7)

# axs[1].barh(X -0.4 - 0.8/12 + 0.8/6, mgo_system_func_dependence['NO_1'] + [100], color = color_dict['blue'], height = 0.8/6,label='1 - ' + adsorbate_config['NO_1'])
# axs[1].barh(X -0.4 - 0.8/12 + 2*0.8/6, mgo_system_func_dependence['NO_2'] + [100], color = color_dict['red'], height = 0.8/6,label='2 - ' + adsorbate_config['NO_2'])
# axs[1].barh(X -0.4 - 0.8/12 + 3*0.8/6, mgo_system_func_dependence['NO_3']  + [100], color = color_dict['green'], height = 0.8/6,label='3 - ' +adsorbate_config['NO_3'])
# axs[1].barh(X -0.4 - 0.8/12 + 4*0.8/6, mgo_system_func_dependence['NO_4']  + [100], color = color_dict['orange'], height = 0.8/6,label='4 - ' +adsorbate_config['NO_4'])
# axs[1].barh(X -0.4 - 0.8/12 + 5*0.8/6, mgo_system_func_dependence['NO_5']  + [100], color = color_dict['yellow'], height = 0.8/6,label='5 - ' +adsorbate_config['NO_5'])
# axs[1].barh(X -0.4 - 0.8/12 + 6*0.8/6, mgo_system_func_dependence['NO_6']  + [final_hads['MgO']['NO_6'][0]], xerr = [0.0]*6 + [15], color = color_dict['cyan'], height = 0.8/6,label='6 - ' +adsorbate_config['NO_6'])
# # ax[1].bar(30+1.35,[-240] ,yerr=[8],color = color_dict['cyan'], width = 0.4, ecolor='black', capsize=2)


# axs[1].plot([-210]*2,[-5, 40],  '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
# # axs[1].fill_between([-5,40],[-250+30, -250+30],[-250-30,-250-30],color='tab:gray',edgecolor=None,alpha=0.2)

# axs[1].fill_betweenx([-5,40],[-210 - mol_hads['MgO']['NO'][1], -210 + mol_hads['MgO']['NO'][1]], [-210 - mol_hads['MgO']['NO'][1],-210 + mol_hads['MgO']['NO'][1]],color='tab:gray',edgecolor=None,alpha=0.2)

# X = np.arange(7)

# axs[2].barh(X - 0.4 - 0.8/6 + 0.8/3, mgo_system_func_dependence['CH3OH_1'] + [-700], xerr = [0]*6 + [final_hads['MgO']['CH3OH_1'][1]], color = color_dict['red'], height = 0.8/3,label='Monomer')
# axs[2].barh(X - 0.4 - 0.8/6 + 2*0.8/3, mgo_system_func_dependence['CH3OH_2']  + [final_hads['MgO']['CH3OH_2'][0]], xerr = [0]*6 + [final_hads['MgO']['CH3OH_2'][1]], color = color_dict['green'], height = 0.8/3,label='Dimer')
# axs[2].barh(X - 0.4 - 0.8/6 + 3*0.8/3, mgo_system_func_dependence['CH3OH_4']  + [final_hads['MgO']['CH3OH_4'][0]], xerr = [0]*6 + [final_hads['MgO']['CH3OH_4'][1]], color = color_dict['orange'], height = 0.8/3,label='Cluster')
# # ax[1].bar(30+1.35,[-240] ,yerr=[8],color = color_dict['cyan'], width = 0.4, ecolor='black', capsize=2)


# axs[2].plot([mol_hads['MgO']['CH3OH_4'][0]]*2,[-5, 40],  '-',color = color_dict['black'], linewidth = 1,label=r'Expt $H_\textrm{ads}$')
# axs[2].fill_betweenx([-5,40],mol_hads['MgO']['CH3OH_4'][0] - mol_hads['MgO']['CH3OH_4'][1], mol_hads['MgO']['CH3OH_4'][0] + mol_hads['MgO']['CH3OH_4'][1],[mol_hads['MgO']['CH3OH_4'][0] - mol_hads['MgO']['CH3OH_4'][1],mol_hads['MgO']['CH3OH_4'][0] + mol_hads['MgO']['CH3OH_4'][1]],color='tab:gray',edgecolor=None,alpha=0.2)


# axs[1].set_xlim([-4000,0])
# legend = axs[1].legend(frameon=False,fontsize=6,loc=3, framealpha=1)

# export_legend(legend)
# axs[1].set_xlim([-400,0])
# legend.remove()

# axs[0].yaxis.set_label_position("right")
# axs[0].yaxis.tick_right()
# axs[0].set_yticks(X)
# axs[0].set_yticklabels(xc_list_nice + ['SKZCAM'])

# axs[1].yaxis.set_label_position("right")
# axs[1].yaxis.tick_right()
# axs[1].set_yticks(X)
# axs[1].set_yticklabels(xc_list_nice + ['SKZCAM'])

# axs[2].yaxis.set_label_position("right")
# axs[2].yaxis.tick_right()
# axs[2].set_yticks(X)
# axs[2].set_yticklabels(xc_list_nice + ['SKZCAM'])

# # axs[2].set_xlabel(r'Adsorption Enthalpy, $H_\textrm{ads}$ (meV)')


# plt.savefig('Fig_04.png')

In [ ]:
exp_ene_list = {
    'CH4' : mol_hads['MgO']['CH4'],
    'C2H6': mol_hads['MgO']['C2H6'],
    'CO': mol_hads['MgO']['CO'],
    'N2O': mol_hads['MgO']['N2O'],
    'C6H6': mol_hads['MgO']['C6H6'],
    'H2O': mol_hads['MgO']['H2O'],
    'NH3': mol_hads['MgO']['NH3'],
    'CO2': mol_hads['MgO']['CO2_3'], 
    'NO': mol_hads['MgO']['NO'],
    'H2O_clus': mol_hads['MgO']['H2O_4'] ,
    'CH3OH_clus': mol_hads['MgO']['CH3OH_4'],
    'CH4_ml': mol_hads['MgO']['CH4_ML'],
    'C2H6_ml': mol_hads['MgO']['C2H6_ML'] 
}

skzcam_ene_list = {
    'CH4' : final_hads['MgO']['CH4'],
    'C2H6': final_hads['MgO']['C2H6'],
    'CO': final_hads['MgO']['CO'],
    'N2O': final_hads['MgO']['N2O_1'],
    'C6H6': final_hads['MgO']['C6H6'],
    'H2O': final_hads['MgO']['H2O'],
    'NH3': final_hads['MgO']['NH3'],
    'CO2': final_hads['MgO']['CO2_3'], 
    'NO': final_hads['MgO']['NO_6'],
    'H2O_clus': final_hads['MgO']['H2O_4'] ,
    'CH3OH_clus': final_hads['MgO']['CH3OH_4'],
    'CH4_ml': final_hads['MgO']['CH4_ML'],
    'C2H6_ml': final_hads['MgO']['C2H6_ML'] 
}

rpa_ene_list = {
    'CH4' : -97 + 17,
    'C2H6': -140 + 21,
    'CO': -92 + 24,
    'N2O': -207 + 1,
    'C6H6': 200,
    'H2O': -479 + 65,
    'NH3': -532 + 60,
    'CO2': -492 + 50, 
    'NO': 200,
    'H2O_clus':  200,
    'CH3OH_clus': 200,
    'CH4_ml': -507/4 + 17,
    'C2H6_ml': -788/4 + 21
}

rpa_rse_ene_list = {
    'CH4' : -141.4288291 + 17,
    'C2H6': -203.5148896 + 21,
    'CO': -289.00248 + 24,
    'N2O': -271.039442 + 1,
    'C6H6': 200,
    'H2O': -565.6875413 + 65,
    'NH3': -605.1305917 + 60,
    'CO2': -494.0607615 + 50, 
    'NO': 200,
    'H2O_clus':  200,
    'CH3OH_clus': 200,
    'CH4_ml': -697.7719902/4 + 17,
    'C2H6_ml': -1107.095375/4 + 21
}

fig, axs = plt.subplots(figsize=(6,3),dpi=300, constrained_layout=True)

ax2 = axs.twinx()

def convert_ax2(axs):
    y1, y2 = axs.get_ylim()
    ax2.set_ylim(y1/43.3634, y2/43.3634)
    ax2.figure.canvas.draw()    

axs.callbacks.connect("ylim_changed", convert_ax2)  

axs.errorbar([1,2,3,4,5,6,7,8], [skzcam_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [skzcam_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5,label='SKZCAM')

axs.errorbar([1,2,3,4,5,6,7,8], [exp_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [exp_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'],label='Experiment')

axs.errorbar([1,2,3,4,5,6,7,8], [rpa_ene_list[x] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="x",alpha=0.5,color=color_dict['orange'],label='RPA')

axs.errorbar([1,2,3,4,5,6,7,8], [rpa_rse_ene_list[x] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="s",alpha=0.5,color=color_dict['orange'],markerfacecolor='None',label='RPA + rSE')

# axs.errorbar([1,2,3,4,5,6,7,8,10,11,12,14,15],-np.array([-100,-100,140,230,520,-100,581,536,220,653,757,-100,-100]),[0]*13,fmt="x",alpha=0.5,color=color_dict['green'],label=r'Experiment - $\nu=13$')


axs.errorbar([10,11,12], [skzcam_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [skzcam_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5)

axs.errorbar([10,11,12], [exp_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [exp_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'])

axs.errorbar([14,15], [skzcam_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [skzcam_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5)

axs.errorbar([14,15], [exp_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [exp_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'])

axs.errorbar([14,15], [rpa_ene_list[x] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="x",alpha=0.5,color=color_dict['orange'])

axs.errorbar([14,15], [rpa_rse_ene_list[x] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="s",alpha=0.5,color=color_dict['orange'],markerfacecolor='None')

# axs.errorbar([1,2,3,4,5,6,7,8,9,10], my_ene_list,my_ene_list_err,color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5,label='SKZCAM')

# axs.errorbar([1,2,3,4,5,6,7,8,9,10], [-x for x in exp_ene_list],exp_ene_list_err,color=color_dict['red'],capsize=2,alpha=0.5,fmt="o",label='Experiment')

axs.set_xticks([1,2,3,4,5,6,7,8,10,11,12,14,15])
axs.set_xticklabels([r'CH\textsubscript{4}', r'C\textsubscript{2}H\textsubscript{6}','CO',r'N\textsubscript{2}O', r'C\textsubscript{6}H\textsubscript{6}',  r'H\textsubscript{2}O', r'NH\textsubscript{3}', r'CO\textsubscript{2}'] + ['NO',r'H\textsubscript{2}O',r'CH\textsubscript{3}OH'] + [r'CH\textsubscript{4}',r'C\textsubscript{2}H\textsubscript{6}']) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (meV)')
ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
axs.set_xlabel(r'Molecule on MgO (001)')
axs.set_ylim([-0,-1000])
axs.legend(frameon=False,loc='upper left')


plt.savefig('Combined_estimates_MgO.png')


In [ ]:
# Let's plot out the old experiment graph

exp_ene_list = {
    'CH4' : [113,16],
    'C2H6': [218,26],
    'CO': [182,21],
    'N2O': [266,33],
    'C6H6': [530,60],
    'H2O': [520,122],
    'NH3': [650,82],
    'CO2': [615,100], 
    'NO': [250,29],
    'H2O_clus': [704,81] ,
    'CH3OH_clus': [852,109],
    'CH4_ml': [127,16],
    'C2H6_ml': [233,26] 
}

skzcam_ene_list = {
    'CH4' : [116,10],
    'C2H6': [190,27],
    'CO': [182,10],
    'N2O': [266,13],
    'C6H6': [456,40],
    'H2O': [529,30],
    'NH3': [600,40],
    'CO2': [680,70], 
    'NO': [240,8],
    'H2O_clus': [662.8,40] ,
    'CH3OH_clus': [741,35],
    'CH4_ml': [140,9],
    'C2H6_ml': [215,14] 
}

rpa_ene_list = {
    'CH4' : -97 + 17,
    'C2H6': -140 + 21,
    'CO': -92 + 24,
    'N2O': -207 + 1,
    'C6H6': 200,
    'H2O': -479 + 65,
    'NH3': -532 + 60,
    'CO2': -492 + 50, 
    'NO': 200,
    'H2O_clus':  200,
    'CH3OH_clus': 200,
    'CH4_ml': -507/4 + 17,
    'C2H6_ml': -788/4 + 21
}

rpa_rse_ene_list = {
    'CH4' : -141.4288291 + 17,
    'C2H6': -203.5148896 + 21,
    'CO': -289.00248 + 24,
    'N2O': -271.039442 + 1,
    'C6H6': 200,
    'H2O': -565.6875413 + 65,
    'NH3': -605.1305917 + 60,
    'CO2': -494.0607615 + 50, 
    'NO': 200,
    'H2O_clus':  200,
    'CH3OH_clus': 200,
    'CH4_ml': -697.7719902/4 + 17,
    'C2H6_ml': -1107.095375/4 + 21
}

fig, axs = plt.subplots(figsize=(6,3),dpi=300, constrained_layout=True)

ax2 = axs.twinx()

def convert_ax2(axs):
    y1, y2 = axs.get_ylim()
    ax2.set_ylim(y1/43.3634, y2/43.3634)
    ax2.figure.canvas.draw()    

axs.callbacks.connect("ylim_changed", convert_ax2)  

axs.errorbar([1,2,3,4,5,6,7,8], [-skzcam_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [skzcam_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5,label='SKZCAM')

axs.errorbar([1,2,3,4,5,6,7,8], [-exp_ene_list[x][0] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [exp_ene_list[x][1] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'],label='Experiment')

axs.errorbar([1,2,3,4,5,6,7,8], [rpa_ene_list[x] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="x",alpha=0.5,color=color_dict['orange'],label='RPA')

axs.errorbar([1,2,3,4,5,6,7,8], [rpa_rse_ene_list[x] for x in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="s",alpha=0.5,color=color_dict['orange'],markerfacecolor='None',label='RPA + rSE')

# axs.errorbar([1,2,3,4,5,6,7,8,10,11,12,14,15],-np.array([-100,-100,140,230,520,-100,581,536,220,653,757,-100,-100]),[0]*13,fmt="x",alpha=0.5,color=color_dict['green'],label=r'Experiment - $\nu=13$')


axs.errorbar([10,11,12], [-skzcam_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [skzcam_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5)

axs.errorbar([10,11,12], [-exp_ene_list[x][0] for x in ['NO','H2O_clus','CH3OH_clus']],   [exp_ene_list[x][1] for x in ['NO','H2O_clus','CH3OH_clus']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'])

axs.errorbar([14,15], [-skzcam_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [skzcam_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5)

axs.errorbar([14,15], [-exp_ene_list[x][0] for x in ['CH4_ml','C2H6_ml']],   [exp_ene_list[x][1] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'])

axs.errorbar([14,15], [rpa_ene_list[x] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="x",alpha=0.5,color=color_dict['orange'])

axs.errorbar([14,15], [rpa_rse_ene_list[x] for x in ['CH4_ml','C2H6_ml']],capsize=2,fmt="s",alpha=0.5,color=color_dict['orange'],markerfacecolor='None')

# axs.errorbar([1,2,3,4,5,6,7,8,9,10], my_ene_list,my_ene_list_err,color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5,label='SKZCAM')

# axs.errorbar([1,2,3,4,5,6,7,8,9,10], [-x for x in exp_ene_list],exp_ene_list_err,color=color_dict['red'],capsize=2,alpha=0.5,fmt="o",label='Experiment')

axs.set_xticks([1,2,3,4,5,6,7,8,10,11,12,14,15])
axs.set_xticklabels([r'CH\textsubscript{4}', r'C\textsubscript{2}H\textsubscript{6}','CO',r'N\textsubscript{2}O', r'C\textsubscript{6}H\textsubscript{6}',  r'H\textsubscript{2}O', r'NH\textsubscript{3}', r'CO\textsubscript{2}'] + ['NO',r'H\textsubscript{2}O',r'CH\textsubscript{3}OH'] + [r'CH\textsubscript{4}',r'C\textsubscript{2}H\textsubscript{6}']) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (meV)')
ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
axs.set_xlabel(r'Molecule on MgO (001)')
axs.set_ylim([-0,-1000])
axs.legend(frameon=False,loc='upper left')


plt.savefig('Combined_estimates_MgO.png')

In [ ]:
final_hads

{'MgO': {'CH4': [-157.87656496046833, 5.928783485568514],
  'C2H6': [-228.84485862614005, 24.694723978358198],
  'CO': [-237.13074304657462, 6.93618100881116],
  'N2O_1': [-251.18448984140184, 17.894404143204653],
  'N2O_2': [-181.44869431257783, 4.411474180855896],
  'C6H6': [-469.04201707810023, 13.140064848732923],
  'H2O': [-685.1919421930257, 20.860531780666193],
  'NH3': [-715.1751573176407, 94.49630898450357],
  'CO2_2': [-279.0033844540683, 7.586294499160735],
  'CO2_3': [-1007.7029411466973, 47.636360975592595],
  'NO_1': [90151328.09363137, 70550737.04697864],
  'NO_2': [87664806.47742979, 99666236.87613809],
  'NO_3': [87665601.24580844, 99665369.25149527],
  'NO_4': [87662878.72322588, 99665171.75391565],
  'NO_5': [119914779.678251, 788232187.1546577]},
 'TiO2_rutile': {'CH4': [-254.1982414641585, 8.698531765044903],
  'CH3OH': [-1388.6990318975097, 33.51911612199289],
  'CO2_1': [-363.91538115172875, 21.03361932701504],
  'CO2_2': [-436.92782347908724, 17.310069980063233]

In [ ]:
# Let's plot out the old experiment graph

exp_ene_list = {
    'rutile_CH4': mol_hads['TiO2_rutile']['CH4'][:2],
    'rutile_CH3OH': mol_hads['TiO2_rutile']['CH3OH'][:2],
    'rutile_CO2': mol_hads['TiO2_rutile']['CO2'][:2],
    'rutile_H2O': mol_hads['TiO2_rutile']['H2O'][:2],
    'anatase_H2O': mol_hads['TiO2_anatase']['H2O'][:2],
    'anatase_NH3': mol_hads['TiO2_anatase']['NH3'][:2]
}

skzcam_ene_list = {
    'rutile_CH4': final_hads['TiO2_rutile']['CH4'],
    'rutile_CH3OH': final_hads['TiO2_rutile']['CH3OH'],
    'rutile_CO2': final_hads['TiO2_rutile']['CO2_2'],
    'rutile_H2O': final_hads['TiO2_rutile']['H2O'],
    'anatase_H2O': final_hads['TiO2_anatase']['H2O'],
    'anatase_NH3': final_hads['TiO2_anatase']['NH3']
}

print(exp_ene_list)

print(skzcam_ene_list)

{'rutile_CH4': [-255.444373974472, 26.955009294815437], 'rutile_CH3OH': [-1212.619540241254, 108.29571381387613], 'rutile_CO2': [-500.59422045856894, 56.12984288449803], 'rutile_H2O': [-959.7313626182915, 96.08668019210681], 'anatase_H2O': [-797.01233029532, 81.49926339726551], 'anatase_NH3': [-1183.3070020005914, 114.16239230745362]}
{'rutile_CH4': [-249.71005589084808, 21.7178079424401], 'rutile_CH3OH': [-1337.521359022114, 80.04684844615211], 'rutile_CO2': [-458.7815868215389, 37.72637322172359], 'rutile_H2O': [-1051.1597164266104, 74.24922743014392], 'anatase_H2O': [-872.4184974614515, 80.46648126218727], 'anatase_NH3': [-1141.3636130606476, 117.36005813061585]}


In [ ]:
fig, axs = plt.subplots(figsize=(3.25,3),dpi=300, constrained_layout=True)

ax2 = axs.twinx()

def convert_ax2(axs):
    y1, y2 = axs.get_ylim()
    ax2.set_ylim(y1/43.3634, y2/43.3634)
    ax2.figure.canvas.draw()    

axs.callbacks.connect("ylim_changed", convert_ax2)  

axs.errorbar([1,2,3,4], [skzcam_ene_list[x][0] for x in ['rutile_CH4', 'rutile_CH3OH','rutile_CO2','rutile_H2O']],   [skzcam_ene_list[x][1] for x in ['rutile_CH4', 'rutile_CH3OH','rutile_CO2','rutile_H2O']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5,label='SKZCAM')

axs.errorbar([1,2,3,4], [exp_ene_list[x][0] for x in ['rutile_CH4', 'rutile_CH3OH','rutile_CO2','rutile_H2O']],   [exp_ene_list[x][1] for x in ['rutile_CH4', 'rutile_CH3OH','rutile_CO2','rutile_H2O']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'],label='Experiment')


axs.errorbar([6,7], [skzcam_ene_list[x][0] for x in ['anatase_H2O','anatase_NH3']],   [skzcam_ene_list[x][1] for x in ['anatase_H2O','anatase_NH3']],color=color_dict['blue'],capsize=2,fmt="o",alpha=0.5)

axs.errorbar([6,7], [exp_ene_list[x][0] for x in ['anatase_H2O','anatase_NH3']],   [exp_ene_list[x][1] for x in ['anatase_H2O','anatase_NH3']],capsize=2,fmt="o",alpha=0.5,color=color_dict['red'])



axs.set_xticks([1,2,3,4,6,7])
axs.set_xticklabels([r'rutile CH\textsubscript{4}', r'rutile CH\textsubscript{3}OH', r'rutile CO\textsubscript{2}', r'rutile H\textsubscript{2}O'] + [r'anatase H\textsubscript{2}O', r'anatase NH\textsubscript{3}'],rotation = 90) #[r'H\textsubscript{2}O', r'NO',r'CO',r'C\textsubscript{2}H\textsubscript{6}',r'CH\textsubscript{4}'])

axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (meV)')
ax2.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ (kcal/mol)')
axs.set_xlabel(r'Molecule on TiO$_2$ ')
axs.set_ylim([-0,-2000])
axs.legend(frameon=False,loc='upper left')


plt.savefig('Combined_estimates_TiO2.png')

In [ ]:
# Let's plot out the TiO2 experimental results 

In [ ]:
mol_hads

{'MgO': {'CO': [-178.78365016377464,
   19.344183140985198,
   'Campbell and Sellers (C&S) Hads with nu = 13.85'],
  'H2O': [-520.286332650521, 121.26195402412539, 'Sauer estimate at 100K'],
  'CH3OH': [-937.9663965113973, 31.711775640959335, 'C&S Hads with nu = 15.3'],
  'CO2_2': [-427.78712965466457,
   41.22530833324714,
   'Eact Meixner. Convert nu = 13 to nu = 14'],
  'CO2_3': [-654.4226412063603,
   91.21600814065803,
   'Eact half of alpha and beta peaks. Convert nu = 13 to nu = 14'],
  'NO': [-249.7788967505489, 26.637891538405846, 'C&S Hads with nu = 14.0'],
  'N2O2': [-249.7788967505489, 26.637891538405846, 'C&S Hads with nu = 14.0'],
  'CH4': [-113.01831965005242,
   14.904534551250892,
   'Dohnalek original E0 with nu=13.1'],
  'C2H6': [-217.52744306490007,
   23.783831730719506,
   'Dohnalek original E0 with nu=14.9'],
  'CH4_ML': [-128.87565209936113,
   14.904534551250892,
   'Dohnalek original E0 with nu=13.1'],
  'C2H6_ML': [-232.65927664397896,
   23.783831730719506,
